<a href="https://colab.research.google.com/github/Renlim61/MVP_Product001_2025_Tier120pbc/blob/v18_EPICs/phase1_rag_mvp_v18_EPIC5_2_COMPLETE_FIXED_v11_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Production Readiness Track
##phase1_rag_mvp_v18_EPIC5.2.ipynb
## this will be the begining of v18 and its roadmap EPICs
## ## product roadmap.

## CELL 0 / STEP 0 – Install & Imports

In [1]:
# Commented out IPython magic to ensure Python compatibility.

# Commented out IPython magic to ensure Python compatibility.
# CELL 0 / STEP 0 – Install & Imports

# ============================================================

# Run this once at the top of the notebook (Colab style).

#=============================================================
%pip install -q faiss-cpu openai gradio PyPDF2 python-docx

import os
import io
import sys
import time
import json
import shutil
import pickle
import sqlite3
from uuid import uuid4
from datetime import datetime, timedelta, timezone
from typing import List, Dict, Any, Tuple, Optional

import numpy as np
import faiss
import gradio as gr

from PyPDF2 import PdfReader
from docx import Document as DocxDocument

# Global datetime import for the whole notebook
from datetime import datetime, timezone, timedelta

# Colab detection
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive as colab_drive

# OpenAI client (v1 library)
from openai import OpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.0 MB/s eta 0:00:00


## CELL 1 — Execution flags & persistence-safe paths (Colab/Drive)

In [2]:
# CELL 1 — Execution flags & persistence-safe paths (Colab/Drive)

#============================================================
# If True, we only run the DB pre-flight checks and stop BEFORE launching Gradio.
RUN_PREFLIGHT_ONLY = False

import os

# ============================================================

# PERSISTENCE CONFIG (Colab-safe)

# ============================================================

# Colab runtime storage under /content is ephemeral. Persist DB + indexes to Drive.
USE_DRIVE_PERSISTENCE = True          # Set False only if running locally
DRIVE_SUBFOLDER = "rag_mvp"           # Folder under MyDrive

# If IN_COLAB is not defined for any reason, define it safely.
try:
    IN_COLAB
except NameError:
    try:
        import google.colab  # type: ignore
        IN_COLAB = True
    except Exception:
        IN_COLAB = False

if IN_COLAB and USE_DRIVE_PERSISTENCE:
    try:
        from google.colab import drive as colab_drive  # type: ignore
        colab_drive.mount("/content/drive", force_remount=False)
    except Exception as e:
        print("⚠️ Drive mount issue (continuing):", e)

    BASE_DIR = f"/content/drive/MyDrive/{DRIVE_SUBFOLDER}"
else:
    BASE_DIR = os.path.join(os.getcwd(), DRIVE_SUBFOLDER)

os.makedirs(BASE_DIR, exist_ok=True)

# Persistent DB paths
# - v17 and earlier use: rag_documents.db
# - v18 uses: rag_documents_v18.db (side-by-side schema evolution)
DB_V17_PATH = os.path.join(BASE_DIR, "rag_documents.db")
DB_V18_PATH = os.path.join(BASE_DIR, "rag_documents_v18.db")

# STEP 1 (v18) — Initialize v18 DB once by cloning v17 DB (if present).
# This preserves backward compatibility for v17 and prior notebooks.
import shutil
if not os.path.exists(DB_V18_PATH):
    if os.path.exists(DB_V17_PATH):
        shutil.copy2(DB_V17_PATH, DB_V18_PATH)
        print(f"[v18] Initialized DB by cloning v17 DB: {DB_V17_PATH} -> {DB_V18_PATH}")
    else:
        # No prior DB to clone; v18 will create tables on first use
        print(f"[v18] No v17 DB found at {DB_V17_PATH}. Starting with a fresh v18 DB at {DB_V18_PATH}.")
else:
    print(f"[v18] Using existing v18 DB: {DB_V18_PATH}")

# Active DB path for this notebook/version
DB_PATH = DB_V18_PATH# Persistent index/log directories
INDEX_DIR = os.path.join(BASE_DIR, "indexes")
os.makedirs(INDEX_DIR, exist_ok=True)

TRACE_LOG_PATH = os.path.join(BASE_DIR, "debug_trace_v16.log")

print("PERSISTENCE CONFIG")
print("IN_COLAB:", IN_COLAB)
print("BASE_DIR:", BASE_DIR)
print("DB_PATH :", DB_PATH)
print("INDEX_DIR:", INDEX_DIR)
print("TRACE_LOG_PATH:", TRACE_LOG_PATH)

Mounted at /content/drive
[v18] Using existing v18 DB: /content/drive/MyDrive/rag_mvp/rag_documents_v18.db
PERSISTENCE CONFIG
IN_COLAB: True
BASE_DIR: /content/drive/MyDrive/rag_mvp
DB_PATH : /content/drive/MyDrive/rag_mvp/rag_documents_v18.db
INDEX_DIR: /content/drive/MyDrive/rag_mvp/indexes
TRACE_LOG_PATH: /content/drive/MyDrive/rag_mvp/debug_trace_v16.log


## CELL 1.75 — Global Configuration Constants

In [3]:
# CELL 1.75 — Global Configuration Constants

# ============================================================


# -------- Chunking --------
CHUNK_SIZE = 800          # tokens/characters per chunk (adjust as needed)
CHUNK_OVERLAP = 100       # overlap between chunks

# -------- Embeddings / Index --------
EMBEDDING_DIM = 1536      # OpenAI text-embedding-3-small
DEFAULT_EMBED_MODEL = "text-embedding-3-small"

# -------- Retrieval --------
TOP_K = 5
# -------- Session Memory (EPIC 3.1) --------
SESSION_MEM_MAX_TURNS = 3  # last N Q/A pairs per session (per cohort)

# -------- Limits --------
MAX_FILES_PER_UPLOAD = 20
MAX_FILE_SIZE_MB = 25

print("CONFIG CONSTANTS LOADED")
print("CHUNK_SIZE =", CHUNK_SIZE)
print("CHUNK_OVERLAP =", CHUNK_OVERLAP)
print("DEFAULT_EMBED_MODEL =", DEFAULT_EMBED_MODEL)

# ============================================================

CONFIG CONSTANTS LOADED
CHUNK_SIZE = 800
CHUNK_OVERLAP = 100
DEFAULT_EMBED_MODEL = text-embedding-3-small


## CELL 1.9 — OpenAI Client Factory

In [4]:
# CELL 1.9 — OpenAI Client Factory

# ============================================================


import os

def build_openai_client(api_key: str):
    """
    Returns an OpenAI client using the new SDK interface.
    Centralized so ingestion and chat use the same construction.
    """
    if not api_key:
        raise ValueError("OpenAI API key is required.")

    try:
        from openai import OpenAI
    except ImportError as e:
        raise RuntimeError("openai package not installed.") from e

    return OpenAI(api_key=api_key)

print("✅ build_openai_client() is defined.")

# ============================================================

✅ build_openai_client() is defined.


## CELL 1.95 — Model Defaults + Resolver

In [5]:
# CELL 1.95 — Model Defaults + Resolver

# ============================================================


# Defaults (used across embed + chat if user does not override)
EMBED_MODEL_DEFAULT = "text-embedding-3-small"
CHAT_MODEL_DEFAULT  = "gpt-4.1-mini"

def resolve_models(chat_model: str | None, embed_model: str | None) -> tuple[str, str]:
    """
    Normalize selected models and apply defaults.
    Returns (resolved_chat_model, resolved_embed_model).
    """
    resolved_chat = (chat_model or "").strip() or CHAT_MODEL_DEFAULT
    resolved_embed = (embed_model or "").strip() or EMBED_MODEL_DEFAULT
    return resolved_chat, resolved_embed

print("✅ resolve_models() is defined.")
print("   CHAT_MODEL_DEFAULT =", CHAT_MODEL_DEFAULT)
print("   EMBED_MODEL_DEFAULT =", EMBED_MODEL_DEFAULT)

# ============================================================

✅ resolve_models() is defined.
   CHAT_MODEL_DEFAULT = gpt-4.1-mini
   EMBED_MODEL_DEFAULT = text-embedding-3-small


## CELL 1.5 / STEP 1.5 – Validate OpenAI Key & Models

In [6]:
# CELL 1.5 / STEP 1.5 – Validate OpenAI Key & Models

def validate_openai_key_and_models(api_key: str, chat_model: str, embed_model: str) -> str:
    """
    Lightweight validation:
    - Instantiate client
    - Do a tiny chat completion
    - Do a small embedding call
    Returns a human-readable status string.
    """
    if not api_key:
        return "❌ Please provide an OpenAI API key."

    try:
        client = build_openai_client(api_key)
        resolved_chat, resolved_embed = resolve_models(chat_model, embed_model)

        # Tiny chat test
        _ = client.chat.completions.create(
            model=resolved_chat,
            messages=[
                {"role": "system", "content": "Model availability test."},
                {"role": "user", "content": "Respond with 'OK' only."},
            ],
            max_tokens=2,
            temperature=0.0,
        )

        # Tiny embedding test
        _ = client.embeddings.create(
            model=resolved_embed,
            input=["test"],
        )

        return f"✅ OpenAI key valid. Chat model: `{resolved_chat}`, Embed model: `{resolved_embed}`"

    except Exception as e:
        return f"❌ Error validating key/models: {e}"

#============================================================

## CELL 1.25 — Soft DB sanity gate (bootstrap + report)

In [7]:
# CELL 1.25 — Soft DB sanity gate (bootstrap + report)

#============================================================
import sqlite3
import time

REQUIRED_TABLES = [
    "documents",
    "cohort_docs",
    "cohorts",
    "cohort_meta",
    "users",
    "chat_history",
    "audit_log",
    "model_registry",
]

def soft_db_sanity_gate(db_path: str, base_dir: str, required_tables: list[str]) -> None:
    print("="*80)
    print("DB SANITY GATE — PRE-FLIGHT CHECK")
    print("="*80)
    print("BASE_DIR:", base_dir)
    print("DB_PATH :", db_path)

    os.makedirs(os.path.dirname(db_path), exist_ok=True)

    # Create DB file if missing (do NOT fail on first run)
    first_create = not os.path.exists(db_path)
    if first_create:
        # Touch/create by connecting
        conn = sqlite3.connect(db_path)
        conn.commit()
        conn.close()

    # Bootstrap minimal schema so the app can start from a clean environment
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # Minimal table definitions (aligned with MVP expectations)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS documents (
            id              TEXT PRIMARY KEY,
            doc_name        TEXT NOT NULL,
            cohort_name     TEXT NOT NULL,
            index_id        TEXT NOT NULL,
            n_chunks        INTEGER NOT NULL,
            embed_model     TEXT NOT NULL,
            created_at      TEXT NOT NULL
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS cohort_docs (
            cohort_name TEXT NOT NULL,
            doc_name    TEXT NOT NULL
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS cohorts (
            id              INTEGER PRIMARY KEY AUTOINCREMENT,
            name            TEXT NOT NULL UNIQUE,
            description     TEXT,
            owner_user_id   TEXT,
            created_at      TEXT NOT NULL,
            updated_at      TEXT NOT NULL
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS cohort_meta (
            cohort_name    TEXT PRIMARY KEY,
            owner_user_id  TEXT,
            is_shared      INTEGER DEFAULT 0,
            allow_clone    INTEGER DEFAULT 0,
            created_ts     TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS users (
            user_id      TEXT PRIMARY KEY,
            display_name TEXT,
            role         TEXT,
            created_at   TEXT NOT NULL
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            id             INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id        TEXT,
            role           TEXT,
            cohort_name    TEXT,
            original_query TEXT,
            improved_query TEXT,
            which_prompt   TEXT,
            answer         TEXT,
            chat_model     TEXT,
            created_at     TEXT NOT NULL
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS audit_log (
            id        INTEGER PRIMARY KEY AUTOINCREMENT,
            ts        TEXT NOT NULL,
            username  TEXT,
            role      TEXT,
            action    TEXT NOT NULL,
            details   TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS model_registry (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            provider TEXT,
            model_id TEXT,
            display_name TEXT,
            model_type TEXT,
            enabled INTEGER DEFAULT 1,
            is_default INTEGER DEFAULT 0,
            cost_score INTEGER DEFAULT 2,
            latency_score INTEGER DEFAULT 2,
            max_context_tokens INTEGER,
            api_base TEXT,
            notes TEXT
        )
    """)

    conn.commit()

    # Inspect tables
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
    tables = [r[0] for r in cur.fetchall()]
    missing = [t for t in required_tables if t not in set(tables)]

    # File stats
    exists = os.path.exists(db_path)
    size = os.path.getsize(db_path) if exists else None
    mtime = time.ctime(os.path.getmtime(db_path)) if exists else None

    print("Exists  :", exists)
    print("Size    :", size)
    print("Modified:", mtime)
    print("Tables  :", tables)

    if missing:
        print("❌ DB SANITY GATE FAILED — still missing required tables:", missing)
        raise RuntimeError(f"DB schema incomplete after bootstrap. Missing tables: {missing}")

    print("✅ DB SANITY GATE PASSED")
    print("="*80)

    conn.close()

# Run the gate
soft_db_sanity_gate(DB_PATH, BASE_DIR, REQUIRED_TABLES)

# Optional stop (without raising a traceback)
STOP_AFTER_PREFLIGHT = bool(RUN_PREFLIGHT_ONLY)
if STOP_AFTER_PREFLIGHT:
    print("Stopping after pre-flight by configuration (RUN_PREFLIGHT_ONLY=True).")

# ============================================================

DB SANITY GATE — PRE-FLIGHT CHECK
BASE_DIR: /content/drive/MyDrive/rag_mvp
DB_PATH : /content/drive/MyDrive/rag_mvp/rag_documents_v18.db
Exists  : True
Size    : 229376
Modified: Sat Jan  3 00:09:50 2026
Tables  : ['app_meta', 'audit_log', 'chat_history', 'cohort_docs', 'cohort_meta', 'cohorts', 'demo_prompts', 'documents', 'eval_questions', 'eval_runs', 'eval_sets', 'model_registry', 'rag_trace_events', 'session_memory_store', 'sqlite_sequence', 'users']
✅ DB SANITY GATE PASSED


## CELL 2 / STEP 2 – Document Loading & Chunking

In [8]:
# CELL 2 / STEP 2 – Document Loading & Chunking

def load_pdf(file_bytes: bytes) -> str:
    reader = PdfReader(io.BytesIO(file_bytes))
    texts = []
    for page in reader.pages:
        try:
            txt = page.extract_text() or ""
        except Exception:
            txt = ""
        texts.append(txt)
    return "\n".join(texts)

def load_docx(file_bytes: bytes) -> str:
    f = io.BytesIO(file_bytes)
    doc = DocxDocument(f)
    return "\n".join(p.text for p in doc.paragraphs)

def load_txt(file_bytes: bytes, encoding: str = "utf-8") -> str:
    return file_bytes.decode(encoding, errors="ignore")

def load_file_to_text(file_obj) -> Tuple[str, str]:
    """
    Accepts either:
    - a string filepath (when gr.File(type="filepath") is used), or
    - a file-like object with a .name attribute (older behavior).

    Returns:
      (text_content, original_filename)
    """
    # Case 1: gr.File(type="filepath") -> we get a string path
    if isinstance(file_obj, str):
        path = file_obj
        name = os.path.basename(path)
    else:
        # Case 2: some object with a .name attribute
        path = getattr(file_obj, "name", None)
        if path is None:
            raise ValueError("Unsupported file object from uploader.")
        name = os.path.basename(path)

    with open(path, "rb") as f:
        data = f.read()

    lower = name.lower()
    if lower.endswith(".pdf"):
        text = load_pdf(data)
    elif lower.endswith(".docx"):
        text = load_docx(data)
    elif lower.endswith(".txt"):
        text = load_txt(data)
    else:
        raise ValueError(f"Unsupported file type for {name}")

    return text, name


def chunk_text(text: str, chunk_size: int = CHUNK_SIZE, overlap: int = CHUNK_OVERLAP) -> List[str]:
    """
    Simple sliding-window chunking.
    """
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    tokens = text.split()
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk = " ".join(chunk_tokens)
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

# ============================================================

## CELL 3 / STEP 3 – Embedding & FAISS Index Helpers

In [9]:
# CELL 3 / STEP 3 – Embedding & FAISS Index Helpers

def embed_texts(
    api_key: str,
    embed_model: str,
    texts: List[str],
    batch_size: int = 32,
) -> np.ndarray:
    """
    Embed a list of texts using OpenAI embeddings.
    Returns an ndarray of shape (N, D).
    """
    client = build_openai_client(api_key)
    resolved_chat, resolved_embed = resolve_models("", embed_model)

    vectors: List[List[float]] = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        resp = client.embeddings.create(model=resolved_embed, input=batch)
        for d in resp.data:
            vectors.append(d.embedding)

    arr = np.array(vectors, dtype="float32")
    return arr

def build_faiss_index(vectors: np.ndarray) -> faiss.IndexFlatIP:
    """
    Build a simple inner-product FAISS index from vectors.
    """
    norm = np.linalg.norm(vectors, axis=1, keepdims=True) + 1e-10
    normed = vectors / norm
    dim = normed.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(normed)
    return index

def save_index(index: faiss.IndexFlatIP, index_id: str):
    path = os.path.join(INDEX_DIR, f"{index_id}.faiss")
    faiss.write_index(index, path)

def load_index(index_id: str) -> faiss.IndexFlatIP:
    path = os.path.join(INDEX_DIR, f"{index_id}.faiss")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Index file not found: {path}")
    return faiss.read_index(path)

def save_metadata(index_id: str, meta: Dict[str, Any]):
    path = os.path.join(INDEX_DIR, f"{index_id}.pkl")
    with open(path, "wb") as f:
        pickle.dump(meta, f)

def load_metadata(index_id: str) -> Dict[str, Any]:
    path = os.path.join(INDEX_DIR, f"{index_id}.pkl")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Metadata file not found: {path}")
    with open(path, "rb") as f:
        return pickle.load(f)

# ============================================================

## CELL 4 / STEP 4 – SQLite Persistence for Documents & Cohorts

In [10]:
# CELL 4 / STEP 4 – SQLite Persistence for Documents & Cohorts
import os
import sqlite3
from uuid import uuid4

assert "BASE_DIR" in globals(), "BASE_DIR is not defined. Run CELL 1 first."
assert "DB_PATH"  in globals(), "DB_PATH is not defined. Run CELL 1 first."
print("STEP 4 starting with DB_PATH:", DB_PATH)

def get_db_conn():
    # Ensure BASE_DIR exists (defined in CELL 1)
    os.makedirs(BASE_DIR, exist_ok=True)
    # Ensure DB folder exists
    os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
    return sqlite3.connect(DB_PATH)



def ensure_docs_table():
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS documents (
            id              TEXT PRIMARY KEY,
            doc_name        TEXT NOT NULL,
            cohort_name     TEXT NOT NULL,
            index_id        TEXT NOT NULL,
            n_chunks        INTEGER NOT NULL,
            embed_model     TEXT NOT NULL,
            created_at      TEXT NOT NULL
        )
        """
    )
    conn.commit()
    conn.close()


def ensure_cohort_table():
    """
    Ensure both:
      - cohorts: cohort metadata (required by v16 tests)
      - cohort_docs: mapping of cohort_name -> doc_name (used by the app)
    """
    conn = get_db_conn()
    cur = conn.cursor()

    # ---- New table required by tests ----
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS cohorts (
            id              INTEGER PRIMARY KEY AUTOINCREMENT,
            name            TEXT NOT NULL UNIQUE,
            description     TEXT,
            owner_user_id   TEXT,
            created_at      TEXT NOT NULL,
            updated_at      TEXT NOT NULL
        )
        """
    )

    # ---- Existing mapping table (unchanged behavior) ----
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS cohort_docs (
            cohort_name TEXT NOT NULL,
            doc_name    TEXT NOT NULL
        )
        """
    )

    conn.commit()
    conn.close()


def list_cohorts() -> List[str]:
    # We keep existing behavior: list distinct names from cohort_docs
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("SELECT DISTINCT cohort_name FROM cohort_docs ORDER BY cohort_name ASC")
    rows = cur.fetchall()
    conn.close()
    print("Cohorts:", list_cohorts()[:20], "count=", len(list_cohorts()))

    return [r[0] for r in rows]


def list_docs_in_cohort(cohort_name: str) -> List[str]:
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "SELECT doc_name FROM cohort_docs WHERE cohort_name = ? ORDER BY doc_name ASC",
        (cohort_name,),
    )
    rows = cur.fetchall()
    conn.close()
    return [r[0] for r in rows]


def add_docs_to_cohort(cohort_name: str, doc_names: List[str]):
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    for dn in doc_names:
        cur.execute(
            "INSERT INTO cohort_docs (cohort_name, doc_name) VALUES (?, ?)",
            (cohort_name, dn),
        )
    conn.commit()
    conn.close()


def rename_cohort(old_name: str, new_name: str):
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "UPDATE cohort_docs SET cohort_name = ? WHERE cohort_name = ?",
        (new_name, old_name),
    )
    cur.execute(
        "UPDATE documents SET cohort_name = ? WHERE cohort_name = ?",
        (new_name, old_name),
    )
    conn.commit()
    conn.close()


def delete_cohort(cohort_name: str, reassign_to: Optional[str] = None) -> str:
    """
    Delete a cohort. If reassign_to is provided, documents move there.
    Otherwise, documents are deleted (and their indexes removed).
    NOTE: This is intentionally explicit to avoid orphaned docs.
    """
    ensure_docs_table()
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute(
        "SELECT id, index_id FROM documents WHERE cohort_name = ?",
        (cohort_name,),
    )
    docs = cur.fetchall()

    if reassign_to:
        # Just move docs
        cur.execute(
            "UPDATE documents SET cohort_name = ? WHERE cohort_name = ?",
            (reassign_to, cohort_name),
        )
        cur.execute(
            "UPDATE cohort_docs SET cohort_name = ? WHERE cohort_name = ?",
            (reassign_to, cohort_name),
        )
        msg = f"✅ Cohort '{cohort_name}' renamed/reassigned to '{reassign_to}'. No indexes deleted."
    else:
        # Delete docs and indexes
        for doc_id, index_id in docs:
            # Remove index & metadata
            faiss_path = os.path.join(INDEX_DIR, f"{index_id}.faiss")
            pkl_path = os.path.join(INDEX_DIR, f"{index_id}.pkl")
            for p in [faiss_path, pkl_path]:
                if os.path.exists(p):
                    os.remove(p)
            cur.execute("DELETE FROM documents WHERE id = ?", (doc_id,))

        cur.execute("DELETE FROM cohort_docs WHERE cohort_name = ?", (cohort_name,))
        msg = f"✅ Cohort '{cohort_name}' and its documents/indexes were deleted."

    conn.commit()
    conn.close()
    return msg


def register_document(
    doc_name: str,
    cohort_name: str,
    index_id: str,
    n_chunks: int,
    embed_model: str,
):
    ensure_docs_table()
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    doc_id = str(uuid4())

    # ✅ Use dt.datetime (module alias) so the bottom `import datetime`
    #    in the self-test cell cannot break this.
    created_at = dt.datetime.now(dt.timezone.utc).isoformat()

    cur.execute(
        """
        INSERT INTO documents (id, doc_name, cohort_name, index_id, n_chunks,
                               embed_model, created_at)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """,
        (doc_id, doc_name, cohort_name, index_id, n_chunks, embed_model, created_at),
    )
    cur.execute(
        "INSERT INTO cohort_docs (cohort_name, doc_name) VALUES (?, ?)",
        (cohort_name, doc_name),
    )
    conn.commit()
    conn.close()
    return doc_id


def get_doc_index_id(doc_name: str, cohort_name: str) -> Optional[str]:
    ensure_docs_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT index_id
        FROM documents
        WHERE doc_name = ? AND cohort_name = ?
        """,
        (doc_name, cohort_name),
    )
    row = cur.fetchone()
    conn.close()
    if row:
        return row[0]
    return None



def list_index_ids_for_doc(doc_name: str, cohort_name: str) -> List[str]:
    """Return all index_id values for a given (cohort_name, doc_name)."""
    ensure_docs_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT index_id
        FROM documents
        WHERE doc_name = ? AND cohort_name = ?
        """,
        (doc_name, cohort_name),
    )
    rows = cur.fetchall() or []
    conn.close()
    return [r[0] for r in rows if r and r[0]]


def delete_index_artifacts(index_id: str) -> None:
    """Best-effort delete of on-disk FAISS + metadata artifacts for an index_id."""
    try:
        faiss_path = os.path.join(INDEX_DIR, f"{index_id}.faiss")
        meta_path = os.path.join(INDEX_DIR, f"{index_id}.pkl")
        if os.path.exists(faiss_path):
            os.remove(faiss_path)
        if os.path.exists(meta_path):
            os.remove(meta_path)
    except Exception as e:
        trace_log(f"delete_index_artifacts WARNING index_id={index_id}: {e}")


def overwrite_document_in_cohort(
    cohort_name: str,
    doc_name: str,
) -> int:
    """Remove existing records + artifacts for (cohort_name, doc_name).

    Returns the number of existing document rows removed.
    """
    ensure_docs_table()
    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()

    # Gather existing index ids to clean up artifacts
    index_ids = list_index_ids_for_doc(doc_name=doc_name, cohort_name=cohort_name)

    # Delete DB rows (documents + cohort_docs mapping)
    cur.execute(
        """DELETE FROM documents WHERE doc_name = ? AND cohort_name = ?""",
        (doc_name, cohort_name),
    )
    deleted_docs = cur.rowcount or 0

    cur.execute(
        """DELETE FROM cohort_docs WHERE cohort_name = ? AND doc_name = ?""",
        (cohort_name, doc_name),
    )

    conn.commit()
    conn.close()

    # Delete index artifacts (best-effort)
    for ix in index_ids:
        delete_index_artifacts(ix)

    return deleted_docs


def list_all_documents() -> List[Tuple[str, str, str]]:
    """
    Return list of (doc_name, cohort_name, created_at).
    """
    ensure_docs_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "SELECT doc_name, cohort_name, created_at FROM documents ORDER BY created_at DESC"
    )
    rows = cur.fetchall()
    conn.close()
    return rows
# ------------------------------------------------------------
# SCHEMA BOOTSTRAP (ensures required tables exist for tests)
# ------------------------------------------------------------
ensure_docs_table()
ensure_cohort_table()

# ============================================================

STEP 4 starting with DB_PATH: /content/drive/MyDrive/rag_mvp/rag_documents_v18.db


In [11]:
# CELL 4.5 / STEP 4.5 – EPIC 3.6–4.1 DB Extensions (Session Memory, Traces, Eval)

import json
import time
import sqlite3

def ensure_epic_tables():
    conn = get_db_conn()
    cur = conn.cursor()

    # ============================================================
    # EPIC 3.6 – Optional persisted session memory (disabled by default)
    # ============================================================
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS session_memory_store (
            user_id     TEXT NOT NULL,
            scope       TEXT NOT NULL DEFAULT 'session',
            memory_json TEXT NOT NULL,
            updated_at  TEXT NOT NULL,
            expires_at  TEXT,
            PRIMARY KEY (user_id, scope)
        )
        """
    )

    # ============================================================
    # EPIC 3.7 – Trace events (lightweight observability)
    # ============================================================
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS rag_trace_events (
            trace_id        TEXT PRIMARY KEY,
            created_at      TEXT NOT NULL,
            user_id         TEXT,
            cohort_name     TEXT,
            model_id        TEXT,
            provenance_mode TEXT,
            memory_scope    TEXT,
            memory_used     INTEGER,
            rag_used        INTEGER,
            retrieved_k     INTEGER,
            retrieval_query TEXT,
            note            TEXT
        )
        """
    )

    # ============================================================
    # EPIC 4.1 – Evaluation harness tables
    # ============================================================
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS eval_sets (
            eval_set_id TEXT PRIMARY KEY,
            name        TEXT NOT NULL,
            description TEXT,
            created_at  TEXT NOT NULL
        )
        """
    )
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS eval_questions (
            eval_q_id        TEXT PRIMARY KEY,
            eval_set_id      TEXT NOT NULL,
            cohort_name      TEXT,
            question         TEXT NOT NULL,
            provenance_mode  TEXT NOT NULL DEFAULT 'Explain Sources',
            expect_evidence  INTEGER NOT NULL DEFAULT 1,
            expect_refusal   INTEGER NOT NULL DEFAULT 0,
            created_at       TEXT NOT NULL,
            FOREIGN KEY(eval_set_id) REFERENCES eval_sets(eval_set_id)
        )
        """
    )
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS eval_runs (
            eval_run_id   TEXT PRIMARY KEY,
            eval_set_id   TEXT NOT NULL,
            model_id      TEXT,
            created_at    TEXT NOT NULL,
            summary_json  TEXT NOT NULL,
            FOREIGN KEY(eval_set_id) REFERENCES eval_sets(eval_set_id)
        )
        """
    )

    conn.commit()
    conn.close()

ensure_epic_tables()
print("✅ STEP 4.5 complete: EPIC 3.6–4.1 tables ensured.")

✅ STEP 4.5 complete: EPIC 3.6–4.1 tables ensured.


## CELL 4.5 / STEP 4.5 – Users & Chat History (7-Day Retention)

In [12]:
# CELL 4.5 / STEP 4.5 – Users & Chat History (7-Day Retention)

from dataclasses import dataclass
import datetime as dt

# ============================================================
# USER IDENTITY MODEL

@dataclass
class SessionUser:
    username: str | None = None
    role: str = "anonymous"   # "anonymous", "user", "admin"

    @property
    def is_authenticated(self) -> bool:
        return bool(self.username and str(self.username).strip()) and self.role in ("user", "admin")

    @property
    def is_admin(self) -> bool:
        return self.role == "admin"


def require_login(session_user, action: str = "access this feature") -> SessionUser:
    """Enforce authenticated access (no access for anonymous users).

    Accepts SessionUser, dict, or None and returns a normalized SessionUser.
    Raises PermissionError if not authenticated.
    """
    if isinstance(session_user, SessionUser):
        su = session_user
    elif isinstance(session_user, dict):
        su = SessionUser(
            username=((session_user.get("username") or "").strip() or None),
            role=(session_user.get("role") or "anonymous"),
        )
    else:
        su = SessionUser(username=None, role="anonymous")

    if not su.is_authenticated:
        raise PermissionError(f"Login required to {action}.")
    return su

# MVP in-memory auth store (will be replaced later by ICAM/SSO)
USERS = {
    "admin": {"password": "admin123", "role": "admin"},
    "demo1":  {"password": "demo1123",  "role": "user"},
    "demo2": {"password": "demo2123", "role": "user"},
}

# ============================================================
# EPIC 3.1 — Session-scoped memory helpers (in-memory only)

# ============================================================

def init_session_memory_state() -> dict:
    """
    Session memory scope is configurable: per cohort (default) or per session.  # EPIC 3.3
    Shape:
      {"cohort_name": <str|None>, "turns": [{"q": str, "a": str}, ...]}
    """
    return {"enabled": True, "scope": "cohort", "cohort_name": None, "turns": [], "max_turns": SESSION_MEM_MAX_TURNS}  # EPIC 3.3


def format_session_memory_prompt(turns: list[dict]) -> str:
    """
    Convert last N Q/A pairs into a compact system prompt appendix.
    Keep it short—this is "lightweight memory".
    """
    if not turns:
        return ""

    lines = []
    lines.append("You have access to the prior Q/A from this session (same cohort).")
    lines.append("Use it only to maintain continuity if relevant; do not invent facts.\n")

    last_q = (turns[-1].get("q") or "").strip()
    if last_q:
        lines.append(f"Last user question (most recent): {last_q}")
        lines.append("")


    for i, t in enumerate(turns, start=1):
        q = (t.get("q") or "").strip()
        a = (t.get("a") or "").strip()
        if not q and not a:
            continue
        lines.append(f"Session Q{i}: {q}")
        lines.append(f"Session A{i}: {a}")
        lines.append("")
    return "\n".join(lines).strip()


def update_session_memory_state(mem: dict, cohort_name: str, q: str, a: str, max_turns: int) -> dict:
    """
    - If cohort changes, clear memory
    - Append the new Q/A
    - Trim to last max_turns
    """
    if not isinstance(mem, dict):
        mem = init_session_memory_state()

    # EPIC 3.2 — Admin toggle: if disabled, keep memory empty and do not append
    if not mem.get("enabled", True):
        mem["cohort_name"] = cohort_name
        mem["turns"] = []
        return mem

    cur_cohort = mem.get("cohort_name")
    scope = (mem.get("scope") or "cohort").lower()  # EPIC 3.3
    if scope == "cohort" and cur_cohort != cohort_name:
        mem = init_session_memory_state()
        mem["cohort_name"] = cohort_name
    else:
        mem["cohort_name"] = cohort_name

    turns = mem.get("turns") or []
    turns.append({"q": (q or "").strip(), "a": (a or "").strip()})

    # Trim
    if max_turns and len(turns) > max_turns:
        turns = turns[-max_turns:]

    mem["turns"] = turns
    return mem

# ============================================================
# USERS TABLE

def ensure_user_table():
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS users (
            user_id      TEXT PRIMARY KEY,
            display_name TEXT,
            role         TEXT,
            created_at   TEXT NOT NULL
        )
        """
    )
    conn.commit()
    conn.close()


def upsert_user(user_id: str, role: str, display_name: Optional[str] = None):
    """
    Basic ICAM-ready user record.
    Inserts new or updates existing users.
    """
    if not user_id:
        return

    ensure_user_table()
    conn = get_db_conn()
    cur = conn.cursor()
    now = dt.datetime.now(dt.timezone.utc).isoformat()

    cur.execute(
        """
        INSERT INTO users (user_id, display_name, role, created_at)
        VALUES (?, ?, ?, ?)
        ON CONFLICT(user_id) DO UPDATE SET
            display_name = COALESCE(?, users.display_name),
            role = COALESCE(?, users.role)
        """,
        (user_id, display_name, role, now, display_name, role),
    )
    conn.commit()
    conn.close()

# ============================================================
# CHAT HISTORY TABLE

def ensure_chat_history_table():
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS chat_history (
            id             INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id        TEXT,
            role           TEXT,
            cohort_name    TEXT,
            original_query TEXT,
            improved_query TEXT,
            which_prompt   TEXT,
            answer         TEXT,
            chat_model     TEXT,
            created_at     TEXT NOT NULL
        )
        """
    )
    conn.commit()
    conn.close()

# ============================================================
# 7-DAY RETENTION

def prune_chat_history(days: int = 7):
    ensure_chat_history_table()
    cutoff = dt.datetime.now(dt.timezone.utc) - dt.timedelta(days=days)
    cutoff_iso = cutoff.isoformat()

    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("DELETE FROM chat_history WHERE created_at < ?", (cutoff_iso,))
    conn.commit()
    conn.close()

# ============================================================
#v14 BEHAVIOR: Save Chat Interaction (DETAILED LOGGING)

def save_chat_interaction(
    user_id: str,
    role: str,
    cohort_name: str,
    original_query: str,
    improved_query: str,
    which_prompt: str,
    answer: str,
    chat_model: str,
):
    """
    This is your existing v16 logging mechanism.
    Now fully preserved and compatible with v16.
    """
    ensure_chat_history_table()
    prune_chat_history(days=7)

    conn = get_db_conn()
    cur = conn.cursor()
    now = dt.datetime.now(dt.timezone.utc).isoformat()

    cur.execute(
        """
        INSERT INTO chat_history (
            user_id, role, cohort_name, original_query, improved_query,
            which_prompt, answer, chat_model, created_at
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        (
            user_id or None,
            role or None,
            cohort_name or None,
            original_query,
            improved_query,
            which_prompt,
            answer,
            chat_model,
            now,
        ),
    )
    conn.commit()
    conn.close()

# ============================================================
# v16 REQUIRED FUNCTIONS – FLEXIBLE SIGNATURES

def _extract_user_id(user: Any) -> str:
    """
    Helper to derive a stable user_id from various representations.
    """
    if user is None:
        return "anonymous"

    # dict-like
    if isinstance(user, dict):
        for key in ("username", "user_id", "name"):
            v = user.get(key)
            if v:
                return str(v)

    # attribute-based (SessionUser or other objects)
    for attr in ("username", "user_id", "name"):
        try:
            v = getattr(user, attr, None)
            if v:
                return str(v)
        except Exception:
            pass

    # mapping-like (sqlite3.Row, etc.)
    try:
        for key in ("username", "user_id", "name"):
            if key in user:
                v = user[key]
                if v:
                    return str(v)
    except Exception:
        pass

    return "anonymous"


def save_chat_history(
    messages=None,
    *args,
    user: Any = None,
    user_id: Optional[str] = None,
    cohort_name: str = "default",
    cohort: Optional[str] = None,
    **kwargs,
):
    """
    v16 Test Suite Requirement:
    Supports BOTH:
      1) save_chat_history(messages=[{role, content}, ...], user=..., cohort=...)
      2) save_chat_history(user=..., cohort=..., question="Q", answer="A", model_used="...")

    In (2) we write a single row using the Q/A fields.
    """
    ensure_chat_history_table()
    prune_chat_history(days=7)

    # Normalize cohort alias
    if cohort is not None:
        cohort_name = cohort

    # Derive user_id if needed
    if user_id is None:
        user_id = _extract_user_id(user)

    # ---- Path 1: test-style call with question/answer/model_used ----
    question = kwargs.get("question")
    answer = kwargs.get("answer")
    model_used = kwargs.get("model_used")

    if messages is None and (question is not None or answer is not None):
        now = dt.datetime.now(dt.timezone.utc).isoformat()
        conn = get_db_conn()
        cur = conn.cursor()

        cur.execute(
            """
            INSERT INTO chat_history (
                user_id, role, cohort_name, original_query,
                improved_query, which_prompt, answer,
                chat_model, created_at
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                user_id,          # user_id
                "user",           # role
                cohort_name,      # cohort_name
                question or "",   # original_query
                None,             # improved_query
                None,             # which_prompt
                answer or "",     # answer
                model_used,       # chat_model
                now,              # created_at
            ),
        )

        conn.commit()
        conn.close()
        return  # We're done for this style of call

    # ---- Path 2: normal messages-based usage ----
    if messages is None:
        messages = kwargs.get("messages", None)

    # If still no messages, treat as no-op
    if messages is None:
        return

    now = dt.datetime.now(dt.timezone.utc).isoformat()
    conn = get_db_conn()
    cur = conn.cursor()

    for m in messages:
        role = m.get("role", "user")
        content = m.get("content", "")

        cur.execute(
            """
            INSERT INTO chat_history (
                user_id, role, cohort_name, original_query,
                improved_query, which_prompt, answer,
                chat_model, created_at
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                user_id,
                role,
                cohort_name,
                content,   # original_query
                None,
                None,
                None,
                None,
                now,
            ),
        )

    conn.commit()
    conn.close()

    # Update session memory (EPIC 3.1)
    session_mem = update_session_memory_state(
        mem=session_mem,
        cohort_name=cohort_name,
        q=final_question,
        a=raw_answer or "",
        max_turns=int((session_mem or {}).get('max_turns') or SESSION_MEM_MAX_TURNS),  # EPIC 3.3

    )


def load_chat_history(
    *args,
    **kwargs,
):
    """
    v16 Test Suite Requirement:
    Returns list[dict] with keys: role, content, created_at.

    For simplicity and maximum compatibility with the tests,
    we ignore user/cohort filters here and just return the
    oldest messages up to `limit`.
    """
    ensure_chat_history_table()

    limit = int(kwargs.get("limit", 50))

    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute(
        """
        SELECT role, original_query, created_at
        FROM chat_history
        ORDER BY created_at ASC
        LIMIT ?
        """,
        (limit,),
    )

    rows = cur.fetchall()
    conn.close()

    return [
        {"role": r[0], "content": r[1], "created_at": r[2]}
        for r in rows
    ]

# ============================================================
# RECENT HISTORY (Admin screens / Debug UI)

def get_recent_history(
    user_id: Optional[str] = None,
    cohort_name: Optional[str] = None,
    limit: int = 50,
) -> List[Dict[str, Any]]:
    """
    Returns full detailed history rows for admin/debug views.
    """
    ensure_chat_history_table()

    conn = get_db_conn()
    cur = conn.cursor()

    query = """
        SELECT user_id, role, cohort_name, original_query, improved_query,
               which_prompt, answer, chat_model, created_at
        FROM chat_history
    """
    params = []
    conditions = []

    if user_id:
        conditions.append("user_id = ?")
        params.append(user_id)
    if cohort_name:
        conditions.append("cohort_name = ?")
        params.append(cohort_name)

    if conditions:
        query += " WHERE " + " AND ".join(conditions)

    query += " ORDER BY created_at DESC LIMIT ?"
    params.append(limit)

    cur.execute(query, params)
    rows = cur.fetchall()
    conn.close()

    return [
        {
            "user_id": r[0],
            "role": r[1],
            "cohort_name": r[2],
            "original_query": r[3],
            "improved_query": r[4],
            "which_prompt": r[5],
            "answer": r[6],
            "chat_model": r[7],
            "created_at": r[8],
        }
        for r in rows
    ]

# ============================================================
# ADMIN: LIST USERS

def list_users() -> List[Tuple[str, str, str]]:
    ensure_user_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("SELECT user_id, display_name, role FROM users ORDER BY created_at DESC")
    rows = cur.fetchall()
    conn.close()
    return rows

# ============================================================

In [13]:
# EPIC 5.2 / Admin helper (global)
def _ensure_admin(session_user):
    """Return (SessionUser|None, message). Accepts session_state dict or SessionUser."""
    if not session_user:
        return None, "❌ Not logged in."
    if isinstance(session_user, dict):
        username = (session_user.get("username") or "").strip() or None
        role = (session_user.get("role") or "user").strip() or "user"
    else:
        username = (getattr(session_user, "username", None) or "").strip() or None
        role = (getattr(session_user, "role", "user") or "user")
    su = SessionUser(username=username, role=role)
    if not su.is_admin:
        return None, "❌ Admin privileges required."
    return su, ""


In [14]:
# CELL 4.6 / STEP 4.6 – EPIC 3.6 Session Memory Governance + Export/Import + Optional Persistence

import json
import re
from datetime import datetime as _dt, timedelta as _td

# ============================================================
# EPIC 3.6 – Controls
# ============================================================
EPIC36_PERSIST_MEMORY_ENABLED = False   # OFF by default; enable when ready
EPIC36_PERSIST_TTL_HOURS = 24

# Allow-list categories for memory entries (simple governance)
EPIC36_ALLOWED_CATEGORIES = {"identity", "preference", "task", "other"}

# Hard blocklist patterns (never store)
_EPIC36_SECRET_PATTERNS = [
    r"sk-[A-Za-z0-9]{10,}",                 # OpenAI style keys
    r"-----BEGIN [A-Z ]+PRIVATE KEY-----",  # private keys
    r"password\s*[:=]",                    # password:
    r"api\s*key\s*[:=]",                  # api key:
]

def epic36_is_secret(text: str) -> bool:
    t = (text or "").strip()
    if not t:
        return False
    for p in _EPIC36_SECRET_PATTERNS:
        if re.search(p, t, flags=re.IGNORECASE):
            return True
    return False

def epic36_sanitize_memory_items(items: list[dict]) -> list[dict]:
    """Remove secrets and enforce categories."""
    out = []
    for it in items or []:
        if not isinstance(it, dict):
            continue
        key = str(it.get("key", "")).strip()
        val = str(it.get("value", "")).strip()
        cat = str(it.get("category", "other") or "other").strip().lower()
        if cat not in EPIC36_ALLOWED_CATEGORIES:
            cat = "other"
        if epic36_is_secret(key) or epic36_is_secret(val):
            continue
        if key and val:
            out.append({"category": cat, "key": key, "value": val})
    return out

def epic36_export_session_memory_json(session_mem: dict) -> str:
    """Export current in-memory session memory to a JSON string."""
    if not session_mem:
        return json.dumps({"enabled": False, "scope": "session", "items": []}, indent=2)
    safe = dict(session_mem)
    safe["items"] = epic36_sanitize_memory_items(safe.get("items", []))
    return json.dumps(safe, indent=2)

def epic36_import_session_memory_json(json_text: str, session_mem: dict | None = None) -> dict:
    """Import JSON into the current session memory state (sanitized)."""
    base = session_mem or init_session_memory_state()
    try:
        data = json.loads(json_text or "{}")
        enabled = bool(data.get("enabled", base.get("enabled", False)))
        scope = data.get("scope", base.get("scope", "session"))
        if scope not in ("session", "cohort"):
            scope = "session"
        items = epic36_sanitize_memory_items(data.get("items", []))
        base.update({"enabled": enabled, "scope": scope, "items": items})
        return base
    except Exception:
        return base

def epic36_persist_session_memory(user_id: str, session_mem: dict, scope: str = "session") -> None:
    """Persist memory to SQLite if enabled."""
    if not EPIC36_PERSIST_MEMORY_ENABLED:
        return
    if not user_id:
        return
    mem = dict(session_mem or {})
    mem["items"] = epic36_sanitize_memory_items(mem.get("items", []))
    updated_at = _dt.utcnow().isoformat(timespec="seconds") + "Z"
    expires_at = (_dt.utcnow() + _td(hours=EPIC36_PERSIST_TTL_HOURS)).isoformat(timespec="seconds") + "Z"
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """INSERT INTO session_memory_store (user_id, scope, memory_json, updated_at, expires_at)
             VALUES (?, ?, ?, ?, ?)
             ON CONFLICT(user_id, scope) DO UPDATE SET
                memory_json=excluded.memory_json,
                updated_at=excluded.updated_at,
                expires_at=excluded.expires_at""",
        (user_id, scope, json.dumps(mem), updated_at, expires_at),
    )
    conn.commit()
    conn.close()

def epic36_load_persisted_session_memory(user_id: str, scope: str = "session") -> dict | None:
    """Load persisted memory if enabled and not expired."""
    if not EPIC36_PERSIST_MEMORY_ENABLED:
        return None
    if not user_id:
        return None
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "SELECT memory_json, expires_at FROM session_memory_store WHERE user_id=? AND scope=?",
        (user_id, scope),
    )
    row = cur.fetchone()
    conn.close()
    if not row:
        return None
    mem_json, expires_at = row
    if expires_at:
        try:
            exp = _dt.fromisoformat(expires_at.replace("Z",""))
            if _dt.utcnow() > exp:
                return None
        except Exception:
            pass
    try:
        data = json.loads(mem_json)
        if isinstance(data, dict):
            data["items"] = epic36_sanitize_memory_items(data.get("items", []))
            return data
    except Exception:
        return None
    return None

print("✅ STEP 4.6 loaded: EPIC 3.6 memory governance helpers ready (persistence OFF by default).")

✅ STEP 4.6 loaded: EPIC 3.6 memory governance helpers ready (persistence OFF by default).


## CELL 4.6 / STEP 4.6 – Audit Log (v16)

In [15]:
# CELL 4.6 / STEP 4.6 – Audit Log (v16)

def ensure_audit_table():
    """
    Create an audit_log table if it doesn't already exist.
    """
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS audit_log (
            id        INTEGER PRIMARY KEY AUTOINCREMENT,
            ts        TEXT NOT NULL,
            username  TEXT,
            role      TEXT,
            action    TEXT NOT NULL,
            details   TEXT
        )
        """
    )
    conn.commit()
    conn.close()


def log_audit(username: str, role: str, action: str, details: str = ""):
    """
    Insert a row into the audit_log table.
    - ts: UTC ISO timestamp
    - username / role: may be None/empty for anonymous
    - action: short code, e.g. 'login', 'ask', 'admin_refresh', 'delete_cohort'
    - details: freeform string with context
    """
    ensure_audit_table()
    conn = get_db_conn()
    cur = conn.cursor()

    # Use the global datetime alias `dt` so we don't conflict with any
    # later "import datetime" inside the self-test cell.
    now = dt.datetime.now(dt.timezone.utc).isoformat()

    cur.execute(
        """
        INSERT INTO audit_log (ts, username, role, action, details)
        VALUES (?, ?, ?, ?, ?)
        """,
        (now, username or "", role or "", action, details or ""),
    )
    conn.commit()
    conn.close()


def trace_log(message: str):
    """
    Append a timestamped debug line to debug_trace_v16.log in BASE_DIR.

    Used to capture errors/warnings that happen inside Gradio callbacks
    where the UI might just show a generic 'Error'.
    """
    try:
        ts = dt.datetime.now(dt.timezone.utc).isoformat()
    except Exception:
        ts = "UNKNOWN_TIME"

    line = f"{ts} {message}\n"
    try:
        with open(TRACE_LOG_PATH, "a", encoding="utf-8") as f:
            f.write(line)
    except Exception as e:
        # Last resort: don't let logging itself crash anything
        print("TRACE_LOG_ERROR", e, line)



import traceback as _tb

def safe_ui_call(fn, fallback=None):
    """Wrap a Gradio callback so exceptions don't surface as generic 'Error' UI outputs.
    Writes full traceback to trace_log() and returns a safe fallback value.
    """
    if fallback is None:
        fallback = "❌ An internal error occurred. See debug trace log."
    def _wrapped(*args, **kwargs):
        try:
            return fn(*args, **kwargs)
        except Exception as e:
            try:
                trace_log(f"[UI ERROR] {getattr(fn, '__name__', 'callback')}: {e}\n{_tb.format_exc()}")
            except Exception:
                pass
            return fallback
    return _wrapped

# ============================================================
# STEP 4.7 – Cohort Ownership & Sharing (v16-safe)

from typing import Optional, List, Dict, Any

def ensure_cohort_meta_table():
    """
    Metadata for cohorts (v16_1 Phase 1):
      - owner_user_id: who created/owns the cohort (REQUIRED)
      - is_shared: 0 = private to owner/admin, 1 = visible to all authenticated users
      - allow_clone: 0 = non-owners cannot clone, 1 = cloning enabled for authenticated users (when shared)
      - created_ts: UTC timestamp

    Notes:
      - Performs a lightweight schema migration if cohort_meta already exists.
    """
    conn = get_db_conn()
    cur = conn.cursor()

    # Create base table if missing
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS cohort_meta (
            cohort_name    TEXT PRIMARY KEY,
            owner_user_id  TEXT,
            is_shared      INTEGER DEFAULT 0,
            created_ts     TEXT
        )
        """
    )

    # ---- Schema migration (add allow_clone if missing) ----
    try:
        cur.execute("PRAGMA table_info(cohort_meta)")
        cols = {r[1] for r in cur.fetchall()}  # r[1] = column name
        if "allow_clone" not in cols:
            cur.execute("ALTER TABLE cohort_meta ADD COLUMN allow_clone INTEGER DEFAULT 0")
        if "description" not in cols:
            cur.execute("ALTER TABLE cohort_meta ADD COLUMN description TEXT")
        if "intended_audience" not in cols:
            cur.execute("ALTER TABLE cohort_meta ADD COLUMN intended_audience TEXT")
        if "updated_ts" not in cols:
            cur.execute("ALTER TABLE cohort_meta ADD COLUMN updated_ts TEXT")
    except Exception as e:
        # Do not hard-fail cohort operations due to migration issues
        trace_log(f"ensure_cohort_meta_table migration warning: {e}")

    conn.commit()
    conn.close()

def _extract_username_from_user(user: SessionUser | dict | None) -> Optional[str]:
    """
    Helper: accept SessionUser, dict, or None and pull out a username string.
    """
    if user is None:
        return None

    # If SessionUser dataclass
    if isinstance(user, SessionUser):
        return user.username

    # If dict-like
    if isinstance(user, dict):
        # Try common keys in order
        for k in ("username", "user_id", "name"):
            try:
                v = user.get(k)
                if v:
                    return str(v)
            except Exception:
                pass

    return None


def set_cohort_owner(cohort_name: str, user: SessionUser | dict | None):
    """
    Register the owner of a cohort, but do NOT change it once set.

    HARD REQUIREMENT (v16_1 Phase 1 security):
      - Cohorts must have an authenticated owner.
      - We do NOT silently fall back to "anonymous".

    Behavior:
      - On first creation: insert owner_user_id and is_shared=0 (private).
      - On subsequent calls for the same cohort_name: owner remains unchanged.
    """
    if not cohort_name:
        return

    from datetime import datetime, timezone

    owner = _extract_username_from_user(user)
    if not owner or not str(owner).strip() or str(owner).strip().lower() == "anonymous":
        raise ValueError("Cohort owner is required. Please log in before creating/updating a cohort.")


    ensure_cohort_meta_table()
    conn = get_db_conn()
    cur = conn.cursor()

    now = datetime.now(timezone.utc).isoformat()

    # Insert cohort_meta row if missing; if it already exists with an invalid owner, repair the owner once.
    cur.execute(
        """
        INSERT INTO cohort_meta (cohort_name, owner_user_id, is_shared, created_ts)
        VALUES (?, ?, 0, ?)
        ON CONFLICT(cohort_name) DO UPDATE SET
            owner_user_id = excluded.owner_user_id
        WHERE cohort_meta.owner_user_id IS NULL
           OR TRIM(cohort_meta.owner_user_id) = ''
           OR LOWER(TRIM(cohort_meta.owner_user_id)) = 'anonymous'
        """,
        (cohort_name, owner, now),
    )


    conn.commit()
    conn.close()


def set_cohort_sharing(cohort_name: str, is_shared: bool | int):
    """
    Update ONLY the is_shared flag for a cohort in cohort_meta.

    is_shared:
        False/0 -> private (owner + admins)
        True/1  -> shared (all users can see; non-owners are read-only)
    """
    if not cohort_name:
        return

    ensure_cohort_meta_table()
    conn = get_db_conn()
    cur = conn.cursor()
    shared_flag = 1 if is_shared else 0

    cur.execute(
        """
        UPDATE cohort_meta
        SET is_shared = ?
        WHERE cohort_name = ?
        """,
        (shared_flag, cohort_name),
    )
    conn.commit()
    conn.close()

def set_cohort_clone_allowed(cohort_name: str, allow_clone: bool | int):
    """
    Update ONLY the allow_clone flag for a cohort in cohort_meta.

    allow_clone:
        False/0 -> non-owners cannot clone (even if shared)
        True/1  -> if shared, any authenticated user may clone
    """
    if not cohort_name:
        return

    ensure_cohort_meta_table()
    conn = get_db_conn()
    cur = conn.cursor()
    flag = 1 if allow_clone else 0

    cur.execute(
        """
        UPDATE cohort_meta
        SET allow_clone = ?
        WHERE cohort_name = ?
        """,
        (flag, cohort_name),
    )
    conn.commit()
    conn.close()

def set_cohort_allow_clone(cohort_name: str, allow_clone: bool) -> None:
    """
    Backward-compatible alias for older call sites.
    """
    return set_cohort_clone_allowed(cohort_name, allow_clone)


#def set_cohort_clone_allowed(cohort_name: str, allow_clone: bool | int):
#    """Backward-compatible alias for earlier UI code."""
#    return set_cohort_clone_allowed(cohort_name, allow_clone)

def _get_cohort_meta(cohort_name: str) -> tuple[str | None, int, int]:
    """
    Returns (owner_user_id, is_shared, allow_clone).
    If cohort not found in meta, returns (None, 0, 0).
    """
    ensure_cohort_meta_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "SELECT owner_user_id, COALESCE(is_shared,0), COALESCE(allow_clone,0) FROM cohort_meta WHERE cohort_name = ?",
        (cohort_name,),
    )
    row = cur.fetchone()
    conn.close()
    if not row:
        return None, 0, 0
    return (row[0], int(row[1] or 0), int(row[2] or 0))


def can_clone_cohort(session_user: Any, source_cohort: str) -> tuple[bool, str | None]:
    """
    Returns (allowed, message_if_denied).

    Rules (Option A):
      - Admin: may always clone.
      - Owner: may always clone.
      - Other authenticated users:
          * allowed ONLY if source cohort is_shared=1 AND allow_clone=1
      - Anonymous: not allowed.
    """
    if not source_cohort:
        return False, "❌ Please select a cohort to clone."

    username = _extract_user_id(session_user)
    if not username or username == "anonymous":
        return False, "❌ You must be logged in to clone a cohort."

    # Determine admin role
    role = None
    if isinstance(session_user, dict):
        role = session_user.get("role")
    elif isinstance(session_user, SessionUser):
        role = "admin" if session_user.is_admin else "user"
    else:
        role = USERS.get(username, {}).get("role", "user")

    if role == "admin":
        return True, None

    owner, is_shared, allow_clone = _get_cohort_meta(source_cohort)

    # Owner may always clone
    if owner and owner == username:
        return True, None

    # Non-owner: only if shared + cloning allowed
    if is_shared == 1 and allow_clone == 1:
        return True, None

    return False, "❌ Cloning is not permitted for this cohort (owner has disabled cloning)."


def clone_cohort(source_cohort: str, target_cohort: str, new_owner: str) -> str:
    """
    Clone a cohort by duplicating its document rows and index artifacts.

    Implementation notes:
      - We copy each source FAISS index (.faiss) and metadata (.pkl) to a new index_id.
      - We create new rows in documents and cohort_docs for the target cohort.
      - The cloned cohort is created as PRIVATE by default (is_shared=0, allow_clone=0).
    """
    if not source_cohort or not target_cohort:
        return "❌ Source and target cohort names are required."

    if not new_owner or not str(new_owner).strip() or str(new_owner).strip().lower() == "anonymous":
        return "❌ You must be logged in to clone a cohort (owner is required)."


    target_cohort = target_cohort.strip()
    if cohort_exists(target_cohort):
        return f"❌ Target cohort '{target_cohort}' already exists. Choose a different name."

    ensure_docs_table()
    ensure_cohort_table()
    ensure_cohort_meta_table()

    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute(
        "SELECT doc_name, index_id, n_chunks, embed_model FROM documents WHERE cohort_name = ?",
        (source_cohort,),
    )
    rows = cur.fetchall()

    if not rows:
        conn.close()
        return f"❌ Source cohort '{source_cohort}' has no documents to clone."

    created_at = dt.datetime.now(dt.timezone.utc).isoformat()

    # Create a meta row for the new cohort (private by default)
    cur.execute(
        """
        INSERT INTO cohort_meta (cohort_name, owner_user_id, is_shared, allow_clone, created_ts)
        VALUES (?, ?, 0, 0, ?)
        ON CONFLICT(cohort_name) DO NOTHING
        """,
        (target_cohort, (new_owner or "").strip(), created_at),
    )

    cloned_count = 0
    for (doc_name, src_index_id, n_chunks, embed_model) in rows:
        new_index_id = str(uuid4())

        # Copy FAISS + PKL artifacts to new ids
        src_faiss = os.path.join(INDEX_DIR, f"{src_index_id}.faiss")
        src_pkl = os.path.join(INDEX_DIR, f"{src_index_id}.pkl")
        dst_faiss = os.path.join(INDEX_DIR, f"{new_index_id}.faiss")
        dst_pkl = os.path.join(INDEX_DIR, f"{new_index_id}.pkl")

        if not os.path.exists(src_faiss) or not os.path.exists(src_pkl):
            trace_log(f"clone_cohort warning: missing artifacts for index_id={src_index_id}")
            continue

        shutil.copyfile(src_faiss, dst_faiss)
        shutil.copyfile(src_pkl, dst_pkl)

        # Insert new document row
        new_doc_id = str(uuid4())
        cur.execute(
            """
            INSERT INTO documents (id, doc_name, cohort_name, index_id, n_chunks, embed_model, created_at)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """,
            (new_doc_id, doc_name, target_cohort, new_index_id, int(n_chunks), embed_model, created_at),
        )

        # Map doc to cohort_docs
        cur.execute(
            "INSERT INTO cohort_docs (cohort_name, doc_name) VALUES (?, ?)",
            (target_cohort, doc_name),
        )

        cloned_count += 1

    conn.commit()
    conn.close()

    return f"✅ Cloned cohort '{source_cohort}' into '{target_cohort}' with {cloned_count} documents."


def cohort_exists(cohort_name: str) -> bool:
    """
    Returns True if a cohort with this name already exists in cohort_docs.
    This is global (not per-user) to avoid confusing duplicate names.
    """
    if not cohort_name:
        return False

    ensure_cohort_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "SELECT 1 FROM cohort_docs WHERE cohort_name = ? LIMIT 1",
        (cohort_name,),
    )
    row = cur.fetchone()
    conn.close()
    return row is not None


def list_cohorts_for_user(user: SessionUser | dict | None) -> list[str]:
    """
    Return list of cohort names visible to the given user.

    Rules:
      - Admins: all cohorts (global)
      - Non-admin:
          * Cohorts where they are owner (cohort_meta.owner_user_id)
          * Cohorts marked is_shared = 1
      - Anonymous: only shared cohorts (is_shared = 1)

    If anything goes wrong with the metadata logic, falls back to global list_cohorts().
    Also writes debug info to the v16 trace log.
    """
    trace_log(f"list_cohorts_for_user called with user={user!r}")

    ensure_cohort_table()
    ensure_cohort_meta_table()

    conn = get_db_conn()
    cur = conn.cursor()

    try:
        # Determine if user is admin, safely
        is_admin = False
        username = None

        if isinstance(user, SessionUser):
            is_admin = user.is_admin
            username = user.username
        elif isinstance(user, dict):
            username = _extract_username_from_user(user)
            role = user.get("role")
            is_admin = (role == "admin")

        if is_admin:
            # Admin sees all distinct cohorts
            cur.execute(
                """
                SELECT DISTINCT cohort_name
                FROM cohort_docs
                ORDER BY cohort_name
                """
            )
        else:
            # Non-admin or anonymous
            if username:
                # Logged-in non-admin -> owner or shared
                cur.execute(
                    """
                    SELECT DISTINCT cd.cohort_name
                    FROM cohort_docs cd
                    LEFT JOIN cohort_meta cm
                      ON cd.cohort_name = cm.cohort_name
                    WHERE cm.owner_user_id = ?
                       OR cm.is_shared = 1
                       OR cm.cohort_name IS NULL   -- safety: cohorts without meta still appear
                    ORDER BY cd.cohort_name
                    """,
                    (username,),
                )
            else:
                # Anonymous -> only shared (or cohorts w/o meta as a fallback)
                cur.execute(
                    """
                    SELECT DISTINCT cd.cohort_name
                    FROM cohort_docs cd
                    LEFT JOIN cohort_meta cm
                      ON cd.cohort_name = cm.cohort_name
                    WHERE cm.is_shared = 1
                       OR cm.cohort_name IS NULL
                    ORDER BY cd.cohort_name
                    """
                )

        rows = cur.fetchall()
        conn.close()
        names = [r[0] for r in rows]

        # Final safety net: do not leak global cohorts; return empty.
        if not names:
            trace_log("list_cohorts_for_user -> no rows; returning empty")
            names = []

        trace_log(f"list_cohorts_for_user returning {names}")
        return names

    except Exception as e:
        conn.close()
        trace_log(f"list_cohorts_for_user ERROR: {e}")
        return []

# ------------------------------------------------------------
# UI HELPERS FOR COHORT LISTS (used in STEP 10)
# ------------------------------------------------------------

def get_cohorts_for_user(username: Optional[str]) -> List[str]:
    """
    Convenience wrapper for the Gradio UI:
    take a simple username string and delegate to list_cohorts_for_user().

    Also logs to the v16 trace file for easier debugging of Refresh Cohorts.
    """
    try:
        trace_log(f"get_cohorts_for_user called with username={username!r}")

        if not username or not str(username).strip():
            # No anonymous access: callers must be logged in.
            return []
        role = USERS.get(username, {}).get("role", "user")
        user = SessionUser(username=str(username).strip(), role=role)

        names = list_cohorts_for_user(user)
        trace_log(f"get_cohorts_for_user returning {names}")
        return names

    except Exception as e:
        trace_log(f"get_cohorts_for_user ERROR for username={username!r}: {e}")
        # No anonymous fallback; return empty to avoid leaking data.
        return []

def get_all_cohorts() -> List[List[str]]:
    """
    For the Admin tab: return [[cohort_name, owner], ...].
    """
    ensure_cohort_table()
    ensure_cohort_meta_table()

    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT cd.cohort_name,
               COALESCE(cm.owner_user_id, 'unknown') AS owner
        FROM cohort_docs cd
        LEFT JOIN cohort_meta cm
          ON cd.cohort_name = cm.cohort_name
        GROUP BY cd.cohort_name
        ORDER BY cd.cohort_name
        """
    )
    rows = cur.fetchall()
    conn.close()

    return [[name, owner] for (name, owner) in rows]

def get_allowed_setup_actions(current_user_dict: dict | None) -> tuple[list[str], str, str, bool]:
    """
    Returns:
      - action_choices
      - action_value (safe default)
      - info_message
      - info_visible
    """
    # Build a SessionUser safely
    if not current_user_dict or not current_user_dict.get("username"):
        user_obj = SessionUser(username=None, role="anonymous")
    else:
        user_obj = SessionUser(
            username=current_user_dict["username"],
            role=current_user_dict.get("role", "user"),
        )

    # Determine if clone is possible for this user
    cloneable = list_cloneable_shared_cohorts_for_user(user_obj)  # you already have / can add this helper

    choices = [ACTION_CREATE, ACTION_APPEND]
    msg = ""
    visible = False

    if cloneable:
        choices.append(ACTION_CLONE)
    else:
        # “Greyed out” effect: omit the choice entirely
        msg = (
            "Clone is unavailable because no shared cohorts have been marked as clone-eligible for your account.\n\n"
            "Ask the cohort owner (or an admin) to enable **Allow cloning** on a shared cohort."
        )
        visible = True

    # Always return a valid selection (prevents the ValueError you hit on logout)
    return choices, DEFAULT_ACTION, msg, visible

# ============================================================


def get_cohort_meta(cohort_name: str) -> dict:
    """Return cohort_meta fields (sharing + description/audience)."""
    ensure_cohort_meta_table()
    cn = (cohort_name or "").strip()
    if not cn:
        return {}
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT cohort_name, owner_user_id, is_shared, allow_clone,
               COALESCE(description,''), COALESCE(intended_audience,''), COALESCE(updated_ts,''), COALESCE(created_ts,'')
        FROM cohort_meta
        WHERE cohort_name = ?
        """,
        (cn,),
    )
    row = cur.fetchone()
    conn.close()
    if not row:
        return {}
    return {
        "cohort_name": row[0],
        "owner_user_id": row[1],
        "is_shared": int(row[2] or 0),
        "allow_clone": int(row[3] or 0),
        "description": row[4] or "",
        "intended_audience": row[5] or "",
        "updated_ts": row[6] or "",
        "created_ts": row[7] or "",
    }

def upsert_cohort_description_and_audience(cohort_name: str, description: str, intended_audience: str):
    """Upsert only description + intended_audience into cohort_meta for v18 EPIC 5.1/5.2."""
    ensure_cohort_meta_table()
    cn = (cohort_name or "").strip()
    if not cn:
        raise ValueError("cohort_name is required")
    desc = (description or "").strip()
    aud = (intended_audience or "").strip()
    now = dt.datetime.now(dt.timezone.utc).isoformat()

    conn = get_db_conn()
    cur = conn.cursor()
    # Ensure row exists (preserve existing sharing fields if present)
    cur.execute("SELECT cohort_name FROM cohort_meta WHERE cohort_name = ?", (cn,))
    exists = cur.fetchone() is not None
    if not exists:
        cur.execute(
            """
            INSERT INTO cohort_meta (cohort_name, owner_user_id, is_shared, allow_clone, created_ts, updated_ts, description, intended_audience)
            VALUES (?, NULL, 0, 0, ?, ?, ?, ?)
            """,
            (cn, now, now, desc, aud),
        )
    else:
        cur.execute(
            """
            UPDATE cohort_meta
            SET description = ?,
                intended_audience = ?,
                updated_ts = ?
            WHERE cohort_name = ?
            """,
            (desc, aud, now, cn),
        )
    conn.commit()
    conn.close()


## CELL 4.7 / STEP 4.7 – FAPs (Frequently Asked Prompts) (v16.1)

In [16]:
# CELL 4.7 / STEP 4.7 – FAPs (Frequently Asked Prompts) (v18)
# Stores curated Frequently Asked Prompts (FAPs) per cohort in SQLite.
# FAPs can be preloaded via admin file upload (CSV/TXT/JSON).

def ensure_demo_prompts_table():
    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS demo_prompts (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            cohort_name TEXT NOT NULL,
            prompt_text TEXT NOT NULL,
            sort_order INTEGER DEFAULT 0,
            display_order INTEGER DEFAULT 0,
            admin_note TEXT,
            created_at TEXT
        )
    """)
    conn.commit()

    # --- migrations ---
    cur.execute("PRAGMA table_info(demo_prompts)")
    cols = {row[1] for row in cur.fetchall()}

    if "display_order" not in cols:
        cur.execute("ALTER TABLE demo_prompts ADD COLUMN display_order INTEGER DEFAULT 0")
    if "admin_note" not in cols:
        cur.execute("ALTER TABLE demo_prompts ADD COLUMN admin_note TEXT")

    conn.commit()
    conn.close()

def clear_demo_prompts_for_cohort(cohort_name: str):
    ensure_demo_prompts_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("DELETE FROM demo_prompts WHERE cohort_name = ?", (cohort_name,))
    conn.commit()
    conn.close()

def list_demo_prompts_for_cohort(cohort_name: str) -> list[str]:
    ensure_demo_prompts_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT prompt_text
        FROM demo_prompts
        WHERE cohort_name = ?
        ORDER BY COALESCE(sort_order,0), id
        """,
        (cohort_name,),
    )
    rows = cur.fetchall()
    conn.close()
    return [r[0] for r in rows if r and r[0]]

def upsert_demo_prompts_for_cohort(cohort_name: str, prompts: list[str], replace: bool = True) -> int:
    """
    Insert demo prompts for a cohort. If replace=True, clears existing prompts first.
    Returns number of prompts inserted.
    """
    ensure_demo_prompts_table()
    cohort = (cohort_name or "").strip()
    if not cohort:
        return 0

    cleaned: list[str] = []
    seen: set[str] = set()
    for p in (prompts or []):
        t = (p or "").strip()
        if not t:
            continue
        if t in seen:
            continue
        seen.add(t)
        cleaned.append(t)

    if replace:
        clear_demo_prompts_for_cohort(cohort)

    if not cleaned:
        return 0

    conn = get_db_conn()
    cur = conn.cursor()
    for i, t in enumerate(cleaned, start=1):
        cur.execute(
            "INSERT INTO demo_prompts (cohort_name, prompt_text, display_order) VALUES (?,?,?)",
            (cohort, t, i),
        )
    conn.commit()
    conn.close()
    return len(cleaned)

def parse_demo_prompts_file(file_path: str) -> list[str]:
    """
    Parse demo prompts from a local file path.
    Supported:
      - .txt: one question per line (blank lines ignored)
      - .csv: one prompt per line OR header with columns 'prompt'/'question'/'text'
      - .json: either list[str] or {"prompts":[...]}
    """
    if not file_path:
        return []
    fp = str(file_path)
    ext = os.path.splitext(fp)[1].lower()
    try:
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            raw = f.read()
    except Exception:
        return []

    if ext == ".json":
        try:
            obj = json.loads(raw)
            if isinstance(obj, list):
                return [str(x) for x in obj]
            if isinstance(obj, dict) and isinstance(obj.get("prompts"), list):
                return [str(x) for x in obj.get("prompts")]
        except Exception:
            return []
        return []

    lines = [ln.strip() for ln in raw.replace("\r\n", "\n").replace("\r", "\n").split("\n")]
    lines = [ln for ln in lines if ln]

    if ext == ".txt":
        return lines

    if ext == ".csv":
        if not lines:
            return []
        header = [h.strip().strip('"').strip("'").lower() for h in lines[0].split(",")]
        if any(h in ("prompt", "question", "text") for h in header):
            col_idx = None
            for k in ("prompt", "question", "text"):
                if k in header:
                    col_idx = header.index(k)
                    break
            out = []
            for row in lines[1:]:
                parts = [p.strip().strip('"').strip("'") for p in row.split(",")]
                if col_idx is not None and col_idx < len(parts):
                    out.append(parts[col_idx])
            return out
        # no header: each row is a prompt (first column)
        out = []
        for row in lines:
            parts = [p.strip().strip('"').strip("'") for p in row.split(",")]
            if parts and parts[0]:
                out.append(parts[0])
        return out

    return lines

# ============================================================


# ----------------------------
# v18 – FAPs (Naming Aliases)
# ----------------------------
# We keep the underlying SQLite table name 'demo_prompts' for backward compatibility,
# but expose 'FAP' naming in the UI and in wrapper helpers.

def ensure_faps_table():
    return ensure_demo_prompts_table()

def list_faps_for_cohort(cohort_name: str) -> list[str]:
    return list_demo_prompts_for_cohort(cohort_name)

def upsert_faps_for_cohort(cohort_name: str, prompts: list[str], replace: bool = True) -> int:
    return upsert_demo_prompts_for_cohort(cohort_name, prompts, replace=replace)

def clear_faps_for_cohort(cohort_name: str) -> None:
    return clear_demo_prompts_for_cohort(cohort_name)



## CELL 5.0 / STEP 5.0 – Model Registry Core (v16_1, with schema migration)

In [17]:
# CELL 5.0 / STEP 5.0 – Model Registry Core (v16_1, with schema migration)

import sqlite3
from typing import List, Dict, Any

def ensure_model_registry_table():
    """
    Ensure the model_registry table exists with the v6_15 schema.
    If an older schema is detected (missing model_id or other key fields),
    we DROP and recreate the table.

    This is safe for the MVP since we don't rely on persistent custom models yet.
    """
    conn = get_db_conn()
    cur = conn.cursor()

    # Does the table exist?
    cur.execute(
        "SELECT name FROM sqlite_master WHERE type='table' AND name='model_registry'"
    )
    row = cur.fetchone()

    if row:
        # Table exists – inspect its columns
        cur.execute("PRAGMA table_info(model_registry)")
        cols_info = cur.fetchall()
        existing_cols = {c[1] for c in cols_info}  # c[1] is the column name

        required_cols = {
            "provider",
            "model_id",
            "display_name",
            "model_type",
            "enabled",
            "is_default",
            "cost_score",
            "latency_score",
            "max_context_tokens",
            "api_base",
            "notes",
        }

        # If the key v16 columns are missing, drop and recreate
        if not required_cols.issubset(existing_cols):
            print("DEBUG: model_registry schema mismatch detected. Dropping old table.")
            cur.execute("DROP TABLE IF EXISTS model_registry")
            conn.commit()

    # (Re)create the table with the correct v16 schema
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS model_registry (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            provider TEXT,
            model_id TEXT,
            display_name TEXT,
            model_type TEXT,
            enabled INTEGER DEFAULT 1,
            is_default INTEGER DEFAULT 0,
            cost_score INTEGER DEFAULT 2,
            latency_score INTEGER DEFAULT 2,
            max_context_tokens INTEGER,
            api_base TEXT,
            notes TEXT
        )
        """
    )
    conn.commit()
    conn.close()


def seed_default_models():
    """
    Seed the registry with standard defaults if empty.
    """
    ensure_model_registry_table()
    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute("SELECT COUNT(*) FROM model_registry")
    count = cur.fetchone()[0]

    if count == 0:
        defaults = [
            # Default Chat Model
            {
                "provider": "openai",
                "model_id": "gpt-4.1-mini",
                "display_name": "GPT-4.1 Mini",
                "model_type": "chat",
                "enabled": 1,
                "is_default": 1,
                "cost_score": 1,
                "latency_score": 1,
                "max_context_tokens": 128_000,
                "api_base": None,
                "notes": "Primary chat model",
            },

            # Additional Chat Models (EPIC 5.2)
            {
                "provider": "openai",
                "model_id": "gpt-5",
                "display_name": "GPT-5",
                "model_type": "chat",
                "enabled": 1,
                "is_default": 0,
                "cost_score": 3,
                "latency_score": 2,
                "max_context_tokens": 128_000,
                "api_base": None,
                "notes": "High-capability chat model",
            },
            {
                "provider": "openai",
                "model_id": "gpt-5-mini",
                "display_name": "GPT-5 Mini",
                "model_type": "chat",
                "enabled": 1,
                "is_default": 0,
                "cost_score": 2,
                "latency_score": 1,
                "max_context_tokens": 128_000,
                "api_base": None,
                "notes": "Lower-cost GPT-5 variant",
            },
            {
                "provider": "openai",
                "model_id": "",
                "display_name": "GPT-5 Chat",
                "model_type": "chat",
                "enabled": 1,
                "is_default": 0,
                "cost_score": 2,
                "latency_score": 1,
                "max_context_tokens": 128_000,
                "api_base": None,
                "notes": "Chat-optimized GPT-5 variant",
            },

            # Default Embedding Model
            {
                "provider": "openai",
                "model_id": "text-embedding-3-small",
                "display_name": "text-embedding-3-small",
                "model_type": "embed",
                "enabled": 1,
                "is_default": 1,
                "cost_score": 1,
                "latency_score": 1,
                "max_context_tokens": None,
                "api_base": None,
                "notes": "Primary embedding model",
            },
        ]

        for row in defaults:
            cur.execute(
                """
                INSERT INTO model_registry (
                    provider, model_id, display_name, model_type,
                    enabled, is_default, cost_score, latency_score,
                    max_context_tokens, api_base, notes
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                (
                    row["provider"],
                    row["model_id"],
                    row["display_name"],
                    row["model_type"],
                    row["enabled"],
                    row["is_default"],
                    row["cost_score"],
                    row["latency_score"],
                    row["max_context_tokens"],
                    row["api_base"],
                    row["notes"],
                ),
            )

        conn.commit()

    conn.close()


def list_models(model_type: str | None = None, only_enabled: bool = True) -> List[Dict[str, Any]]:
    """
    Return raw dict rows from the model_registry table.

    Args:
        model_type: 'chat', 'embed', etc. (optional)
        only_enabled: filter to enabled == 1
    """
    ensure_model_registry_table()
    conn = get_db_conn()
    cur = conn.cursor()

    sql = """
        SELECT provider, model_id, display_name, model_type,
               enabled, is_default, cost_score, latency_score,
               max_context_tokens, api_base, notes
        FROM model_registry
        WHERE 1=1
    """
    params: list[Any] = []

    if model_type:
        sql += " AND model_type = ?"
        params.append(model_type)

    if only_enabled:
        sql += " AND enabled = 1"

    sql += " ORDER BY model_type, is_default DESC, model_id"

    cur.execute(sql, params)
    rows = cur.fetchall()
    conn.close()

    keys = [
        "provider", "model_id", "display_name", "model_type", "enabled",
        "is_default", "cost_score", "latency_score", "max_context_tokens",
        "api_base", "notes",
    ]

    return [dict(zip(keys, r)) for r in rows]


def add_or_update_model(
    model_type: str,
    model_id: str,
    provider: str = "openai",
    display_name: str | None = None,
    enabled: bool = True,
    is_default: bool = False,
    cost_score: int = 2,
    latency_score: int = 2,
    max_context_tokens: int | None = None,
    api_base: str | None = None,
    notes: str | None = None,
) -> None:
    """Upsert a model row into model_registry.

    This is the single, canonical write-path used by:
      - EPIC 5.2 seeding
      - Admin "Model Settings" enable/disable + default selection

    Notes:
      - display_name defaults to model_id
      - enabled/is_default are stored as INTEGER 0/1
      - If is_default=True, this also enforces a single default per model_type.
    """
    ensure_model_registry_table()
    conn = get_db_conn()
    cur = conn.cursor()

    mid = (model_id or "").strip()
    mtype = (model_type or "").strip()
    if not mid or not mtype:
        conn.close()
        raise ValueError("model_type and model_id are required")

    dn = (display_name or mid).strip()

    cur.execute(
        """
        SELECT id FROM model_registry
        WHERE model_type = ? AND model_id = ?
        """,
        (mtype, mid),
    )
    row = cur.fetchone()

    if row:
        cur.execute(
            """
            UPDATE model_registry
            SET provider=?,
                display_name=?,
                enabled=?,
                is_default=?,
                cost_score=?,
                latency_score=?,
                max_context_tokens=?,
                api_base=?,
                notes=?
            WHERE model_type=? AND model_id=?
            """,
            (
                provider,
                dn,
                1 if enabled else 0,
                1 if is_default else 0,
                int(cost_score) if cost_score is not None else 2,
                int(latency_score) if latency_score is not None else 2,
                max_context_tokens,
                api_base,
                notes,
                mtype,
                mid,
            ),
        )
    else:
        cur.execute(
            """
            INSERT INTO model_registry
                (provider, model_id, display_name, model_type, enabled, is_default,
                 cost_score, latency_score, max_context_tokens, api_base, notes)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                provider,
                mid,
                dn,
                mtype,
                1 if enabled else 0,
                1 if is_default else 0,
                int(cost_score) if cost_score is not None else 2,
                int(latency_score) if latency_score is not None else 2,
                max_context_tokens,
                api_base,
                notes,
            ),
        )

    conn.commit()

    # Enforce single default per type (and avoid leaving the DB in a multi-default state)
    if is_default:
        conn.close()
        set_default_model(mtype, mid)
        return

    conn.close()


def set_default_model(model_type: str, model_id: str) -> None:
    """Mark exactly one model as default for a given model_type."""
    ensure_model_registry_table()
    conn = get_db_conn()
    cur = conn.cursor()

    mid = (model_id or "").strip()
    mtype = (model_type or "").strip()
    if not mid or not mtype:
        conn.close()
        raise ValueError("model_type and model_id are required")

    # Clear existing defaults for this type
    cur.execute(
        """
        UPDATE model_registry
        SET is_default = 0
        WHERE model_type = ?
        """,
        (mtype,),
    )

    # Set requested default (ensure it is enabled as well)
    cur.execute(
        """
        UPDATE model_registry
        SET is_default = 1,
            enabled = 1
        WHERE model_type = ? AND model_id = ?
        """,
        (mtype, mid),
    )

    # If the model doesn't exist yet, create it as enabled default.
    if cur.rowcount == 0:
        conn.commit()
        conn.close()
        add_or_update_model(model_type=mtype, model_id=mid, enabled=True, is_default=True)
        return

    conn.commit()
    conn.close()


def load_model_registry() -> List[Dict[str, Any]]:
    """
    Loads all models from the registry and ensures defaults exist.
    """
    ensure_model_registry_table()
    seed_default_models()  # <-- CRITICAL LINE (missing previously)

    conn = get_db_conn()
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()

    cur.execute(
        """
        SELECT
            id,
            provider,
            model_id,
            model_id AS model,          -- backward compatibility
            display_name,
            model_type AS type,         -- backward compatibility
            enabled,
            is_default,
            cost_score,
            latency_score,
            max_context_tokens,
            api_base,
            notes
        FROM model_registry
        ORDER BY model_type, is_default DESC, model_id
        """
    )

    rows = [dict(r) for r in cur.fetchall()]
    conn.close()
    return rows

# ============================================================

## CELL 5.1 / STEP 5.1 – ModelConfig & Lookup Helpers (v16)

In [18]:
# CELL 5.1 / STEP 5.1 – ModelConfig & Lookup Helpers (v16)
from dataclasses import dataclass
from typing import Optional, List, Dict, Any

@dataclass
class ModelConfig:
    model_id: str
    provider: str = "openai"
    display_name: str | None = None
    model_type: str = "chat"   # 'chat', 'embed', 'rerank', etc.
    is_default: bool = False
    enabled: bool = True
    cost_score: int = 2
    latency_score: int = 2
    max_context_tokens: int | None = None
    api_base: str | None = None
    notes: str | None = None

    def as_kwargs(self) -> dict:
        """
        Common kwargs we might pass into a client, e.g. OpenAI.
        For now this is mostly for future extensibility.
        """
        kw = {"model": self.model_id}
        if self.api_base:
            kw["api_base"] = self.api_base
        return kw


def load_model_configs(model_type: str, only_enabled: bool = True) -> list[ModelConfig]:
    """
    Load models from the registry as ModelConfig objects.
    Relies on list_models(...) from STEP 5.0.
    """
    rows = list_models(model_type=model_type, only_enabled=only_enabled)
    configs: list[ModelConfig] = []
    for row in rows:
        configs.append(
            ModelConfig(
                model_id=row["model_id"],
                provider=row["provider"],
                display_name=row["display_name"],
                model_type=row["model_type"],
                is_default=row["is_default"],
                enabled=row["enabled"],
                cost_score=row["cost_score"],
                latency_score=row["latency_score"],
                max_context_tokens=row["max_context_tokens"],
                api_base=row["api_base"],
                notes=row["notes"],
            )
        )
    return configs


def get_default_model(model_type: str) -> Optional[Dict[str, Any]]:
    """
    Low-level helper used by the *Config() helpers below.

    Looks in the model_registry for the default model of a given type.
    Strategy:
      1) Use list_models(model_type, only_enabled=True)
      2) Return the one where is_default = True, if present
      3) Otherwise return the first enabled model
      4) If none exist, return None
    """
    rows = list_models(model_type=model_type, only_enabled=True)
    if not rows:
        return None

def get_default_chat_model_id() -> str:
    """Return the registry default chat model_id (fallback to gpt-4.1-mini)."""
    row = get_default_model("chat")
    if row and row.get("model_id"):
        return str(row["model_id"])
    return "gpt-4.1-mini"


    for row in rows:
        if row.get("is_default"):
            return row

    # Fallback: first enabled of that type
    return rows[0]


def get_default_chat_model_config() -> ModelConfig:
    """
    Returns a ModelConfig for the default chat model.
    If none is explicitly set, falls back to the first enabled chat model.
    """
    default_row = get_default_model("chat")
    if default_row:
        return ModelConfig(
            model_id=default_row["model_id"],
            provider=default_row["provider"],
            display_name=default_row["display_name"],
            model_type=default_row["model_type"],
            is_default=default_row["is_default"],
            enabled=default_row["enabled"],
            cost_score=default_row["cost_score"],
            latency_score=default_row["latency_score"],
            max_context_tokens=default_row["max_context_tokens"],
            api_base=default_row["api_base"],
            notes=default_row["notes"],
        )

    # Fallback: first enabled chat model
    configs = load_model_configs("chat", only_enabled=True)
    if configs:
        return configs[0]

    # Last resort hard-coded default (should not happen due to seeding)
    return ModelConfig(
        model_id="gpt-4.1-mini",
        provider="openai",
        display_name="GPT-4.1 Mini (fallback)",
        model_type="chat",
        is_default=True,
        enabled=True,
        cost_score=1,
        latency_score=1,
        max_context_tokens=128_000,
    )


def get_default_embed_model_config() -> ModelConfig:
    """
    Returns a ModelConfig for the default embedding model.
    """
    default_row = get_default_model("embed")
    if default_row:
        return ModelConfig(
            model_id=default_row["model_id"],
            provider=default_row["provider"],
            display_name=default_row["display_name"],
            model_type=default_row["model_type"],
            is_default=default_row["is_default"],
            enabled=default_row["enabled"],
            cost_score=default_row["cost_score"],
            latency_score=default_row["latency_score"],
            max_context_tokens=default_row["max_context_tokens"],
            api_base=default_row["api_base"],
            notes=default_row["notes"],
        )

    # Fallback: first enabled embed model
    configs = load_model_configs("embed", only_enabled=True)
    if configs:
        return configs[0]

    # Last resort hard-coded default
    return ModelConfig(
        model_id="text-embedding-3-small",
        provider="openai",
        display_name="text-embedding-3-small (fallback)",
        model_type="embed",
        is_default=True,
        enabled=True,
        cost_score=1,
        latency_score=1,
    )
def list_chat_models() -> list[str]:
    """Return enabled chat models for the Ask tab dropdown.

    Source of truth:
      1) If the EPIC 5.2 model registry exists, return enabled chat models from it.
      2) Otherwise, fall back to the preloaded EPIC52_CHAT_MODELS list.
    """
    # Preferred: EPIC 5.2 registry (persisted enabled flags)
    try:
        enabled = [m.get("model_id") for m in (list_models("chat", only_enabled=True) or []) if m.get("model_id")]
        enabled = [m for m in enabled if m]
        if enabled:
            return enabled
    except Exception:
        pass

    # Fallback: static list
    return list(EPIC52_CHAT_MODELS)

def list_all_chat_models() -> list[str]:
    """
    Returns the complete set of chat models that admins can enable/disable.
    This is the EPIC 5.2 curated list (not an API discovery).
    """
    return EPIC52_CHAT_MODELS[:]



# ============================================================
# EPIC 5.2 — Robust Chat-Model Registry Helpers (override/fix)
# ============================================================
# NOTE: These helpers intentionally override any earlier partial implementations.
EPIC52_CHAT_MODELS = ["gpt-4.1-mini", "gpt-5", "gpt-5-mini", ""]

def list_all_chat_models() -> list[str]:
    """Canonical list of chat models exposed in EPIC 5.2."""
    return list(EPIC52_CHAT_MODELS)

def seed_epic52_chat_models(default_model_id: str = "gpt-4.1-mini") -> None:
    """Ensure EPIC 5.2 models exist in model_registry and set a sane default."""
    try:
        ensure_model_registry_table()
    except Exception as e:
        trace_log(f"seed_epic52_chat_models ensure table ERROR: {e}")
        return

    for mid in EPIC52_CHAT_MODELS:
        try:  # Prefer existing helper if present
            add_or_update_model(model_type="chat", model_id=mid, enabled=True, is_default=(mid == default_model_id))
        except Exception:
            # Fallback: write directly
            try:
                conn = get_db_conn()
                cur = conn.cursor()
                cur.execute(
                    """
                    INSERT INTO model_registry (model_type, model_id, enabled, is_default)
                    VALUES (?, ?, ?, ?)
                    ON CONFLICT(model_type, model_id) DO UPDATE SET
                        enabled=excluded.enabled,
                        is_default=excluded.is_default
                    """,
                    ("chat", mid, 1, 1 if mid == default_model_id else 0),
                )
                conn.commit()
                conn.close()
            except Exception as e:
                trace_log(f"seed_epic52_chat_models DB write ERROR model={mid}: {e}")

    # Ensure only one default (if helper exists)
    try:
        set_default_model("chat", default_model_id)
    except Exception:
        pass

def get_allowed_chat_models() -> list[str]:
    """Enabled chat models, falling back to EPIC 5.2 defaults."""
    try:
        rows = list_models("chat", only_enabled=True)
    except Exception as e:
        trace_log(f"get_allowed_chat_models list_models ERROR: {e}")
        return list(EPIC52_CHAT_MODELS)

    if not rows:
        return list(EPIC52_CHAT_MODELS)

    out: list[str] = []
    for r in rows:
        if isinstance(r, dict):
            mid = r.get("model_id")
        else:
            mid = r[1] if isinstance(r, (list, tuple)) and len(r) > 1 else None
        if mid:
            out.append(str(mid))
    return out or list(EPIC52_CHAT_MODELS)

def get_default_model(model_type: str = "chat") -> dict | None:
    """Return the default model row (enabled preferred); otherwise first enabled."""
    try:
        rows = list_models(model_type, only_enabled=True)
    except Exception as e:
        trace_log(f"get_default_model list_models ERROR: {e}")
        rows = []

    if not rows:
        return None

    for r in rows:
        if isinstance(r, dict) and r.get("is_default"):
            return r

    return rows[0] if isinstance(rows[0], dict) else None

def get_default_chat_model_id() -> str:
    row = get_default_model("chat")
    if isinstance(row, dict) and row.get("model_id"):
        return str(row["model_id"])
    return EPIC52_CHAT_MODELS[0]

# Seed models on import/run (idempotent)
try:
    seed_epic52_chat_models(default_model_id="gpt-4.1-mini")
except Exception as _e:
    trace_log(f"EPIC 5.2 seed models ERROR: {_e}")

# ============================================================
# EPIC 5.2 — list_chat_models() override (Ask tab dropdown)
# ============================================================
def list_chat_models_static() -> list[str]:
    """Return enabled chat models for the Ask tab dropdown."""
    try:
        return get_allowed_chat_models()
    except Exception:
        return ["gpt-4.1-mini", "gpt-5", "gpt-5-mini", ""]


# Ensure EPIC 5.2 chat models exist on startup
try:
    seed_epic52_chat_models()
except Exception as e:
    trace_log(f"seed_epic52_chat_models WARNING: {e}")


## CELL 5.2 / STEP 5.2 – RAG Retrieval Over a Cohort (v16)

In [19]:
# CELL 5.2 / STEP 5.2 – RAG Retrieval Over a Cohort (v16)

# ============================================================
# EPIC 2.3 (v17_1) – Retrieval Quality Controls

# ============================================================

RAG_TOPK_SMALL = 3
RAG_TOPK_DEFAULT = 5
RAG_TOPK_LARGE = 8

# Since you normalize embeddings, FAISS scores here are cosine-like similarity.
# Start conservative; tune after observing scores in Why panel.
RAG_SCORE_THRESHOLD_DEFAULT = 0.25

# ============================================================
# EPIC 4.0.1 (v17_4) – Optional Query Rewrite for Retrieval
# ============================================================
EPIC40_QUERY_REWRITE_ENABLED = True  # set False to disable
EPIC40_QUERY_REWRITE_MAX_TOKENS = 80


# Optional diversity control (leave None to disable)
RAG_MAX_CHUNKS_PER_DOC_DEFAULT = 2


def choose_top_k_for_query(query: str) -> int:
    """
    Heuristic: choose top_k based on query length/complexity.
    Simple + explainable; no extra dependencies.
    """
    q = (query or "").strip().lower()
    if not q:
        return RAG_TOPK_DEFAULT

    words = q.split()
    n_words = len(words)

    # "Complex" indicators
    complex_markers = [
        "compare", "contrast", "summarize", "synthesize", "evaluate",
        "pros and cons", "tradeoff", "why", "how", "explain",
        "include", "including", "as well as"
    ]
    has_complex = any(m in q for m in complex_markers) or q.count("?") >= 2

    if n_words <= 10 and not has_complex:
        return RAG_TOPK_SMALL

    if n_words >= 25 or has_complex:
        return RAG_TOPK_LARGE

    return RAG_TOPK_DEFAULT




def build_context_from_index(
    api_key: str,
    chat_model: str,
    embed_model: str,
    cohort_name: str,
    query: str,
    top_k: int = 5,
    score_threshold: float | None = None,
    max_chunks_per_doc: int | None = None,
):

    # EPIC 3.3.1 — Session-scope questions may run without a cohort
    if not cohort_name:
        return "", [], []

    """
    Given a cohort and query:
    - Load all documents for that cohort
    - For each doc's index, perform similarity search
    - Aggregate top_k results across docs
    Returns:
      - concatenated context string
      - list of (doc_name, rank, score) for citations
    """

    ensure_docs_table()
    resolved_chat, resolved_embed = resolve_models(chat_model, embed_model)
    client = build_openai_client(api_key)

    # 1. Load all docs & their index references
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT doc_name, index_id
        FROM documents
        WHERE cohort_name = ?
        """,
        (cohort_name,),
    )
    rows = cur.fetchall()
    conn.close()

    if not rows:
        return "", [], []

    # 2. Embed query once
    q_embed_resp = client.embeddings.create(model=resolved_embed, input=[query])
    q_vec = np.array(q_embed_resp.data[0].embedding, dtype="float32")
    q_vec = q_vec / (np.linalg.norm(q_vec) + 1e-10)

    all_hits = []  # (doc_name, idx, score, text)

    # 3. Perform similarity search in each doc index
    for doc_name, index_id in rows:
        try:
            index = load_index(index_id)
            meta = load_metadata(index_id)  # {"chunks": [...]}
        except Exception:
            continue

        D, I = index.search(q_vec[np.newaxis, :], top_k)
        scores = D[0]
        idxs = I[0]

        for score, idx in zip(scores, idxs):
            if idx < 0:
                continue

            score_f = float(score)

            # EPIC 2.3.2 — Similarity thresholding (skip weak hits)
            if score_threshold is not None and score_f < float(score_threshold):
                continue

            chunks = meta.get("chunks", [])
            if idx >= len(chunks):
                continue

            text_chunk = chunks[idx]
            if not text_chunk or not str(text_chunk).strip():
                continue

            all_hits.append((doc_name, idx, score_f, text_chunk))


    if not all_hits:
        return "", [], []

    # 4. Sort & select top results
    # 4. Sort & select top results
    all_hits.sort(key=lambda x: x[2], reverse=True)

    # EPIC 2.3 — Optional diversity control (limit chunks per document)
    if max_chunks_per_doc is not None and max_chunks_per_doc > 0:
        selected = []
        per_doc_counts = {}
        for (dn, idx, sc, txt) in all_hits:
            per_doc_counts[dn] = per_doc_counts.get(dn, 0) + 1
            if per_doc_counts[dn] > max_chunks_per_doc:
                continue
            selected.append((dn, idx, sc, txt))
            if len(selected) >= top_k:
                break
        top_hits = selected
    else:
        top_hits = all_hits[:top_k]


    context_parts = []
    citations = []

    for rank, (doc_name, idx, score, text) in enumerate(top_hits, start=1):
        header = f"[{rank}] From {doc_name} (chunk #{idx}, score={score:.3f})"
        context_parts.append(header + "\n" + text)
        citations.append((doc_name, rank, score))

    context = "\n\n".join(context_parts)
    return context, citations, top_hits

# ============================================================
# UPDATED answer_with_rag() — Now uses Routing Brain (v16)

def answer_with_rag(
    api_key: str,
    chat_model: str,
    embed_model: str,
    cohort_name: str,
    query: str,
    system_prompt: str = "",
    user_pref: str | None = None,
    top_k: int | None = None,
    score_threshold: float | None = None,
    max_chunks_per_doc: int | None = None,
    provenance_mode: str = 'Off',  # EPIC 3.4
    session_mem: dict | None = None,
):
    """
    Perform RAG retrieval and return:
        - answer_markdown
        - raw_answer_text
        - model_used
        - why_md_text
        - footnote_text
    """

    answer_source = "Cohort Context"  # EPIC 3.3 — provenance label


    # EPIC 2.3 — Retrieval policy
    chosen_top_k = top_k if top_k is not None else choose_top_k_for_query(query)
    chosen_threshold = score_threshold if score_threshold is not None else RAG_SCORE_THRESHOLD_DEFAULT
    chosen_max_per_doc = (
        max_chunks_per_doc
        if max_chunks_per_doc is not None
        else RAG_MAX_CHUNKS_PER_DOC_DEFAULT
    )

    # 1) Build RAG context
    context, citations, top_hits = build_context_from_index(
        api_key=api_key,
        chat_model=chat_model,
        embed_model=embed_model,
        cohort_name=cohort_name,
        query=query,
        top_k=chosen_top_k,
        score_threshold=chosen_threshold,
        max_chunks_per_doc=chosen_max_per_doc,
    )

    if not context:


        # EPIC 3.2 Option 1: If session memory is enabled and has turns, fall back to session memory only.


        if session_mem and session_mem.get('enabled') and session_mem.get('turns'):


            context = format_session_memory_prompt(session_mem.get('turns') or [])
            answer_source = "Session Memory (fallback)"  # EPIC 3.3


            citations, top_hits = [], []


        else:


            return (


                "I could not find any context for this query in the selected cohort.",


                "",


                "N/A",


                "",


                "",


            )

    # 2) Default system prompt
    if not system_prompt:
        system_prompt = (
            "You are a helpful assistant answering questions based on the provided context.\n"
            "If the answer cannot be found in the context, say you do not know."
        )


    # EPIC 3.4 — Provenance-mode prompt shaping
    _prov = (provenance_mode or "Off").strip()
    _prov_l = _prov.lower()
    if _prov_l.startswith("explain"):
        system_prompt += (
            "\n\n"
            "EPIC 3.4 — When you answer, separate information by source.\n"
            "Provide two labeled sections:\n"
            "1) 'From retrieved documents' — only statements supported by the provided context. Tag each bullet with [DOC].\n"
            "2) 'Model reasoning / general knowledge' — any inference, interpretation, or external knowledge. Tag each bullet with [MODEL].\n"
            "Do not present [MODEL] statements as if they were stated in the documents.\n"
        )
    elif "strict" in _prov_l:
        system_prompt += (
            "\n\n"
            "EPIC 3.4 — Strict Evidence Only: Use ONLY the provided context.\n"
            "If the context does not support the answer, say so clearly and do not add external facts.\n"
        )

    # 3) Construct chat messages

        # 2.5) Response Constraints (MUST FOLLOW)
    # Enforce user-requested output constraints such as "only one", "exactly N", etc.
    # Keep this in the system prompt so it applies regardless of provenance mode.
    constraints_block = """Response Constraints (MUST FOLLOW):
1) If the user requests a specific count (e.g., 'one', '2', 'top 3'), return exactly that count—no more.
2) If the user requests 'only X' (e.g., only an identifier), do not include additional commentary, explanation, or extra items.
3) If multiple candidates exist, select the single best candidate using the strongest support in the retrieved context.
4) If constraints conflict with providing a correct answer from context, say: 'Insufficient information in the provided context.' Do not speculate.
""".strip()

    system_prompt = (system_prompt or "").rstrip() + "\n\n" + constraints_block

    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": (
                """Use the retrieved context to answer the question.
You may use prior session context only to resolve references (e.g., "it", "they", "that council").

If the context explicitly states the answer, quote or paraphrase it.
If the context implies an answer at a high level but does not list details, provide a clearly labeled:
"Summary based on context:" and do not invent specifics.
In the summary, only describe what the retrieved context suggests; do not add general background or “typically” statements.
Only say "I do not know" if neither an explicit answer nor a reasonable high-level summary is possible.

=== CONTEXT START ===
"""
                f"{context}\n"
                """=== CONTEXT END ===

QUESTION: """
                f"{query}"
            ),
        },
    ]

# 4) Call the routing brain
    answer_text, raw_answer_text, model_used = call_chat_model(
        api_key=api_key,
        messages=messages,
        task_type="rag_answer",
        user_pref=user_pref,
        context_size=len(context),
    )

    # 5) Build answer markdown with citations + model info
    md = answer_text + "\n\n---\n\n**Cited sources:**\n"
    for doc_name, rank, score in citations:
        md += f"- [{rank}] `{doc_name}` (score={score:.3f})\n"
    md += f"\n\n**Model Used:** `{model_used}`\n"

    # EPIC 2.1/2.2 — Why panel + footnote
    doc_names = [h[0] for h in top_hits] if top_hits else []
    unique_docs = sorted(set(doc_names))

    why_lines = []
    # EPIC 3.3 — show whether answer used cohort context or session memory
    why_lines.append(f"**Answer source:** {answer_source}")
    why_lines.append(f"**Model used:** `{model_used}`")
    why_lines.append(f"**Provenance mode:** `{(provenance_mode or 'Off')}`")  # EPIC 3.4
    why_lines.append(
        f"**Retrieved chunks:** {len(top_hits) if top_hits else 0} "
        f"(requested_top_k={chosen_top_k}, score_threshold={chosen_threshold}, max_per_doc={chosen_max_per_doc})"
    )
    why_lines.append("")

    if top_hits:
        why_lines.append("**Top retrieved evidence:**")
        for rank, (dn, idx, score, _txt) in enumerate(top_hits, start=1):
            _ex = (_txt or "").strip().replace("\n", " ")
            _ex = (_ex[:240] + "…") if len(_ex) > 240 else _ex
            why_lines.append(f"- {rank}. `{dn}` • chunk `{idx}` • score `{score:.4f}`")
            if _ex:
                why_lines.append(f"  - _Excerpt:_ “{_ex}”")  # EPIC 3.4
    else:
        why_lines.append("_No retrieval evidence available (no chunks returned)._")

    why_md_text = "\n".join(why_lines)

    footnote_text = (
        f"*Grounded in {len(unique_docs)} doc(s) • Retrieved {len(top_hits) if top_hits else 0} chunk(s) • "
        f"Model: `{model_used}`*"
    )

    return md, raw_answer_text, model_used, why_md_text, footnote_text

    # Build session memory prompt (EPIC 3.1)
    mem_turns = (session_mem or {}).get("turns") or []
    mem_prompt = format_session_memory_prompt(mem_turns)

    # Base system prompt (keep your existing default behavior, but make it explicit)
    base_prompt = (
        "You are a helpful assistant answering questions using retrieved context.\n"
        "You may use prior session context only to resolve references "
        "(such as pronouns or shorthand).\n\n"
        "If the context explicitly states an answer, present it as such.\n"
        "If the context does not list details explicitly but clearly describes a structure "
        "or role at a high level, provide a labeled summary of what it suggests.\n"
        "You MUST clearly label summarized or interpreted information.\n"
        "Do NOT present inferred details as direct statements from the context.\n\n"
        "Only say you do not know if neither explicit information nor a reasonable "
        "high-level summary is possible."
    )

    system_prompt = base_prompt if not mem_prompt else (base_prompt + "\n\n" + mem_prompt)


def epic40_rewrite_query_for_retrieval(api_key: str, chat_model: str, question: str) -> str:
    """EPIC 4.0.1: Rewrite user question into a retrieval-optimized query.
    Falls back to the original question on any failure.

    Notes:
      - Some newer models reject `max_tokens` and require `max_completion_tokens`.
        We try the broad-compatible call first, then fall back only on that error.
    """
    q = (question or "").strip()
    if not api_key or not q:
        return q

    try:
        client = build_openai_client(api_key)
        resolved_chat, _ = resolve_models(
            chat_model,
            EMBED_MODEL_DEFAULT if "EMBED_MODEL_DEFAULT" in globals() else "text-embedding-3-small",
        )

        base_kwargs = {
            "model": resolved_chat,
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "Rewrite the user question into a short, keyword-rich search query "
                        "for retrieving relevant passages. Output ONLY the rewritten query."
                    ),
                },
                {"role": "user", "content": q},
            ],
        }
        max_out = EPIC40_QUERY_REWRITE_MAX_TOKENS

        try:
            resp = client.chat.completions.create(
                **base_kwargs,
                temperature=0.0,
                max_tokens=max_out,
            )
        except Exception as e:
            msg = str(e)
            if ("max_tokens" in msg) and ("max_completion_tokens" in msg):
                resp = client.chat.completions.create(
                    **base_kwargs,
                    max_completion_tokens=max_out,
                )
            else:
                raise

        out = (resp.choices[0].message.content or "").strip()
        if len(out) < 3:
            return q
        return out

    except Exception:
        return q

def answer_question_over_cohort(
    api_key: str,
    username: str,
    cohort_name: str,
    question: str,
    model_id: str,
    provenance_mode: str = "Off",
    system_prompt: str = "",
    session_mem: dict | None = None,
):
    """
    Wrapper for the RAG retrieval + LLM step.
    Returns:
      (answer_markdown, raw_answer_text, model_used, why_md_text, footnote_text)
    """

    # Normalize inputs
    final_question = (question or "").strip()
    cohort_name = (cohort_name or "").strip()
    chat_model, embed_model = resolve_models(model_id, EMBED_MODEL_DEFAULT)

    if not final_question:
        return "❌ Please enter a question.", "", chat_model, "", ""

    # EPIC 4.0.1 — build retrieval query (optional rewrite)
    retrieval_query = epic40_build_retrieval_query(api_key, chat_model, final_question)

    # If we have a cohort, do RAG
    if cohort_name:
        return answer_with_rag(
            api_key=api_key,
            chat_model=chat_model,
            embed_model=embed_model,
            cohort_name=cohort_name,
            query=retrieval_query,
            system_prompt=system_prompt or "",
            user_pref=model_id,
            provenance_mode=provenance_mode or "Off",
            session_mem=session_mem,
        )

    # Cohortless path (allowed only when UI lets it through)
    # Use session memory prompt if available
    mem_turns = (session_mem or {}).get("turns") or []
    mem_prompt = format_session_memory_prompt(mem_turns) if mem_turns else ""

    base_prompt = (system_prompt or "").strip() or (
        "You are a helpful assistant. Answer the user's question clearly and accurately."
    )
    sys_prompt = base_prompt if not mem_prompt else (base_prompt + "\n\n" + mem_prompt)

    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": final_question},
    ]

    answer_text, raw_answer_text, model_used = call_chat_model(
        api_key=api_key,
        messages=messages,
        task_type="rag_answer",
        user_pref=model_id,
        context_size=len(final_question),
        dry_run=False,
    )

    answer_md = (answer_text or raw_answer_text or "").strip()

    why_md = "\n".join(
        [
            "**Answer source:** No cohort selected (no document retrieval).",
            f"**Model used:** `{model_used}`",
            f"**Provenance mode:** `{(provenance_mode or 'Off')}`",
        ]
    )
    footnote = "No cited sources available because no cohort was selected."

    return answer_md, (raw_answer_text or ""), model_used, why_md, footnote


# ============================================================
# EPIC 4.0.1 – Build retrieval query (optional rewrite)
# ============================================================
# EPIC 4.0.1 – Retrieval query builder helper
def epic40_build_retrieval_query(
    api_key: str,
    chat_model: str,
    question: str,
) -> str:
    """
    Build the retrieval query from the user question.
    Optionally applies EPIC 4.0.1 query rewriting.
    """
    retrieval_query = (question or "").strip()
    if EPIC40_QUERY_REWRITE_ENABLED and retrieval_query:
        retrieval_query = epic40_rewrite_query_for_retrieval(
            api_key, chat_model, retrieval_query
        )
    return retrieval_query


# ============================================================

In [20]:
# CELL 5.9 / STEP 5.9 – EPIC 4.1 Evaluation Harness (Admin/Notebook)

import json
from uuid import uuid4
from datetime import datetime as _dt

def _now_iso():
    return _dt.utcnow().isoformat(timespec="seconds") + "Z"

def epic41_create_eval_set(name: str, description: str = "") -> str:
    eval_set_id = str(uuid4())
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        "INSERT INTO eval_sets (eval_set_id, name, description, created_at) VALUES (?, ?, ?, ?)",
        (eval_set_id, name.strip(), description.strip(), _now_iso()),
    )
    conn.commit()
    conn.close()
    return eval_set_id

def epic41_add_eval_question(
    eval_set_id: str,
    question: str,
    cohort_name: str | None = None,
    provenance_mode: str = "Explain Sources",
    expect_evidence: bool = True,
    expect_refusal: bool = False,
) -> str:
    eval_q_id = str(uuid4())
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """INSERT INTO eval_questions
            (eval_q_id, eval_set_id, cohort_name, question, provenance_mode, expect_evidence, expect_refusal, created_at)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
        (
            eval_q_id,
            eval_set_id,
            cohort_name,
            question.strip(),
            provenance_mode,
            1 if expect_evidence else 0,
            1 if expect_refusal else 0,
            _now_iso(),
        ),
    )
    conn.commit()
    conn.close()
    return eval_q_id

def epic41_list_eval_sets():
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("SELECT eval_set_id, name, description, created_at FROM eval_sets ORDER BY created_at DESC")
    rows = cur.fetchall()
    conn.close()
    return rows

def epic41_list_eval_questions(eval_set_id: str):
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """SELECT eval_q_id, cohort_name, question, provenance_mode, expect_evidence, expect_refusal
             FROM eval_questions WHERE eval_set_id=? ORDER BY created_at ASC""",
        (eval_set_id,),
    )
    rows = cur.fetchall()
    conn.close()
    return rows

def epic37_log_trace_event(
    user_id: str | None,
    cohort_name: str | None,
    model_id: str | None,
    provenance_mode: str | None,
    memory_scope: str | None,
    memory_used: bool,
    rag_used: bool,
    retrieved_k: int | None,
    retrieval_query: str | None,
    note: str = "",
):
    try:
        conn = get_db_conn()
        cur = conn.cursor()
        cur.execute(
            """INSERT INTO rag_trace_events
               (trace_id, created_at, user_id, cohort_name, model_id, provenance_mode, memory_scope, memory_used, rag_used, retrieved_k, retrieval_query, note)
               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            (
                str(uuid4()),
                _now_iso(),
                user_id,
                cohort_name,
                model_id,
                provenance_mode,
                memory_scope,
                1 if memory_used else 0,
                1 if rag_used else 0,
                retrieved_k,
                retrieval_query,
                note[:500],
            ),
        )
        conn.commit()
        conn.close()
    except Exception:
        pass

def epic41_run_eval_set(
    eval_set_id: str,
    api_key: str,
    username: str,
    default_cohort: str | None,
    model_id: str,
    session_mem: dict | None,
) -> str:
    """Runs eval questions by calling answer_question_over_cohort (EPIC 3.4/3.5 pipeline).
    Produces a markdown summary + stores a JSON summary row.
    """
    qs = epic41_list_eval_questions(eval_set_id)
    if not qs:
        return "No questions found for this eval set."

    results = []
    hit_retrieval = 0
    refused_ok = 0
    total = len(qs)

    for (eval_q_id, cohort_name, q, prov_mode, expect_evidence, expect_refusal) in qs:
        c = cohort_name or default_cohort
        # If no cohort and strict, expect refusal
        answer_md, why_md, foot_md = answer_question_over_cohort(
            api_key=api_key,
            username=username,
            cohort_name=c or "",
            question=q,
            model_id=model_id,
            provenance_mode=prov_mode,
            session_mem=session_mem,
        )

        # Heuristics from Why panel
        rag_used = ("Retrieved" in (why_md or "")) or ("chunks" in (why_md or "").lower())
        if rag_used:
            hit_retrieval += 1

        refused = ("could not find any context" in (answer_md or "").lower()) or ("strict evidence" in (answer_md or "").lower()) or ("no supporting" in (answer_md or "").lower())
        if bool(expect_refusal) == refused:
            refused_ok += 1

        results.append({
            "eval_q_id": eval_q_id,
            "cohort_name": c,
            "question": q,
            "provenance_mode": prov_mode,
            "answer_head": (answer_md or "")[:200],
            "rag_used": rag_used,
            "refused": refused,
            "expect_refusal": bool(expect_refusal),
        })

    summary = {
        "total": total,
        "retrieval_hit_rate": hit_retrieval / max(total, 1),
        "refusal_match_rate": refused_ok / max(total, 1),
        "results": results,
    }

    # persist run
    conn = get_db_conn()
    cur = conn.cursor()
    run_id = str(uuid4())
    cur.execute(
        "INSERT INTO eval_runs (eval_run_id, eval_set_id, model_id, created_at, summary_json) VALUES (?, ?, ?, ?, ?)",
        (run_id, eval_set_id, model_id, _now_iso(), json.dumps(summary)),
    )
    conn.commit()
    conn.close()

    # markdown report
    md = []
    md.append(f"## EPIC 4.1 Eval Run\n")
    md.append(f"- **Eval Set:** `{eval_set_id}`\n- **Model:** `{model_id}`\n- **Total Questions:** {total}\n")
    md.append(f"- **Retrieval hit rate (heuristic):** {summary['retrieval_hit_rate']:.2%}\n")
    md.append(f"- **Refusal correctness:** {summary['refusal_match_rate']:.2%}\n")
    md.append("\n---\n")
    for r in results:
        md.append(f"### Q: {r['question']}")
        md.append(f"- Cohort: `{r['cohort_name']}` | Provenance: `{r['provenance_mode']}`")
        md.append(f"- RAG used: `{r['rag_used']}` | Refused: `{r['refused']}` (expected `{r['expect_refusal']}`)")
        md.append(f"- Answer head: {r['answer_head']}")
        md.append("")
    return "\n".join(md)

print("✅ STEP 5.9 loaded: EPIC 4.1 eval harness helpers available.")

✅ STEP 5.9 loaded: EPIC 4.1 eval harness helpers available.


## CELL 6 / STEP 6 – Build Cohort from Uploaded Docs (v16)

In [21]:
# CELL 6 / STEP 6 – Build Cohort from Uploaded Docs (v16)
def build_cohort_from_files(
    api_key: str,
    embed_model: str,
    cohort_name: str,
    files: List[Any],
    overwrite_existing: bool = False,
) -> str:
    """Ingest a list of uploaded files into a *single* cohort.

    For each file we:
      - load the text
      - chunk it
      - embed with the given embedding model
      - build & save a FAISS index
      - save metadata (including the chunks)
      - register the document in the SQLite `documents` table

    Returns a human-readable summary string for the UI.
    """
    ensure_docs_table()
    ensure_cohort_table()
    os.makedirs(INDEX_DIR, exist_ok=True)

    success_count = 0
    total_chunks = 0
    messages: List[str] = []

    for file_obj in files:
        try:
            # 1) Load text & derive a stable doc_name
            text, doc_name = load_file_to_text(file_obj)
            # Optional overwrite-by-filename (MVP): if a doc with the same name exists in this cohort,
            # remove prior DB rows + on-disk index artifacts, then rebuild this doc's index.
            if overwrite_existing:
                try:
                    removed = overwrite_document_in_cohort(cohort_name=cohort_name, doc_name=doc_name)
                    if removed:
                        messages.append(f"ℹ️ {doc_name}: existing document overwritten; rebuilding index.")
                except Exception as e:
                    trace_log(f"overwrite_document_in_cohort WARNING cohort={cohort_name} doc={doc_name}: {e}")

            if not text or not text.strip():
                messages.append(f"⚠️ {doc_name}: no extractable text, skipped.")
                continue

            # 2) Chunk
            chunks = chunk_text(text)
            if not chunks:
                messages.append(f"⚠️ {doc_name}: produced 0 chunks, skipped.")
                continue

            # 3) Embed
            vectors = embed_texts(
                api_key=api_key,
                embed_model=embed_model,
                texts=chunks,
            )

            # 4) Build FAISS index
            index = build_faiss_index(vectors)

            # 5) Save index & metadata
            index_id = str(uuid4())
            save_index(index, index_id)

            meta = {
                "cohort_name": cohort_name,
                "doc_name": doc_name,
                "chunks": chunks,
                "embed_model": embed_model,
            }
            save_metadata(index_id, meta)

            # 6) Register in SQLite
            register_document(
                doc_name=doc_name,
                cohort_name=cohort_name,
                index_id=index_id,
                n_chunks=len(chunks),
                embed_model=embed_model,
            )

            success_count += 1
            total_chunks += len(chunks)
            messages.append(f"✅ {doc_name}: {len(chunks)} chunks embedded.")

        except Exception as e:
            # Best-effort error capture per-file
            name = getattr(file_obj, "name", str(file_obj))
            messages.append(f"❌ {name}: {e}")

    if success_count == 0:
        detail = "\n".join(messages) if messages else ""
        return "❌ No documents were successfully processed." + (f"\n{detail}" if detail else "")

    summary = (
        f"✅ Built cohort '{cohort_name}' with {success_count} document(s) "
        f"and {total_chunks} total chunks."
    )
    if messages:
        summary += "\n" + "\n".join(messages)
    return summary


def build_cohort_index(
    api_key: str,
    cohort_name: str,
    files: List[Any],
    owner: Optional[str] = None,
) -> str:
    """
    High-level helper used by the Gradio UI (v16):

      - Selects default embedding model (from model registry)
      - Builds the cohort
      - Stores cohort ownership
    """
    if not api_key or not api_key.strip():
        return "❌ OpenAI API key is required."

    if not cohort_name or not cohort_name.strip():
        return "❌ Cohort name is required."

    if (not owner) or (not str(owner).strip()) or (str(owner).strip().lower() in ("anonymous","none","null")):
        return "❌ You must be logged in to create a cohort (owner is required)."

    if not files:
        return "❌ Please upload at least one file."

    # ✔️ NEW — correct default embedding model call
    embed_cfg = get_default_embed_model_config()
    embed_model = embed_cfg.model_id

    # Build FAISS index + metadata
    result_msg = build_cohort_from_files(
        api_key=api_key,
        embed_model=embed_model,
        cohort_name=cohort_name.strip(),
        files=files,
    )

    # Save cohort owner metadata (required)


    # Save cohort owner metadata (required)
    try:
        role = USERS.get(owner, {}).get("role", "user")
        user_obj = SessionUser(username=str(owner).strip(), role=role)
        set_cohort_owner(cohort_name.strip(), user_obj)
    except Exception as e:
        trace_log(f"build_cohort_index set_cohort_owner ERROR: {e}")
        return f"❌ Failed to set cohort owner (cohort will not be treated as valid): {e}"

    return result_msg

# ============================================================

## CELL 6.1 / STEP 6.1 – Routing Brain (v16)

In [22]:
def call_chat_model(*args, **kwargs):
    """
    EPIC 5.2 — Unified model call used by the app's routing interface.

    Backward-compatible with v18 call sites:
      call_chat_model(api_key=..., messages=..., task_type=..., user_pref=...)

    Returns:
      (answer_text, raw_answer_text, model_used)

    Model constraints handled:
      - GPT-5 family uses max_completion_tokens (not max_tokens)
      - gpt-5-mini does NOT support custom temperature values; omit temperature entirely
    """

    api_key = kwargs.get("api_key")
    messages = kwargs.get("messages")
    task_type = (kwargs.get("task_type") or "rag_answer").strip()
    user_pref = kwargs.get("user_pref") or kwargs.get("model_name") or kwargs.get("model") or kwargs.get("chat_model")

    if not api_key:
        raise ValueError("Missing api_key for call_chat_model()")
    if not messages:
        raise ValueError("Missing messages for call_chat_model()")

    client = build_openai_client(api_key)

    # Choose model
    try:
        if user_pref and isinstance(user_pref, str) and user_pref.strip():
            model_used = user_pref.strip()
        else:
            model_used = get_default_model_name("chat")
            if not model_used:
                enabled = list_chat_models()
                model_used = enabled[0] if enabled else None
        if not model_used:
            model_used = "gpt-4.1-mini"
    except Exception:
        model_used = "gpt-4.1-mini"

    model_used = str(model_used).strip()
    is_gpt5 = model_used.startswith("gpt-5")
    is_gpt5_mini = (model_used == "gpt-5-mini")

    # Defaults (caller may override)
    temperature = kwargs.get("temperature", 0.6 if is_gpt5 else 0.3)

    token_budget = kwargs.get("max_completion_tokens") or kwargs.get("max_tokens")
    if not isinstance(token_budget, int) or token_budget <= 0:
        token_budget = 3000 if is_gpt5 else 1200

    def _do_call(omit_temperature: bool = False):
        if is_gpt5:
            params = dict(
                model=model_used,
                messages=messages,
                max_completion_tokens=token_budget,
            )
            # gpt-5-mini: omit temperature entirely
            if (not omit_temperature) and (not is_gpt5_mini):
                params["temperature"] = temperature
            return client.chat.completions.create(**params)
        else:
            params = dict(
                model=model_used,
                messages=messages,
                max_tokens=token_budget,
            )
            if not omit_temperature:
                params["temperature"] = temperature
            return client.chat.completions.create(**params)

    try:
        resp = _do_call(omit_temperature=is_gpt5_mini)
        answer_text = (resp.choices[0].message.content or "").strip()
        return answer_text, answer_text, model_used

    except Exception as e:
        msg = str(e)

        # If temperature is rejected (parameter or value), retry once without temperature.
        if "temperature" in msg and ("Unsupported" in msg or "unsupported" in msg):
            resp = _do_call(omit_temperature=True)
            answer_text = (resp.choices[0].message.content or "").strip()
            return answer_text, answer_text, model_used

        raise


def select_chat_model(*args, **kwargs):
    """
    Backward-compatible helper.

    Supports two legacy patterns:
    1) select_chat_model() -> returns a default model name (string)
    2) select_chat_model(client, model_name, messages, **kwargs) -> executes the model call
    """
    # Pattern 1: called with no args during validation or UI init
    if len(args) == 0 and not {"client", "model_name", "messages"}.issubset(set(kwargs.keys())):
        try:
            dm = get_default_model_name("chat")
            if dm:
                return dm
        except Exception:
            pass
        try:
            models = list_chat_models()
            if models:
                return models[0]
        except Exception:
            pass
        return "gpt-4o-mini"

    # Pattern 2: called with explicit client/model/messages
    if len(args) >= 3:
        client, model_name, messages = args[0], args[1], args[2]
        return call_chat_model(client, model_name, messages)

    if {"client", "model_name", "messages"}.issubset(set(kwargs.keys())):
        return call_chat_model(kwargs["client"], kwargs["model_name"], kwargs["messages"])

    raise TypeError("select_chat_model called with unsupported arguments")


## CELL 7 / STEP 7 – Admin Helpers (Stats & Maintenance)

In [23]:
# CELL 7 / STEP 7 – Admin Helpers (Stats & Maintenance)

def get_db_stats() -> str:
    ensure_docs_table()
    ensure_cohort_table()
    ensure_user_table()
    ensure_chat_history_table()

    conn = get_db_conn()
    cur = conn.cursor()

    cur.execute("SELECT COUNT(*) FROM documents")
    n_docs = cur.fetchone()[0]

    cur.execute("SELECT COUNT(DISTINCT cohort_name) FROM cohort_docs")
    n_cohorts = cur.fetchone()[0]

    cur.execute("SELECT COUNT(*) FROM users")
    n_users = cur.fetchone()[0]

    cur.execute("SELECT COUNT(*) FROM chat_history")
    n_chats = cur.fetchone()[0]

    conn.close()

    return (
        f"**DB Stats**\n\n"
        f"- Documents: {n_docs}\n"
        f"- Cohorts: {n_cohorts}\n"
        f"- Users: {n_users}\n"
        f"- Chat records (last 7 days enforced on write): {n_chats}\n"
    )

def describe_users() -> str:
    rows = list_users()
    if not rows:
        return "No users have been recorded yet."
    lines = ["**Known Users**\n"]
    for user_id, display_name, role in rows:
        disp = display_name or "(no display name)"
        r = role or "(no role)"
        lines.append(f"- `{user_id}` – {disp} – role: `{r}`")
    return "\n".join(lines)

def describe_cohorts() -> str:
    """
    Returns a markdown summary of cohorts, including:
      - name
      - owner
      - visibility (private/shared)
      - document count
    """
    ensure_docs_table()
    ensure_cohort_table()
    ensure_cohort_meta_table()

    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute(
        """
        SELECT DISTINCT
            cd.cohort_name,
            COALESCE(cm.owner_user_id, '(none)') AS owner,
            COALESCE(cm.is_shared, 0)            AS is_shared,
            COUNT(DISTINCT cd.doc_name)          AS num_docs
        FROM cohort_docs cd
        LEFT JOIN cohort_meta cm
          ON cd.cohort_name = cm.cohort_name
        GROUP BY cd.cohort_name, owner, is_shared
        ORDER BY cd.cohort_name
        """
    )
    rows = cur.fetchall()
    conn.close()

    if not rows:
        return "No cohorts found."

    lines = ["**Cohorts**", ""]
    for name, owner, is_shared, num_docs in rows:
        share_label = "shared" if is_shared else "private"
        lines.append(
            f"- **{name}** — owner: `{owner}`, visibility: {share_label}, docs: {num_docs}"
        )

    return "\n".join(lines)

def list_demo_prompts_with_notes(cohort_name: str):
    ensure_demo_prompts_table()
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("""
        SELECT id, prompt_text, admin_note
        FROM demo_prompts
        WHERE cohort_name = ?
        ORDER BY COALESCE(display_order, sort_order, 0) ASC, id ASC
    """, ((cohort_name or "").strip(),))
    rows = cur.fetchall()
    conn.close()
    return rows


def save_demo_prompt_note(prompt_id: int | str, note: str):
    ensure_demo_prompts_table()
    pid = int(str(prompt_id).strip())  # robust for gradio dropdown strings
    conn = get_db_conn()
    cur = conn.cursor()
    cur.execute("""
        UPDATE demo_prompts
        SET admin_note = ?
        WHERE id = ?
    """, ((note or "").strip(), pid))
    conn.commit()
    conn.close()

#====================================================
# Add a formatter for the “Why This Answer” panel EPIC 2.1

#====================================================
def format_why_this_answer_md(trace: dict | None) -> str:
    """
    Render a provenance/trace dict into a compact, human-readable markdown block.
    Expected keys (best-effort):
      - chat_model
      - cohort_name
      - top_k
      - retrieved: list[ {doc_name, score, chunk_preview} ]
    """
    if not trace:
        return "_No provenance captured for this answer._"

    chat_model = trace.get("chat_model") or "(unknown)"
    cohort_name = trace.get("cohort_name") or "(none)"
    top_k = trace.get("top_k") or ""
    retrieved = trace.get("retrieved") or []

    lines = []
    lines.append(f"**Model:** `{chat_model}`")
    lines.append(f"**Cohort:** `{cohort_name}`")
    if top_k:
        lines.append(f"**Top-K chunks:** `{top_k}`")
    lines.append("")

    if not retrieved:
        lines.append("_No retrieval chunks were returned._")
        return "\n".join(lines)

    lines.append("### Retrieved evidence")
    for i, r in enumerate(retrieved, start=1):
        doc = r.get("doc_name") or "(unknown doc)"
        score = r.get("score")
        preview = (r.get("chunk_preview") or "").strip()

        score_str = f"{score:.4f}" if isinstance(score, (int, float)) else (str(score) if score is not None else "n/a")
        lines.append(f"**{i}. {doc}**  \nScore: `{score_str}`")
        if preview:
            # Keep previews short; this is a “why” panel, not a data dump.
            lines.append(f"> {preview[:400]}{'…' if len(preview) > 400 else ''}")
        lines.append("")

    return "\n".join(lines).strip()

# ============================================================

## CELL 8 / STEP 8 – Prompt Coach (Optional Query Improvement) – v16

In [24]:
# CELL 8 / STEP 8 – Prompt Coach (Optional Query Improvement) – v16
#
# Uses the v16 Routing Brain (call_chat_model) instead of calling OpenAI directly.
# Signature is kept the same so STEP 10's on_improve_query(...) still works:
#     improve_query(api_key, chat_model, original_query)
#
# In a future step, we can optionally add a user-selected model override and
# pass it into call_chat_model(user_pref=...).

def improve_query(
    api_key: str,
    chat_model: str,       # kept for backward compatibility; routing ignores it
    original_query: str,
) -> str:
    """
    Prompt coach to re-write the user's query for better RAG retrieval.

    Behavior:
    - If the original query is empty/whitespace, returns "".
    - Otherwise, uses the Routing Brain (task_type='question_improve') to pick
      an appropriate chat model and rewrite the question to be clearer, more
      explicit, and RAG-friendly.
    """
    if not original_query.strip():
        return ""

    system_prompt = (
        "You are a prompt coach helping the user improve questions for a RAG system. "
        "Rewrite the query to be explicit, concise, and focused on key details for retrieval. "
        "Preserve the user's intent AND preserve all explicit user constraints verbatim "
        '(quantity limits, formatting requirements, "only", "exactly", "do not", etc.). '
        "Do not broaden scope, and do not add new requests. "
        "If the user requested a specific number of items (e.g., 1), include that number explicitly. "
        "Return ONLY the improved query text."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": original_query},
    ]

    # Use the v16 Routing Brain instead of calling OpenAI directly
    improved, _, model_used = call_chat_model(
        api_key=api_key,
        messages=messages,
        task_type="question_improve",
        user_pref=None,                     # (optional override will come from UI later)
        context_size=len(original_query),   # small, but available for routing heuristics
    )

    # For now we just return the improved text. If desired later, we can:
    # - log model_used to audit_log
    # - display which model did the improvement in the UI.
    return improved.strip()

# ============================================================

## CELL 9 / STEP 9 – Chat History Viewer (User-Facing)

## STEP 9 — Core viewer, auth, admin wrappers


In [25]:
# CELL 9 / STEP 9 – Chat History Viewer (User-Facing)

ABOUT_TOOL_MD = "### About This Tool\n\n#### What this tool is\n- A cohort-based Q&A assistant over your uploaded documents.\n- It retrieves relevant passages and generates answers grounded in those passages.\n- Intended for demos, internal reviews, and rapid prototyping of RAG workflows.\n\n#### What this tool is not\n- It is not authoritative beyond the documents you upload.\n- It may miss information if it is not present in the cohort (or retrieval is weak).\n- It should not be used as the sole basis for high-stakes decisions.\n\n---\n\n### How to Create and Use Cohorts\n\n#### What a cohort is\nA cohort is a named collection of documents that are indexed together for retrieval. When you ask a question, the tool searches within the selected cohort and cites the strongest matching passages.\n\n#### When to use each action\n- **New Cohort**: Use when you are starting a new topic area or a new demo dataset.\n- **Append Files**: Use when you want to add documents to an existing cohort (e.g., new requirements, updated guidance).\n- **Clone Cohort**: Use when you want a copy of an existing cohort as a starting point (e.g., create a variant for a different audience or demo).\n\n#### Suggested naming conventions\n- Use clear, stable names: `WIC_Online_Shopping_BRD`, `WIC_TIG_Guidance_2024`, `SNAP_Fraud_Prevention`\n- Avoid dates in the cohort name unless the cohort is time-bound; put dates in document names instead.\n\n---\n\n### How to Get Better Answers\n- Ask specific questions that include key terms used in the documents.\n- If the tool returns \u201cno context,\u201d try rephrasing with document terminology.\n- Use **Improve Question** to turn a rough question into a more retrieval-friendly version.\n- Use provenance modes (if enabled) when you need strict traceability to the documents.\n\n---\n\n### Coming Next\n- A guided walkthrough experience and, eventually, an avatar-based onboarding mode."

def format_history_markdown(
    user_id: Optional[str],
    cohort_name: Optional[str],
    limit: int = 50,
) -> str:
    """
    Turn recent history into markdown for display.
    """
    hist = get_recent_history(user_id=user_id, cohort_name=cohort_name, limit=limit)
    if not hist:
        return "No chat history found for the given filters (within retention window)."

    lines = []
    lines.append(
        f"**Showing up to {limit} most recent interactions** "
        f"{'(filtered)' if user_id or cohort_name else ''}\n"
    )

    for h in hist:
        ts = h["created_at"]
        u = h["user_id"] or "(anonymous)"
        r = h["role"] or "(none)"
        c = h["cohort_name"] or "(none)"
        which = h["which_prompt"] or "(unknown)"

        lines.append(f"---\n**User:** `{u}`  |  **Role:** `{r}`  |  **Cohort:** `{c}`  |  **When:** {ts}")
        lines.append(f"**Prompt used:** `{which}`")
        lines.append(f"**Original query:**\n{h['original_query']}\n")
        if h["improved_query"]:
            lines.append(f"**Improved query:**\n{h['improved_query']}\n")
        lines.append("**Answer:**")
        lines.append(h["answer"])
        lines.append("")

    return "\n".join(lines)

#============================================================
# NOTE: If RUN_PREFLIGHT_ONLY=True, do not launch Gradio

#============================================================
if globals().get("STOP_AFTER_PREFLIGHT", False):
    print("✅ Pre-flight only: skipping Gradio launch.")
else:
    # CELL 9.5 / STEP 9.5 – Identity & Admin Ops (v14)
    # ============================================================
    def authenticate(username: str, password: str) -> SessionUser | None:
        """
        MVP auth: checks against local USERS dict.
        Returns SessionUser or None if invalid.
        """
        record = USERS.get(username)
        if not record:
            return None
        if password != record["password"]:
            return None
        return SessionUser(username=username, role=record["role"])


    def authenticate_credentials(username: str, password: str):
        """
        Wrapper used by the Gradio login logic in STEP 10.

        It calls authenticate(...) which returns a SessionUser, then:
          - Upserts the user into the `users` table (for admin/history views)
          - Returns a simple dict {username, role} that the UI expects.
        """

        # Use the existing MVP auth
        user = authenticate(username, password)
        if not user:
            return None

        # Make sure the user exists in the DB's `users` table
        try:
            upsert_user(
                user_id=user.username,
                role=user.role,
                display_name=user.username,
            )
        except Exception as e:
            # Don't break login if DB write fails; just log it
            trace_log(f"authenticate_credentials upsert_user ERROR: {e}")

        # UI login code in STEP 10 expects a dict-like object
        return {"username": user.username, "role": user.role}



    def require_admin(user: SessionUser):
        """
        Helper for admin-only actions. Raises PermissionError if not admin.
        """
        if not user or not user.is_admin:
            raise PermissionError("Admin privileges required for this action.")


    def admin_delete_cohort(user: SessionUser, cohort_name: str) -> str:
        """
        Admin-only wrapper around delete_cohort().
        Uses the existing delete_cohort function from STEP 4.
        """
        try:
            require_admin(user)
        except PermissionError as e:
            return f"❌ Not authorized: {e}"

        if not cohort_name:
            return "❌ Please select a cohort to delete."

        try:
            # Use existing v13 delete_cohort logic (no reassignment in this MVP).
            msg = delete_cohort(cohort_name, reassign_to=None)
            log_audit(user.username, user.role, "delete_cohort", f"cohort={cohort_name}")
            return msg
        except Exception as e:
            return f"❌ Error deleting cohort: {e}"

    def admin_view_audit_log(user: SessionUser, limit: int = 50) -> str:
        """
        Admin-only view of recent audit log entries.
        """
        try:
            require_admin(user)
        except PermissionError as e:
            return f"❌ Not authorized: {e}"

        ensure_audit_table()
        conn = get_db_conn()
        cur = conn.cursor()
        cur.execute(
            """
            SELECT ts, username, role, action, details
            FROM audit_log
            ORDER BY id DESC
            LIMIT ?
            """,
            (limit,),
        )
        rows = cur.fetchall()
        conn.close()

        if not rows:
            return "No audit log entries."

        lines = ["**Recent Audit Log Entries**\n"]
        for ts, username, role, action, details in rows:
            u = username or "-"
            r = role or "-"
            d = details or ""
            lines.append(f"- {ts} | user=`{u}` | role=`{r}` | action=`{action}` | {d}")

        return "\n".join(lines)

    # ============================================================
    # STEP 10 — Gradio App, Tabs & Startup (v16_1 with cohort sharing)
    # ============================================================

    # -----------------------------
    # Setup & Cohorts Tab
    # -----------------------------



## STEP 10 — Setup & Cohorts tab


In [26]:
def build_setup_tab(session_state, api_key_state):
        """
        Setup tab (v16_1 Phase 1):
          - Create a new cohort from uploaded files (authenticated users only)
          - Append files to an existing cohort (owner/admin only)
          - Clone a shared cohort when (is_shared=1 AND allow_clone=1)

        IMPORTANT POLICY:
          - No access is allowed unless the user is logged in.
          - Cohorts must always have a valid owner; 'anonymous' is never permitted.
        """
        gr.Markdown("### Build or Manage a Cohort")

        def _require_session_user(session_user, action: str) -> SessionUser:
            # require_login() is defined earlier in the codebase; this wrapper keeps UI callbacks tidy.
            return require_login(session_user, action)

        def _is_admin(su: SessionUser) -> bool:
            return bool(su and su.is_admin)

        def _get_owner(cohort_name: str) -> str | None:
            ensure_cohort_meta_table()
            conn = get_db_conn()
            cur = conn.cursor()
            cur.execute("SELECT owner_user_id FROM cohort_meta WHERE cohort_name = ? LIMIT 1", (cohort_name,))
            row = cur.fetchone()
            conn.close()
            return row[0] if row else None

        def _can_write(session_user, cohort_name: str) -> bool:
            try:
                su = _require_session_user(session_user, "modify cohorts")
            except PermissionError:
                return False
            if _is_admin(su):
                return True
            owner = _get_owner(cohort_name)
            return bool(owner and su.username and owner == su.username)

        def _list_cloneable_shared_cohorts() -> list[str]:
            """
            Cloneable cohorts = is_shared=1 AND allow_clone=1.
            We join through `documents` to ensure the cohort has content.
            """
            ensure_cohort_meta_table()
            ensure_docs_table()
            conn = get_db_conn()
            cur = conn.cursor()
            cur.execute(
                """
                SELECT DISTINCT cm.cohort_name
                FROM cohort_meta cm
                JOIN documents d
                  ON d.cohort_name = cm.cohort_name
                WHERE cm.is_shared = 1
                  AND COALESCE(cm.allow_clone,0) = 1
                  AND d.cohort_name IS NOT NULL
                  AND TRIM(d.cohort_name) <> ''
                ORDER BY cm.cohort_name
                """
            )
            rows = cur.fetchall()
            conn.close()
            return [r[0] for r in rows]

        # ---------- UI ----------
        with gr.Row():
            action_radio = gr.Radio(
                choices=[ACTION_CREATE, ACTION_APPEND, ACTION_CLONE],
                value=DEFAULT_ACTION,
                label="Action",
            )

        clone_info_md = gr.Markdown("", visible=False)

        with gr.Row():
            new_cohort_name = gr.Textbox(label="New Cohort Name", placeholder="e.g., WIC_Security_2025")
            existing_cohort_dropdown = gr.Dropdown(
                label="Existing Cohort",
                choices=[],
                interactive=True,
                visible=False,
            )

        with gr.Row():
            share_checkbox = gr.Checkbox(label="Shared (visible to all logged-in users)", value=False)
            allow_clone_checkbox = gr.Checkbox(label="Allow other users to clone this cohort", value=False)

        # EPIC 5.1 / 5.2 — Cohort Description + Intended Audience (Admin edit)
        with gr.Accordion("Cohort Description & Intended Audience", open=False):
            cohort_desc_tb = gr.Textbox(label="Cohort Description", lines=4, visible=False)
            cohort_aud_tb  = gr.Textbox(label="Intended Audience", lines=2, visible=False)
            save_meta_btn  = gr.Button("Save Cohort Metadata", variant="primary", visible=False)
            meta_status_md = gr.Markdown("", visible=False)


        file_uploader = gr.File(
            label="Upload Documents (PDF, DOCX, TXT)",
            file_count="multiple",
            type="filepath",
        )

        overwrite_confirm = gr.Checkbox(
            label="If uploaded files include names that already exist in the cohort, OVERWRITE the existing documents and rebuild the cohort index",
            value=False,
        )

        with gr.Row():
            build_btn = gr.Button("Run", variant="primary")

        gr.Markdown("#### Clone a Shared Cohort")
        with gr.Row():
            clone_source_dropdown = gr.Dropdown(
                label="Source Cohort (shared + cloning enabled)",
                choices=[],
                interactive=True,
                visible=False,
            )
            clone_target_name = gr.Textbox(
                label="New Cohort Name (clone target)",
                placeholder="e.g., My_Copy_of_WIC_Security_2025",
                visible=False,
            )

        with gr.Row():
            clone_btn = gr.Button("Clone Cohort", visible=False)

        build_status = gr.Markdown("")

        # ---------- dynamic visibility ----------
        def _toggle_fields(action: str, session_user):
            # ✅ FIX 1: Handle None/empty action by defaulting to CREATE
            if not action or action not in [ACTION_CREATE, ACTION_APPEND,       ACTION_CLONE]:
                action = ACTION_CREATE

            try:
                su = _require_session_user(session_user, "use cohort management")
            except PermissionError:
                # If not logged in, hide everything. (Login gating policy)
                return (
                    gr.update(visible=False),  # new_cohort_name
                    gr.update(visible=False, choices=[], value=None),  # existing_cohort_dropdown
                    gr.update(visible=False),  # share_checkbox
                    gr.update(visible=False),  # allow_clone_checkbox
                    gr.update(visible=False),  # file_uploader
                    gr.update(visible=False),  # build_btn
                    gr.update(visible=False, choices=[], value=None),  # clone_source_dropdown
                    gr.update(visible=False),  # clone_target_name
                    gr.update(visible=False),  # clone_btn
                    gr.update(value="❌ Please log in to manage cohorts.", visible=True),  # clone_info_md
                    gr.update(visible=False, value=""),               # cohort_desc_tb
                    gr.update(visible=False, value=""),               # cohort_aud_tb
                    gr.update(visible=False),                          # save_meta_btn
                    gr.update(visible=False, value=""),               # meta_status_md
                )

            is_create = action == ACTION_CREATE
            is_append = action == ACTION_APPEND
            is_clone = action == ACTION_CLONE
            meta_vis = bool((is_create or is_append))
            meta_save_vis = bool(is_append)

            clone_sources = _list_cloneable_shared_cohorts() if is_clone else []
            has_sources = len(clone_sources) > 0

            msg = ""
            if is_clone and not has_sources:
                msg = "ℹ️ No shared cohorts are available for you to clone yet."

            return (
                gr.update(visible=is_create),
                gr.update(visible=is_append),
                gr.update(visible=is_create or is_append),
                gr.update(visible=is_create or is_append),
                gr.update(visible=is_create or is_append),
                gr.update(visible=is_create or is_append),
                gr.update(visible=is_clone and has_sources, choices=clone_sources, value=None),
                gr.update(visible=is_clone and has_sources),
                gr.update(visible=is_clone and has_sources),
                gr.update(value=msg, visible=bool(msg)),
                gr.update(visible=meta_vis),                          # cohort_desc_tb
                gr.update(visible=meta_vis),                          # cohort_aud_tb
                gr.update(visible=meta_save_vis),                          # save_meta_btn
                gr.update(visible=meta_vis),                          # meta_status_md
            )

        action_radio.change(
            fn=safe_ui_call(_toggle_fields, fallback=(
                gr.update(visible=True),   # new_cohort_name
                gr.update(visible=False),  # existing_cohort_dropdown
                gr.update(visible=True),   # share_checkbox
                gr.update(visible=True),   # allow_clone_checkbox
                gr.update(visible=True),   # file_uploader
                gr.update(visible=True),   # build_btn
                gr.update(visible=False, choices=[], value=None),  # clone_source_dropdown
                gr.update(visible=False, value=""),              # clone_target_name
                gr.update(visible=False),                         # clone_btn
                gr.update(value="❌ UI error. See trace log.", visible=True),  # clone_info_md
                gr.update(visible=False, value=""),               # cohort_desc_tb
                gr.update(visible=False, value=""),               # cohort_aud_tb
                gr.update(visible=False),                          # save_meta_btn
                gr.update(visible=False, value=""),               # meta_status_md
            )),
            inputs=[action_radio, session_state],
            outputs=[
                new_cohort_name,
                existing_cohort_dropdown,
                share_checkbox,
                allow_clone_checkbox,
                file_uploader,
                build_btn,
                clone_source_dropdown,
                clone_target_name,
                clone_btn,
                clone_info_md,
                cohort_desc_tb,
                cohort_aud_tb,
                save_meta_btn,
                meta_status_md,
            ],
        )

        # ✅ FIX 2: REMOVE THIS ENTIRE BLOCK (it causes the bug by re-triggering with action=None)
        # Refresh the cohort-management UI when login/logout changes session_state.
        # This prevents stale "Please log in" messages from persisting after a successful login.
        session_state.change(
            fn=safe_ui_call(_toggle_fields, fallback=(
                gr.update(visible=True),   # new_cohort_name
                gr.update(visible=False),  # existing_cohort_dropdown
                gr.update(visible=True),   # share_checkbox
                gr.update(visible=True),   # allow_clone_checkbox
                gr.update(visible=True),   # file_uploader
                gr.update(visible=True),   # build_btn
                gr.update(visible=False, choices=[], value=None),  # clone_source_dropdown
                gr.update(visible=False, value=""),                # clone_target_name
                gr.update(visible=False),                          # clone_btn
                gr.update(value="", visible=False),                # clone_info_md  (CLEAR)
            )),
            inputs=[action_radio, session_state],
            outputs=[
                new_cohort_name,
                existing_cohort_dropdown,
                share_checkbox,
                allow_clone_checkbox,
                file_uploader,
                build_btn,
                clone_source_dropdown,
                clone_target_name,
                clone_btn,
                clone_info_md,
                cohort_desc_tb,
                cohort_aud_tb,
                save_meta_btn,
                meta_status_md,
            ],
            queue=False,
        )


        # ---------- refresh helpers ----------


        def _refresh_existing_dropdown(session_user):
            try:
                su = _require_session_user(session_user, "view cohorts")
            except PermissionError:
                return gr.update(choices=[], value=None)
            return gr.update(choices=get_cohorts_for_user(su.username), value=None)

        def _refresh_clone_sources(session_user):
            try:
                _require_session_user(session_user, "view cloneable cohorts")
            except PermissionError:
                return gr.update(choices=[], value=None)
            choices = _list_cloneable_shared_cohorts()
            return gr.update(choices=choices, value=None)
        def _normalize_cohort_name(selected: str | None) -> str:
            s = (selected or "").strip()
            if not s:
                return ""
            # If dropdown shows "name/owner", keep only the cohort name
            return s.split("/", 1)[0].strip()

        # ---------- action handlers ----------
       # ---------- action handlers ----------
        def _run_action(action, api_key, session_user, new_name, existing_name, files, overwrite_confirm, is_shared, allow_clone, description, intended_audience):
            """
            MUST return exactly 3 outputs (matches build_btn.click outputs):
              1) build_status (markdown)
              2) file_uploader (File component)
              3) existing_cohort_dropdown (Dropdown component)
            """

            # Helper: refresh existing cohort dropdown choices for this user
            def _refresh_existing_dd(owner_username: str):
                try:
                    choices = get_cohorts_for_user(owner_username)
                except Exception as e:
                    trace_log(f"SETUP refresh existing dropdown ERROR: {e}")
                    choices = []
                return gr.update(choices=choices, value=None)

            # 1) Require session user
            try:
                su = _require_session_user(session_user, "create or modify cohorts")
            except PermissionError as e:
                return f"❌ {e}", gr.update(), gr.update()

            owner = (su.username or "").strip()
            if not owner or owner.lower() == "anonymous":
                return "❌ You must be logged in to create or modify cohorts (owner is required).", gr.update(), gr.update()

            if not api_key or not str(api_key).strip():
                return "❌ Please provide an OpenAI API key.", gr.update(), gr.update()

            # -------------------------
            # CREATE
            # -------------------------
            if action == ACTION_CREATE:
                cohort_name = (new_name or "").strip()

                if not cohort_name:
                    return "❌ Please provide a new cohort name.", gr.update(), gr.update()
                if cohort_exists(cohort_name):
                    return f"❌ Cohort '{cohort_name}' already exists.", gr.update(), gr.update()
                if not files:
                    return "❌ Please upload at least one file.", gr.update(), gr.update()

                # Build index first
                try:
                    msg = build_cohort_index(api_key=api_key, cohort_name=cohort_name, files=files, owner=owner)
                except Exception as e:
                    trace_log(f"Create build_cohort_index ERROR: {e}")
                    return f"❌ Error building cohort: {e}", gr.update(), gr.update()

                # Persist metadata (REQUIRED)
                try:
                    ensure_cohort_meta_table()
                    set_cohort_owner(cohort_name, su)  # required
                    set_cohort_sharing(cohort_name, bool(is_shared))
                    set_cohort_allow_clone(cohort_name, bool(allow_clone))
                    upsert_cohort_description_and_audience(cohort_name, description, intended_audience)
                    upsert_cohort_description_and_audience(cohort_name, description, intended_audience)
                except Exception as e:
                    trace_log(f"Create cohort meta update ERROR: {e}")
                    return f"❌ Failed to persist cohort ownership/sharing metadata: {e}", gr.update(), gr.update()

                try:
                    log_audit(owner, su.role, "create_cohort", f"cohort={cohort_name}, shared={is_shared}, allow_clone={allow_clone}")
                except Exception as e:
                    trace_log(f"log_audit create_cohort ERROR: {e}")

                # SUCCESS: clear uploader + refresh dropdown
                refreshed_dd = _refresh_existing_dd(owner)
                return msg, gr.update(value=None), refreshed_dd

            # -------------------------
            # APPEND
            # -------------------------
            if action == ACTION_APPEND:
                cohort_name = _normalize_cohort_name(existing_name)

                if not cohort_name:
                    return "❌ Please select an existing cohort.", gr.update(), gr.update()
                if not cohort_exists(cohort_name):
                    return f"❌ Cohort '{cohort_name}' does not exist.", gr.update(), gr.update()
                if not files:
                    return "❌ Please upload at least one file.", gr.update(), gr.update()
                if not _can_write(session_user, cohort_name):
                    return "❌ Not authorized: only the cohort owner or an admin may append files.", gr.update(), gr.update()

                # Detect filename collisions (overwrite-by-filename requires explicit confirmation)
                try:
                    existing_docs = set([d.strip().lower() for d in (list_docs_in_cohort(cohort_name) or []) if d])
                except Exception as e:
                    trace_log(f"Append list_docs_in_cohort WARNING cohort={cohort_name}: {e}")
                    existing_docs = set()

                uploaded_names = []
                for f in (files or []):
                    n = getattr(f, "name", None) or getattr(f, "orig_name", None) or ""
                    n = os.path.basename(str(n)).strip()
                    if n:
                        uploaded_names.append(n)

                dupes = sorted({n for n in uploaded_names if n.strip().lower() in existing_docs})

                if dupes and not bool(overwrite_confirm):
                    msg_lines = [
                        "⚠️ **Potential overwrite detected**",
                        "",
                        "The following file name(s) already exist in this cohort:",
                    ] + [f"- `{d}`" for d in dupes] + [
                        "",
                        "To proceed, check the overwrite confirmation box and click **Run** again. Otherwise, clear the upload selection to back out (no changes have been made).",
                    ]
                    return "\n".join(msg_lines), gr.update(), gr.update()


                embed_cfg = get_default_embed_model_config()

                try:
                    msg = build_cohort_from_files(
                        api_key=api_key,
                        embed_model=embed_cfg.model_id,
                        cohort_name=cohort_name,
                        files=files,
                        overwrite_existing=bool(overwrite_confirm)
                    )
                except Exception as e:
                    trace_log(f"Append build_cohort_from_files ERROR: {e}")
                    return f"❌ Error appending files to cohort: {e}", gr.update(), gr.update()

                # Owner/admin may update sharing flags during append
                try:
                    ensure_cohort_meta_table()
                    set_cohort_owner(cohort_name, su)  # repairs invalid owners if needed
                    set_cohort_sharing(cohort_name, bool(is_shared))
                    set_cohort_allow_clone(cohort_name, bool(allow_clone))
                    upsert_cohort_description_and_audience(cohort_name, description, intended_audience)
                except Exception as e:
                    trace_log(f"Append cohort meta update ERROR: {e}")
                    return f"❌ Failed to update cohort metadata: {e}", gr.update(), gr.update()

                try:
                    log_audit(owner, su.role, "append_files", f"cohort={cohort_name}, shared={is_shared}, allow_clone={allow_clone}")
                except Exception as e:
                    trace_log(f"log_audit append_files ERROR: {e}")

                # SUCCESS: clear uploader + refresh dropdown
                refreshed_dd = _refresh_existing_dd(owner)
                return msg, gr.update(value=None), refreshed_dd

            # -------------------------
            # Unsupported
            # -------------------------
            return "❌ Unsupported action selected.", gr.update(), gr.update()

        build_btn.click(
            fn=safe_ui_call(_run_action, fallback=("❌ UI error. See trace log.", gr.update(), gr.update())),
            inputs=[
                action_radio,
                api_key_state,
                session_state,
                new_cohort_name,
                existing_cohort_dropdown,
                file_uploader,
                overwrite_confirm,
                share_checkbox,
                allow_clone_checkbox,
                cohort_desc_tb,
                cohort_aud_tb,
            ],
            outputs=[build_status, file_uploader, existing_cohort_dropdown],

        )


        def _run_clone(api_key, session_user, source_cohort, target_cohort):
            try:
                su = _require_session_user(session_user, "clone cohorts")
            except PermissionError as e:
                return f"❌ {e}"

            if not api_key or not str(api_key).strip():
                return "❌ Please provide an OpenAI API key."
            if not source_cohort:
                return "❌ Please select a source cohort."
            if not target_cohort or not target_cohort.strip():
                return "❌ Please provide a new cohort name for the clone."

            allowed = set(_list_cloneable_shared_cohorts())
            if (source_cohort not in allowed) and not _is_admin(su):
                return "❌ Not authorized: cloning is not enabled for this cohort."

            msg = clone_cohort(source_cohort=source_cohort, target_cohort=target_cohort.strip(), new_owner=su.username)
            try:
                log_audit(su.username, su.role, "clone_cohort", f"{source_cohort} → {target_cohort.strip()}")
            except Exception as e:
                trace_log(f"log_audit clone_cohort ERROR: {e}")

            return msg

        clone_btn.click(
            fn=safe_ui_call(_run_clone),
            inputs=[api_key_state, session_state, clone_source_dropdown, clone_target_name],
            outputs=[build_status],
        )

        # ---------- EPIC 5.1 / 5.2 — Admin cohort meta editor handlers ----------

# ---------- Cohort metadata (Description / Intended Audience) ----------
# Policy: cohort OWNER or ADMIN may edit; anyone with access may view.
        def _load_cohort_meta(session_user, cohort_name):
            try:
                su = _require_session_user(session_user, "edit cohort metadata")
            except PermissionError:
                return (
                    gr.update(value="", visible=True),
                    gr.update(value="", visible=True),
                    gr.update(visible=False),
                    gr.update(value="❌ Please log in to edit cohort metadata.", visible=True),
                )

            cn = (cohort_name or "").strip()
            if not cn:
                return (
                    gr.update(value="", visible=True),
                    gr.update(value="", visible=True),
                    gr.update(visible=False),
                    gr.update(value="", visible=True),
                )

            # Only cohort owner or admin can edit metadata for an existing cohort.
            if not _can_write(session_user, cn):
                meta = get_cohort_meta(cn) or {}
                desc = meta.get("description", "") or ""
                aud  = meta.get("intended_audience", "") or ""
                return (
                    gr.update(value=desc, visible=True),
                    gr.update(value=aud, visible=True),
                    gr.update(visible=False),
                    gr.update(value="ℹ️ Read-only: only the cohort owner or an admin may edit this metadata.", visible=True),
                )

            meta = get_cohort_meta(cn) or {}
            return (
                gr.update(value=meta.get("description", ""), visible=True),
                gr.update(value=meta.get("intended_audience", ""), visible=True),
                gr.update(visible=True),
                gr.update(value="", visible=True),
            )

        def _save_cohort_meta(session_user, cohort_name, description, intended_audience):
            try:
                _require_session_user(session_user, "save cohort metadata")
            except PermissionError:
                return "❌ Please log in to save cohort metadata."

            cn = (cohort_name or "").strip()
            if not cn:
                return "❌ Please select a cohort first."

            if not _can_write(session_user, cn):
                return "❌ Not authorized: only the cohort owner or an admin may edit metadata."

            try:
                upsert_cohort_description_and_audience(cn, description, intended_audience)
                try:
                    su = _require_session_user(session_user, "audit cohort metadata save")
                    log_audit(su.username, su.role, "save_cohort_meta", f"{cn}")
                except Exception as e:
                    trace_log(f"log_audit save_cohort_meta ERROR: {e}")
                return "✅ Cohort metadata saved."
            except Exception as e:
                trace_log(f"_save_cohort_meta ERROR: {e}")
                return f"❌ Error saving cohort metadata: {e}"

        existing_cohort_dropdown.change(
                    fn=safe_ui_call(_load_cohort_meta),
                    inputs=[session_state, existing_cohort_dropdown],
                    outputs=[cohort_desc_tb, cohort_aud_tb, save_meta_btn, meta_status_md],
                    queue=False,
                )

        save_meta_btn.click(
            fn=safe_ui_call(_save_cohort_meta),
            inputs=[api_key_state, session_state, existing_cohort_dropdown, cohort_desc_tb, cohort_aud_tb],
            outputs=[meta_status_md],
            queue=False,
        )

        # Return handles expected by build_interface()
        return {
            "action_radio": action_radio,
            "new_cohort_name": new_cohort_name,
            "existing_cohort_dropdown": existing_cohort_dropdown,
            "share_checkbox": share_checkbox,
            "allow_clone_checkbox": allow_clone_checkbox,
            "file_uploader": file_uploader,
            "build_btn": build_btn,
            "clone_source_dropdown": clone_source_dropdown,
            "clone_target_name": clone_target_name,
            "clone_btn": clone_btn,
            "build_status": build_status,
            "clone_info_md": clone_info_md,
            "refresh_existing_dropdown": _refresh_existing_dropdown,
            "refresh_clone_sources": _refresh_clone_sources,
        }



## STEP 11 — Ask tab


In [27]:
def build_ask_tab(session_state, api_key_state, session_memory_state):
        """
        Ask tab:
          - Choose cohort + chat model
          - Optional prompt improvement
          - Ask question over cohort with RAG
          - View documents in selected cohort
        """
        gr.Markdown("### Ask Questions Over a Cohort")
        # EPIC 4.1 — Expectation Management ("What this is / isn't")

        # EPIC 5.1 / 5.2 — Cohort context (read-only display on Ask tab)
        cohort_desc_md = gr.Markdown("**Cohort Description:** (not set)", visible=False)
        cohort_aud_md  = gr.Markdown("**Intended Audience:** (not set)", visible=False)


        with gr.Row():
            cohort_dropdown = gr.Dropdown(
                label="Cohort",
                choices=[],
                elem_id="ask_cohort_dropdown",
            )
            model_dropdown = gr.Dropdown(
                label="Choose Chat Model",
                choices=list_chat_models(),
                value=get_default_chat_model_id(),
                elem_id="ask_model_dropdown",
            )




            # EPIC 3.4 — Provenance / Evidence mode
            provenance_mode_radio = gr.Radio(
                label="Provenance Mode (EPIC 3.4)",
                choices=["Off", "Explain Sources", "Strict Evidence Only"],
                value="Off",
                interactive=True,
                elem_id="ask_provenance_mode_radio",
            )
        with gr.Row():
            demo_prompt_dropdown = gr.Dropdown(
                label="FAP (optional)",
                choices=[],
                value=None,
                interactive=True,
                visible=False,
                elem_id="ask_demo_prompt_dropdown",
            )

        with gr.Row():
            question_box = gr.Textbox(
                label="Your Question",
                lines=3,
                elem_id="ask_question_box",
            )
            improved_box = gr.Textbox(
                label="Improved Prompt (optional, from Prompt Coach)",
                lines=3,
                elem_id="ask_improved_box",
            )


        with gr.Row():
            improve_btn = gr.Button("Improve Prompt", elem_id="ask_improve_btn")
            ask_btn = gr.Button("Ask Question", elem_id="ask_ask_btn")
            reset_demo_btn = gr.Button("Reset Demo", elem_id="ask_reset_demo_btn")  # EPIC 1.3
            refresh_cohorts_btn = gr.Button("Refresh Cohorts", elem_id="ask_refresh_btn")
            refresh_models_btn = gr.Button("Refresh Models", elem_id="ask_refresh_models_btn")



        with gr.Accordion("Documents in Selected Cohort", open=False):
            docs_md = gr.Markdown("_No cohort selected._")


        gr.Markdown("#### Answer")
        answer_md = gr.Markdown()

        # EPIC 2.1 — Trust/Provenance panel (collapsed by default)
        with gr.Accordion("Why This Answer", open=False):
            why_md = gr.Markdown("")

        # EPIC 2.2 — Always-visible footnote (short provenance)
        answer_footnote_md = gr.Markdown("")

        # ---- Callbacks ----

        def _refresh_cohorts(session_user):
            """Refresh cohort list for the logged-in user only (no anonymous access)."""
            try:
                su = require_login(session_user, "view cohorts")
                names = get_cohorts_for_user(su.username)
                return gr.update(choices=names, value=None)
            except PermissionError:
                return gr.update(choices=[], value=None)
            except Exception as e:
                trace_log(f"ASK _refresh_cohorts ERROR: {e}")
                return gr.update(choices=[], value=None)

        def _refresh_models(session_user):
            """Refresh chat model list from the model registry (login required)."""
            try:
                require_login(session_user, "view chat models")
            except PermissionError:
                return gr.update(choices=[], value=None)
            try:
                choices = list_chat_models()
                default_id = get_default_chat_model_id()
                value = default_id if default_id in choices else (choices[0] if choices else None)
                return gr.update(choices=choices, value=value)
            except Exception as e:
                trace_log(f"ASK _refresh_models ERROR: {e}")
                return gr.update(choices=list_chat_models(), value=get_default_chat_model_id())




        def _refresh_demo_prompts_ui(cohort_name: str, session_user):
            """
            If selected cohort has FAPs, show the Demo Prompt dropdown and populate choices.
            Always clears the question box on cohort change to avoid stale text between demos.
            """
            try:
                su = require_login(session_user, "view FAPs")
            except PermissionError:
                return gr.update(visible=False, choices=[], value=None), gr.update(value="")

            c = (cohort_name or "").strip()
            if not c:
                return gr.update(visible=False, choices=[], value=None), gr.update(value="")

            prompts = list_demo_prompts_for_cohort(c)
            if not prompts:
                return gr.update(visible=False, choices=[], value=None), gr.update(value="")

            return gr.update(visible=True, choices=prompts, value=None), gr.update(value="")

        def _demo_prompt_pick_cb(selected_prompt: str):
            # Populate "Your Question" from the selected demo prompt.
            return gr.update(value=(selected_prompt or "").strip())

        def _reset_demo_cb():
            """
            EPIC 1.3 — Reset Demo
            Clears only UI state (no DB changes):
              - Demo Prompt selection
              - Your Question
              - Improved Prompt
              - Answer
            """
            return (
                    gr.update(value=None),  # demo_prompt_dropdown
                    "",                     # question_box
                    "",                     # improved_box
                    "",                     # answer_md
                    "",                     # why_md
                    "",                     # answer_footnote_md
                )


        def _improve_query_cb(original_query: str, api_key: str):
            if not api_key or not api_key.strip():
                return "❌ OpenAI API key is required."
            try:
                improved = improve_query(
                    api_key=api_key,
                    chat_model=CHAT_MODEL_DEFAULT,
                    original_query=original_query or "",
                )
                if not improved:
                    return original_query
                return improved
            except Exception as e:
                trace_log(f"ASK _improve_query_cb ERROR: {e}")
                return f"❌ Error improving query: {e}"

        def _ask_cb(
            cohort_name: str,
            question: str,
            improved_prompt: str,
            model_id: str,
            provenance_mode: str,  # EPIC 3.4
            session_user,
            api_key: str,
            session_mem: dict,           # NEW
        ):

        # EPIC 3.4.1 — Ensure session_mem always exists for consistent returns
            if not session_mem:
                session_mem = init_session_memory_state()

            def _ret(ans, why="", foot=""):
                return ans, why, foot, session_mem

            try:
                su = require_login(session_user, "ask questions")
            except PermissionError as e:
                return _ret(f"❌ Error while answering question: {e}")

            username = (su.get("username") if isinstance(su, dict) else getattr(su, "username", "")).strip()

            # Authorization: user must have visibility to cohort
            if cohort_name and cohort_name not in set(get_cohorts_for_user(username)):
                return _ret("❌ Not authorized to access this cohort.")

            if not api_key or not api_key.strip():
                return _ret("❌ OpenAI API key is required.")
            # EPIC 3.3.1 — Allow session-scope questions without a cohort
           # EPIC 3.4.1 — Cohort selection rules
            cohort_selected = bool((cohort_name or "").strip())

            if not cohort_selected:
                # Strict Evidence Only requires a cohort (evidence source)
                if provenance_mode == "Strict Evidence Only":
                    return _ret("❌ Strict Evidence Only requires a cohort selection (document evidence).")

                # Allow cohortless questions only when memory scope is session
                if session_mem.get("scope") != "session":
                    return _ret("❌ Please select a cohort, or switch Memory Scope to 'session' to ask without a cohort.")
                # else: allowed to proceed without cohort


            # Decide which prompt is used
            improved_clean = (improved_prompt or "").strip()
            question_clean = (question or "").strip()
            used_improved = bool(improved_clean)

            final_question = improved_clean if used_improved else question_clean
            if not final_question:
                return _ret("❌ Please enter a question.")

            try:
                answer_markdown, raw_answer_text, used_model, why_md_text, footnote_text = answer_question_over_cohort(
                    api_key=api_key,
                    username=username,
                    cohort_name=cohort_name,
                    question=final_question,
                    model_id=model_id or CHAT_MODEL_DEFAULT,
                    provenance_mode=provenance_mode or 'Off',  # EPIC 3.4
                    session_mem=session_mem,
                )

                # Log to chat_history
                try:
                    save_chat_history(
                        user=session_user,
                        cohort=cohort_name,
                        question=final_question,
                        answer=raw_answer_text,
                        model_used=used_model,
                    )
                except Exception as log_e:
                    trace_log(f"ASK save_chat_history ERROR: {log_e}")                # EPIC 3.2 — update in-session memory (ephemeral; not persisted)
                try:
                    if session_mem and session_mem.get("enabled"):
                        session_mem = update_session_memory_state(
                            mem=session_mem,
                            cohort_name=cohort_name,
                            q=final_question,
                            a=raw_answer_text or "",
                            max_turns=int((session_mem or {}).get('max_turns') or SESSION_MEM_MAX_TURNS),  # EPIC 3.3

                        )
                except Exception as mem_e:
                    trace_log(f"ASK session memory update ERROR: {mem_e}")



                # Add a visible note about which prompt was used
                if used_improved:
                    note = "**Note:** Used the *Improved Prompt* for this answer.\n\n"
                else:
                    note = "**Note:** Used your *Original Question* for this answer.\n\n"

                return (note + (answer_markdown or "")), (why_md_text or ""), (footnote_text or ""), session_mem


            except Exception as e:
                trace_log(f"ASK _ask_cb ERROR: {e}")
                return _ret(f"❌ Error while answering question: {e}")

        def _load_docs_for_cohort(session_user, cohort_name: str, session_mem: dict):  # EPIC 3.3
            """Show documents for the selected cohort (login required)."""
            try:
                su = require_login(session_user, "view cohort documents")
            except PermissionError:
                return "❌ Login required to view cohort documents.", session_mem
            if not cohort_name:
                return "_No cohort selected._", session_mem
            # Authorization: user must have visibility to this cohort
            if cohort_name not in set(get_cohorts_for_user(su.username)):
                return "❌ Not authorized to view this cohort.", session_mem
            # EPIC 3.3 — Cohort-scope session memory: clear turns when cohort changes
            session_mem = session_mem if isinstance(session_mem, dict) else init_session_memory_state()
            scope_val = (session_mem.get('scope') or 'cohort').lower()
            prev_cohort = session_mem.get('cohort_name')
            if scope_val == 'cohort' and cohort_name and prev_cohort and cohort_name != prev_cohort:
                enabled = bool(session_mem.get('enabled', True))
                max_turns = int(session_mem.get('max_turns') or SESSION_MEM_MAX_TURNS)
                session_mem = init_session_memory_state()
                session_mem['enabled'] = enabled
                session_mem['max_turns'] = max_turns
                session_mem['scope'] = 'cohort'
            session_mem['cohort_name'] = cohort_name

            try:
                conn = get_db_conn()
                cur = conn.cursor()
                cur.execute(
                    """
                    SELECT doc_name, created_at
                    FROM documents
                    WHERE cohort_name = ?
                    ORDER BY created_at
                    """,
                    (cohort_name,),
                )
                rows = cur.fetchall()
                conn.close()

                if not rows:
                    return f"_No documents found for cohort `{cohort_name}`._", session_mem

                lines = [f"**Documents in cohort `{cohort_name}`:**"]
                for doc_name, created_at in rows:
                    if created_at:
                        lines.append(f"- `{doc_name}` (added {created_at})")
                    else:
                        lines.append(f"- `{doc_name}`")
                return "\n".join(lines), session_mem
            except Exception as e:
                trace_log(f"ASK _load_docs_for_cohort ERROR: {e}")
                return f"❌ Error loading documents for cohort `{cohort_name}`.", session_mem

        # Wire buttons
        refresh_cohorts_btn.click(
            _refresh_cohorts,
            inputs=[session_state],
            outputs=[cohort_dropdown],
        )

        refresh_models_btn.click(
            _refresh_models,
            inputs=[session_state],
            outputs=[model_dropdown],
        )


        improve_btn.click(
            _improve_query_cb,
            inputs=[question_box, api_key_state],
            outputs=[improved_box],
        )

        ask_btn.click(
            _ask_cb,
            inputs=[
                cohort_dropdown,
                question_box,
                improved_box,
                model_dropdown,
                provenance_mode_radio,  # EPIC 3.4
                session_state,
                api_key_state,
                session_memory_state,
            ],
            outputs=[answer_md, why_md, answer_footnote_md, session_memory_state],
        )


        reset_demo_btn.click(
            fn=safe_ui_call(
                _reset_demo_cb,
                fallback=(gr.update(value=None), "", "", "", "", ""),
            ),
            inputs=[],
            outputs=[
                demo_prompt_dropdown,
                question_box,
                improved_box,
                answer_md,
                why_md,
                answer_footnote_md,
            ],
            queue=False,
        )


        def _refresh_cohort_meta_md(cohort_name):
            cn = (cohort_name or "").strip()
            if not cn:
                # Hide meta until a cohort is selected (Option A)
                return (
                    gr.update(value="", visible=False),
                    gr.update(value="", visible=False),
                )

            meta = get_cohort_meta(cn) or {}
            desc = (meta.get("description", "") or "").strip()
            aud  = (meta.get("intended_audience", "") or "").strip()
            desc_out = desc if desc else "(not set)"
            aud_out  = aud if aud else "(not set)"

            return (
                gr.update(value=f"**Cohort Description:** {desc_out}", visible=True),
                gr.update(value=f"**Intended Audience:** {aud_out}", visible=True),
            )

        cohort_dropdown.change(
            _load_docs_for_cohort,
            inputs=[session_state, cohort_dropdown, session_memory_state],  # EPIC 3.3
            outputs=[docs_md, session_memory_state],  # EPIC 3.3
        )

        cohort_dropdown.change(
            _refresh_demo_prompts_ui,
            inputs=[cohort_dropdown, session_state],
            outputs=[demo_prompt_dropdown, question_box],
        )

        cohort_dropdown.change(
            _refresh_cohort_meta_md,
            inputs=[cohort_dropdown],
            outputs=[cohort_desc_md, cohort_aud_md],
            queue=False,
        )


        demo_prompt_dropdown.change(
            _demo_prompt_pick_cb,
            inputs=[demo_prompt_dropdown],
            outputs=[question_box],
        )

        # ============================================================
        # EPIC 3.3 — Session Memory Controls (User-Self-Service)
        # (Additive UI; minimal impact to existing layout)
        # ============================================================
        with gr.Accordion("Session Memory (EPIC 3.3)", open=False):
            session_mem_user_status_md = gr.Markdown("")
            session_mem_user_scope_radio = gr.Radio(
                choices=["cohort", "session"],
                value="cohort",
                label="Memory scope (EPIC 3.3)",
                info="cohort = resets on cohort change; session = persists across cohorts",
            )
            session_mem_user_max_turns = gr.Number(
                label="Max turns (EPIC 3.3)",
                value=SESSION_MEM_MAX_TURNS,
                precision=0,
            )
            with gr.Row():
                session_mem_user_clear_btn = gr.Button("Clear My Session Memory (EPIC 3.3)")
                session_mem_user_refresh_btn = gr.Button("Refresh Viewer (EPIC 3.3)")
            session_mem_user_view_md = gr.Markdown("_Session memory viewer will appear here._")

        def _format_session_mem_view_user(mem: dict) -> str:
            if not isinstance(mem, dict):
                return "_(no session memory state)_"
            enabled = mem.get("enabled", True)
            scope = (mem.get("scope") or "cohort").upper()
            max_turns = int(mem.get("max_turns") or SESSION_MEM_MAX_TURNS)
            cohort_name = mem.get("cohort_name")
            turns = mem.get("turns") or []
            if not enabled:
                return f"_(OFF; scope={scope}; max_turns={max_turns}; cohort={cohort_name})_"
            if not turns:
                return f"_(ON; empty; scope={scope}; max_turns={max_turns}; cohort={cohort_name})_"
            lines = [f"**Session Memory (ON)** — scope={scope}, max_turns={max_turns}, cohort={cohort_name}, turns={len(turns)}\n"]
            for i, t in enumerate(turns[-max_turns:], start=max(1, len(turns)-max_turns+1)):
                q = (t.get("q") or "").strip()
                a = (t.get("a") or "").strip()
                lines.append(f"**{i}. Q:** {q}\n**A:** {a}\n")
            return "\n".join(lines)

        def _user_clear_session_memory(mem: dict):
            mem = mem if isinstance(mem, dict) else init_session_memory_state()
            mem["turns"] = []
            return mem, "✅ Cleared your session memory.", _format_session_mem_view_user(mem)

        def _user_set_session_mem_max_turns(max_turns_val, mem: dict):
            mem = mem if isinstance(mem, dict) else init_session_memory_state()
            try:
                mv = int(max_turns_val or SESSION_MEM_MAX_TURNS)
            except Exception:
                mv = SESSION_MEM_MAX_TURNS
            mv = max(1, min(200, mv))
            mem["max_turns"] = mv
            return mem, f"✅ Max turns set to {mv}.", _format_session_mem_view_user(mem), gr.update(value=mv)

        def _user_set_session_mem_scope(scope_val, mem: dict):
            mem = mem if isinstance(mem, dict) else init_session_memory_state()
            scope_val = (scope_val or "cohort").lower()
            if scope_val not in ("cohort", "session"):
                scope_val = "cohort"
            mem["scope"] = scope_val
            return mem, f"✅ Scope set to {scope_val.upper()}.", _format_session_mem_view_user(mem), gr.update(value=scope_val)

        def _user_refresh_session_mem_view(mem: dict):
            mem = mem if isinstance(mem, dict) else init_session_memory_state()
            return _format_session_mem_view_user(mem)

        # Wire user controls (no admin gating)
        session_mem_user_clear_btn.click(
            _user_clear_session_memory,
            inputs=[session_memory_state],
            outputs=[session_memory_state, session_mem_user_status_md, session_mem_user_view_md],
            queue=False,
        )
        session_mem_user_max_turns.change(
            _user_set_session_mem_max_turns,
            inputs=[session_mem_user_max_turns, session_memory_state],
            outputs=[session_memory_state, session_mem_user_status_md, session_mem_user_view_md, session_mem_user_max_turns],
            queue=False,
            trigger_mode="once",
        )
        session_mem_user_scope_radio.change(
            _user_set_session_mem_scope,
            inputs=[session_mem_user_scope_radio, session_memory_state],
            outputs=[session_memory_state, session_mem_user_status_md, session_mem_user_view_md, session_mem_user_scope_radio],
            queue=False,
            trigger_mode="once",
        )
        session_mem_user_refresh_btn.click(
            _user_refresh_session_mem_view,
            inputs=[session_memory_state],
            outputs=[session_mem_user_view_md],
            queue=False,
        )

        # EPIC 3.3.2 — Keep viewer in sync with state updates (e.g., cohort-scope clear)
        session_memory_state.change(
            _user_refresh_session_mem_view,
            inputs=[session_memory_state],
            outputs=[session_mem_user_view_md],
            queue=False,
        )


        return {
            "cohort_dropdown": cohort_dropdown,
            "model_dropdown": model_dropdown,
            "question_box": question_box,
            "improved_box": improved_box,
            "answer_markdown": answer_md,
            "docs_markdown": docs_md,
        }


    # -----------------------------
    # History Tab
    # -----------------------------


## STEP 12 — History tab


In [28]:
def build_history_tab(session_state):
        """
        Simple history viewer using format_history_markdown.
        """
        gr.Markdown("### Chat History (read-only)")

        with gr.Row():
            history_user_filter = gr.Textbox(
                label="Filter by User ID (optional)",
                placeholder="Leave blank for all users",
            )
            history_cohort_filter = gr.Textbox(
                label="Filter by Cohort (optional)",
                placeholder="Leave blank for all cohorts",
            )

        with gr.Row():
            use_current_user_btn = gr.Button("Use Current User")
            refresh_history_btn = gr.Button("Refresh History")

        history_md = gr.Markdown()

        def _use_current_user(session_user):
            return _extract_user_id(session_user) or ""

        def _refresh_history(session_user, user_id: str, cohort_name: str):
            try:
                su = require_login(session_user, "view chat history")
            except PermissionError as e:
                return f"❌ {e}"

            is_admin = su.is_admin

            u_raw = (user_id or "").strip()
            c_raw = (cohort_name or "").strip()

            # Normalize common "All" sentinel values to None
            if u_raw.lower() in ("all users", "all", "*"):
                u_raw = ""
            if c_raw.lower() in ("all cohorts", "all", "*"):
                c_raw = ""

            u = u_raw or None
            c = c_raw or None

            # Non-admins can only view their own history regardless of filter
            if not is_admin:
                u = su.username

            try:
                return format_history_markdown(u, c, limit=50)
            except Exception as e:
                trace_log(f"HISTORY _refresh_history ERROR: {e}")
                return f"❌ Error loading history: {e}"


        use_current_user_btn.click(
            _use_current_user,
            inputs=[session_state],
            outputs=[history_user_filter],
        )

        refresh_history_btn.click(
            _refresh_history,
            inputs=[session_state, history_user_filter, history_cohort_filter],
            outputs=[history_md],
        )


        return {
            "history_user_filter": history_user_filter,
            "history_cohort_filter": history_cohort_filter,
            "history_markdown": history_md,
        }


    # -----------------------------
    # Admin Tab
    # -----------------------------


## STEP 13 — Admin tab


In [29]:
def build_admin_tab(session_state, session_memory_state):
        """
        Admin tab showing DB stats, users, cohorts, and audit log.
        Fix 3.3: prevent cross-button "stacking" by updating all 4 panes each click.
        """
        admin_handles = {}
        gr.Markdown("### Admin (requires admin role)")
        gr.Markdown("You must be logged in as an admin user to see details here.")
        gr.Markdown("#### Session Memory (EPIC 3.2 / 3.3)")
        session_mem_enabled_cb = gr.Checkbox(
            label="Enable session memory (this browser session)",
            value=True,
        )
        session_mem_status_md = gr.Markdown("")
        # EPIC 3.3 — Additive controls (no layout refactor)
        session_mem_max_turns = gr.Number(
            label="Max session memory turns (EPIC 3.3)",
            value=SESSION_MEM_MAX_TURNS,
            precision=0,
        )
        session_mem_scope_radio = gr.Radio(
            choices=["cohort", "session"],
            value="cohort",
            label="Session memory scope (EPIC 3.3)",
            info="cohort = resets on cohort change; session = persists across cohorts",
        )

        session_mem_clear_btn = gr.Button("Clear Session Memory (EPIC 3.3)")
        session_mem_refresh_btn = gr.Button("Refresh Memory Viewer (EPIC 3.3)")
        session_mem_view_md = gr.Markdown("_Session memory viewer will appear here._")

        def _format_session_mem_view(mem: dict) -> str:
            if not isinstance(mem, dict):
                return "_(no session memory state)_"
            enabled = mem.get('enabled', True)
            max_turns = int(mem.get('max_turns') or SESSION_MEM_MAX_TURNS)
            turns = mem.get('turns') or []
            if not enabled:
                return f"_(session memory OFF; turns cleared; max_turns={max_turns})_"
            if not turns:
                return f"_(session memory ON; empty; max_turns={max_turns})_"
            out = []
            out.append(f"**Turns stored:** {len(turns)}  •  **max_turns:** {max_turns}")
            out.append('')
            for i, t in enumerate(turns[-max_turns:], 1):
                q = (t.get('q') or '').strip()
                a = (t.get('a') or '').strip()
                if q:
                    out.append(f"**Q{i}:** {q}")
                if a:
                    out.append(f"**A{i}:** {a}")
                out.append('---')
            return '\n'.join(out).strip()

        def _toggle_session_memory(session_user, enabled: bool, mem: dict, max_turns_val):  # EPIC 3.3
            su, msg = _ensure_admin(session_user)
            if not su:
                mem = mem if isinstance(mem, dict) else init_session_memory_state()
                return mem, f"❌ {msg}", _format_session_mem_view(mem), gr.update(value=int(mem.get("max_turns") or SESSION_MEM_MAX_TURNS))
            mem = mem if isinstance(mem, dict) else init_session_memory_state()
            mem['enabled'] = bool(enabled)
            try:
                mem['max_turns'] = int(max_turns_val) if max_turns_val is not None else int(mem.get('max_turns') or SESSION_MEM_MAX_TURNS)
            except Exception:
                mem['max_turns'] = int(mem.get('max_turns') or SESSION_MEM_MAX_TURNS)
            if not mem['enabled']:
                mem['turns'] = []
            else:
                turns = mem.get('turns') or []
                if len(turns) > mem['max_turns']:
                    mem['turns'] = turns[-mem['max_turns']:]
            state_txt = '✅ Session memory is **ON**.' if mem['enabled'] else '🛑 Session memory is **OFF** (cleared).'
            return mem, state_txt, _format_session_mem_view(mem), gr.update(value=int(mem.get('max_turns') or SESSION_MEM_MAX_TURNS))

        def _clear_session_memory(session_user, mem: dict):
            su, msg = _ensure_admin(session_user)
            if not su:
                mem = mem if isinstance(mem, dict) else init_session_memory_state()
                return mem, f"❌ {msg}", _format_session_mem_view(mem)
            mem = mem if isinstance(mem, dict) else init_session_memory_state()
            mem['turns'] = []
            return mem, '🧹 Session memory cleared.', _format_session_mem_view(mem)

        def _set_session_mem_max_turns(session_user, max_turns_val, mem: dict):
            su, msg = _ensure_admin(session_user)
            if not su:
                mem = mem if isinstance(mem, dict) else init_session_memory_state()
                return mem, f"❌ {msg}", _format_session_mem_view(mem), gr.update(value=int(mem.get("max_turns") or SESSION_MEM_MAX_TURNS))
            mem = mem if isinstance(mem, dict) else init_session_memory_state()
            try:
                mem['max_turns'] = int(max_turns_val)
            except Exception:
                mem['max_turns'] = int(mem.get('max_turns') or SESSION_MEM_MAX_TURNS)
            turns = mem.get('turns') or []
            if len(turns) > mem['max_turns']:
                mem['turns'] = turns[-mem['max_turns']:]
            return mem, '✅ Updated max_turns.', _format_session_mem_view(mem), gr.update(value=int(mem['max_turns']))

        def _set_session_mem_scope(session_user, scope_val, mem: dict):
            """EPIC 3.3 — Admin can set memory scope (cohort vs session)."""
            su, msg = _ensure_admin(session_user)
            if not su:
                mem = mem if isinstance(mem, dict) else init_session_memory_state()
                return mem, f"❌ {msg}", _format_session_mem_view(mem), gr.update(value=(mem.get("scope") or "cohort"))
            mem = mem if isinstance(mem, dict) else init_session_memory_state()
            scope_val = (scope_val or "cohort").lower()
            if scope_val not in ("cohort", "session"):
                scope_val = "cohort"
            mem["scope"] = scope_val
            status = f"✅ Session memory scope set to: {scope_val.upper()}."
            return mem, status, _format_session_mem_view(mem), gr.update(value=scope_val)

        def _refresh_session_mem_view(mem: dict):
            mem = mem if isinstance(mem, dict) else init_session_memory_state()
            return _format_session_mem_view(mem)

        session_mem_enabled_cb.change(
            _toggle_session_memory,
            inputs=[session_state, session_mem_enabled_cb, session_memory_state, session_mem_max_turns],
            outputs=[session_memory_state, session_mem_status_md, session_mem_view_md, session_mem_max_turns],
            queue=False,
            trigger_mode='once',
        )

        session_mem_clear_btn.click(
            _clear_session_memory,
            inputs=[session_state, session_memory_state],
            outputs=[session_memory_state, session_mem_status_md, session_mem_view_md],
            queue=False,
        )

        session_mem_max_turns.change(
            _set_session_mem_max_turns,
            inputs=[session_state, session_mem_max_turns, session_memory_state],
            outputs=[session_memory_state, session_mem_status_md, session_mem_view_md, session_mem_max_turns],
            queue=False,
            trigger_mode='once',
        )
        # EPIC 3.3 — Scope toggle (admin)
        session_mem_scope_radio.change(
            _set_session_mem_scope,
            inputs=[session_state, session_mem_scope_radio, session_memory_state],
            outputs=[session_memory_state, session_mem_status_md, session_mem_view_md, session_mem_scope_radio],
            queue=False,
            trigger_mode='once',
        )

        session_mem_refresh_btn.click(
            _refresh_session_mem_view,
            inputs=[session_memory_state],
            outputs=[session_mem_view_md],
            queue=False,
        )

        with gr.Row():
            refresh_stats_btn = gr.Button("DB Stats")
            refresh_users_btn = gr.Button("List Users")
            refresh_cohorts_btn = gr.Button("List Cohorts")
            refresh_audit_btn = gr.Button("Audit Log (last 50)")

        admin_stats_md = gr.Markdown()
        admin_users_md = gr.Markdown()
        admin_cohorts_md = gr.Markdown()
        admin_audit_md = gr.Markdown()

        gr.Markdown("#### Delete Cohort (admin only)")
        admin_delete_cohort_dropdown = gr.Dropdown(
            label="Select Cohort to Delete",
            choices=[],
        )
        admin_delete_btn = gr.Button("Delete Selected Cohort")
        admin_delete_status_md = gr.Markdown()
        # ============================================================
                # ============================================================
        # EPIC 5.2 — Admin Chat-Model Controls (enabled + default)
        # ============================================================
        gr.Markdown("### LLM Model Controls (EPIC 5.2)")
        gr.Markdown("Admins can enable/disable chat models and select the default model used on the Ask tab.")

        # Use registry if available; fall back to hardcoded list.
        try:
            all_models = list_all_chat_models()
        except Exception:
            all_models = ["gpt-4.1-mini", "gpt-5", "gpt-5-mini", ""]

        try:
            enabled_models = get_allowed_chat_models()
        except Exception:
            enabled_models = list(all_models)

        try:
            default_model_id = get_default_chat_model_id()
        except Exception:
            default_model_id = all_models[0] if all_models else "gpt-4.1-mini"



        # ---- Sanitize model list (fix blank/phantom checkbox issue) ----
        # Remove empty/whitespace IDs and de-duplicate while preserving order.
        all_models = [m for m in (all_models or []) if isinstance(m, str) and m.strip()]
        all_models = list(dict.fromkeys(all_models))

        with gr.Row():
            admin_chat_models_cbg = gr.CheckboxGroup(
                label="Enabled chat models",
                choices=all_models,
                value=[m for m in (enabled_models or []) if m in all_models],
                interactive=True,
            )

        with gr.Row():
            admin_default_chat_model_dd = gr.Dropdown(
                label="Default chat model",
                choices=[m for m in all_models if m in ((enabled_models or []) or all_models)],
                value=(default_model_id if default_model_id in all_models else (all_models[0] if all_models else None)),
                interactive=True,
            )
            admin_models_save_btn = gr.Button("Save Model Settings", variant="primary")

        admin_models_status_md = gr.Markdown("")

        def _admin_save_model_settings(session_user, enabled_list, default_id):
            su, msg = _ensure_admin(session_user)
            if not su:
                return f"❌ {msg}"

            enabled_list = enabled_list or []
            # Ensure default is enabled
            if default_id and default_id not in enabled_list:
                enabled_list = list(enabled_list) + [default_id]

            try:
                ensure_model_registry_table()
            except Exception as e:
                trace_log(f"ADMIN model settings ensure table ERROR: {e}")

            # Persist enable flags
            for mid in all_models:
                try:
                    add_or_update_model(
                        model_type="chat",
                        model_id=mid,
                        enabled=(mid in enabled_list),
                        is_default=False,
                    )
                except Exception as e:
                    trace_log(f"ADMIN add_or_update_model ERROR model={mid}: {e}")

            # Persist default
            try:
                if default_id:
                    set_default_model("chat", default_id)
            except Exception as e:
                trace_log(f"ADMIN set_default_model ERROR: {e}")

            try:
                log_audit(su.username, su.role, "set_chat_models", f"enabled={enabled_list} default={default_id}")
            except Exception:
                pass

            # Return updated UI state
            new_enabled = [m for m in enabled_list if m in all_models]
            new_default_choices = [m for m in all_models if m in new_enabled] or list(all_models)
            new_default = default_id if (default_id in new_default_choices) else (new_default_choices[0] if new_default_choices else None)

            return (
                "✅ Model settings saved.",
                gr.update(value=new_enabled),
                gr.update(choices=new_default_choices, value=new_default),
            )

        admin_models_save_btn.click(
            _admin_save_model_settings,
            inputs=[session_state, admin_chat_models_cbg, admin_default_chat_model_dd],
            outputs=[admin_models_status_md, admin_chat_models_cbg, admin_default_chat_model_dd],
            queue=False,
        )

# --- Demo Setup (v16.1) ---
        gr.Markdown("### Demo Setup (admin)")
        gr.Markdown("Upload a prompt list to preload curated 'FAPs' for a cohort. These appear in the Ask tab as a dropdown when that cohort is selected.")

        with gr.Row():
            demo_cohort_dropdown = gr.Dropdown(label="Cohort for FAPs", choices=[], value=None)
            demo_refresh_cohorts_btn = gr.Button("Refresh Cohorts")

        with gr.Row():
            demo_prompts_file = gr.File(
                label="Upload FAPs File (.txt, .csv, .json)",
                file_count="single",
                type="filepath",
            )
            demo_replace_checkbox = gr.Checkbox(label="Replace existing FAPs", value=True)

        with gr.Row():
            demo_load_btn = gr.Button("Load FAPs", variant="primary")
            demo_view_btn = gr.Button("View FAPs")
            demo_clear_btn = gr.Button("Clear FAPs")

        demo_prompts_status = gr.Markdown("")
        demo_prompts_preview = gr.Markdown("")
        # --- Demo Prompt Notes (v17.0 / Epic 1.2) ---
        gr.Markdown("#### Demo Prompt Notes (admin only)")

        with gr.Row():
            demo_prompt_select_dd = gr.Dropdown(
                label="Select a Demo Prompt to add notes",
                choices=[],
                value=None,
                interactive=True,
            )

        demo_prompt_note_tb = gr.Textbox(
            label="Admin Note (Presenter Only)",
            lines=4,
            placeholder="Only visible to admins in Demo Setup. Not shown in Ask tab.",
        )

        with gr.Row():
            demo_prompt_note_save_btn = gr.Button("Save Note", variant="primary")
            demo_prompt_note_refresh_btn = gr.Button("Refresh Prompt List")



        def _demo_refresh_cohorts_dd(session_user):
            su, msg = _ensure_admin(session_user)
            if not su:
                return gr.update(choices=[], value=None)
            try:
                rows = get_all_cohorts()
                names = [r[0] for r in rows if r and r[0]]
                return gr.update(choices=names, value=None)
            except Exception as e:
                trace_log(f"ADMIN demo _demo_refresh_cohorts_dd ERROR: {e}")
                return gr.update(choices=[], value=None)

        def _demo_load_prompts_cb(session_user, cohort_name: str, file_path: str, replace: bool):
            su, msg = _ensure_admin(session_user)
            if not su:
                return msg or "❌ Admin privileges required.", ""
            c = (cohort_name or "").strip()
            if not c:
                return "❌ Please select a cohort.", ""
            if not file_path:
                return "❌ Please upload a .txt/.csv/.json prompt file.", ""
            prompts = parse_demo_prompts_file(file_path)
            n = upsert_demo_prompts_for_cohort(c, prompts, replace=bool(replace))
            try:
                log_audit(su.username, su.role, "demo_prompts_load", f"cohort={c} count={n} replace={bool(replace)}")
            except Exception as e:
                trace_log(f"log_audit demo_prompts_load ERROR: {e}")
            preview = list_demo_prompts_for_cohort(c)
            preview_md = "\n".join([f"- {p}" for p in preview[:50]]) if preview else "_No FAPs found._"
            return f"✅ Loaded {n} demo prompt(s) into cohort `{c}`.", preview_md

        def _demo_view_prompts_cb(session_user, cohort_name: str):
            su, msg = _ensure_admin(session_user)
            if not su:
                return msg or "❌ Admin privileges required.", ""
            c = (cohort_name or "").strip()
            if not c:
                return "❌ Please select a cohort.", ""
            preview = list_demo_prompts_for_cohort(c)
            if not preview:
                return f"_No FAPs found for `{c}`._", ""
            md = "\n".join([f"- {p}" for p in preview[:200]])
            return f"**Demo prompts for `{c}` ({len(preview)}):**", md

        def _demo_clear_prompts_cb(session_user, cohort_name: str):
            su, msg = _ensure_admin(session_user)
            if not su:
                return msg or "❌ Admin privileges required.", ""
            c = (cohort_name or "").strip()
            if not c:
                return "❌ Please select a cohort.", ""
            clear_demo_prompts_for_cohort(c)
            try:
                log_audit(su.username, su.role, "demo_prompts_clear", f"cohort={c}")
            except Exception as e:
                trace_log(f"log_audit demo_prompts_clear ERROR: {e}")
            return f"✅ Cleared FAPs for `{c}`.", ""

        def _demo_list_prompts_for_notes(session_user, cohort_name: str):
          su, msg = _ensure_admin(session_user)
          if not su:
              return gr.update(choices=[], value=None)
          c = (cohort_name or "").strip()
          if not c:
              return gr.update(choices=[], value=None)

          rows = list_demo_prompts_with_notes(c)  # returns [(id, prompt_text, admin_note), ...]
          # Dropdown values must be stable; use the ID as the value.
          choices = [(r[1], str(r[0])) for r in rows]  # (label, value)
          return gr.update(choices=choices, value=None)


        def _demo_load_note_into_box(session_user, cohort_name: str, prompt_id: str):
            su, msg = _ensure_admin(session_user)
            if not su:
                return ""
            c = (cohort_name or "").strip()
            if not c or not prompt_id:
                return ""

            pid = int(prompt_id)
            rows = list_demo_prompts_with_notes(c)
            for (rid, text, note) in rows:
                if rid == pid:
                    return note or ""
            return ""


        def _demo_save_note_cb(session_user, cohort_name: str, prompt_id: str, note_text: str):
            su, msg = _ensure_admin(session_user)
            if not su:
                return msg or "❌ Admin privileges required."

            c = (cohort_name or "").strip()
            if not c:
                return "❌ Please select a cohort."
            if not prompt_id:
                return "❌ Please select a demo prompt."

            try:
                save_demo_prompt_note(int(prompt_id), note_text or "")
                log_audit(su.username, su.role, "demo_prompt_note_save", f"cohort={c} prompt_id={prompt_id}")
                return "✅ Admin note saved."
            except Exception as e:
                trace_log(f"ADMIN demo note save ERROR: {e}")
                return f"❌ Error saving note: {e}"


        demo_refresh_cohorts_btn.click(
            _demo_refresh_cohorts_dd,
            inputs=[session_state],
            outputs=[demo_cohort_dropdown],
        )

        demo_load_btn.click(
            _demo_load_prompts_cb,
            inputs=[session_state, demo_cohort_dropdown, demo_prompts_file, demo_replace_checkbox],
            outputs=[demo_prompts_status, demo_prompts_preview],
        )


        demo_view_btn.click(
            _demo_view_prompts_cb,
            inputs=[session_state, demo_cohort_dropdown],
            outputs=[demo_prompts_status, demo_prompts_preview],
        )

        demo_clear_btn.click(
            _demo_clear_prompts_cb,
            inputs=[session_state, demo_cohort_dropdown],
            outputs=[demo_prompts_status, demo_prompts_preview],
        )

        # Refresh prompt list for selected cohort (for notes dropdown)
        demo_prompt_note_refresh_btn.click(
            _demo_list_prompts_for_notes,
            inputs=[session_state, demo_cohort_dropdown],
            outputs=[demo_prompt_select_dd],
        )

        # Also refresh prompt list when cohort changes
        demo_cohort_dropdown.change(
            _demo_list_prompts_for_notes,
            inputs=[session_state, demo_cohort_dropdown],
            outputs=[demo_prompt_select_dd],
        )

        # When an admin selects a prompt, load its note into the textbox
        demo_prompt_select_dd.change(
            _demo_load_note_into_box,
            inputs=[session_state, demo_cohort_dropdown, demo_prompt_select_dd],
            outputs=[demo_prompt_note_tb],
        )

        # Save note
        demo_prompt_note_save_btn.click(
            _demo_save_note_cb,
            inputs=[session_state, demo_cohort_dropdown, demo_prompt_select_dd, demo_prompt_note_tb],
            outputs=[demo_prompts_status],
        )

        return {
            "admin_stats_md": admin_stats_md,
            "admin_users_md": admin_users_md,
            "admin_cohorts_md": admin_cohorts_md,
            "admin_audit_md": admin_audit_md,
            "admin_delete_cohort_dropdown": admin_delete_cohort_dropdown,
            "admin_delete_status_md": admin_delete_status_md,

            "demo_cohort_dropdown": demo_cohort_dropdown,

            # Epic 3.2 – Session Memory
            "session_mem_enabled_cb": session_mem_enabled_cb,
            "session_mem_status_md": session_mem_status_md,
            "session_mem_max_turns": session_mem_max_turns,  # EPIC 3.3
            "session_mem_clear_btn": session_mem_clear_btn,
            "session_mem_refresh_btn": session_mem_refresh_btn,
            "session_mem_view_md": session_mem_view_md,

        }



## STEP 14 — Interface assembly & launch


In [30]:
ACTION_CREATE = "Create new cohort from files"
ACTION_APPEND = "Append files to existing cohort"
ACTION_CLONE  = "Clone an existing cohort"

DEFAULT_ACTION = ACTION_CREATE


# -----------------------------
# Build Gradio Interface
# -----------------------------
def build_interface():
    with gr.Blocks(title="Phase1 RAG MVP v18 EPIC5.1") as demo:
        gr.Markdown("## Phase1 RAG MVP v18 EPIC5.1")

        # Global session state
        session_state = gr.State(value=None)       # {"username": ..., "role": ...} or None
        role_state = gr.State(value="anonymous")   # Optional, not heavily used yet
        api_key_state = gr.State(value="")         # Hidden backing store for API key

        session_memory_state = gr.State(value=init_session_memory_state())

        # ---- Login / Logout row ----
        with gr.Row():
            username_tb = gr.Textbox(label="Username", scale=1)
            password_tb = gr.Textbox(label="Password", type="password", scale=1)
            api_key_tb = gr.Textbox(
                label="OpenAI API Key",
                type="password",
                placeholder="sk-...",
                scale=2,
            )

        with gr.Row():
            login_btn = gr.Button("Login")
            logout_btn = gr.Button("Logout")

        login_status = gr.Markdown("Not logged in.")
        current_user_label = gr.Markdown("**Current user:** (none)")

        # ---- Tabs ----
        with gr.Tabs():
            with gr.Tab("Setup & Cohorts"):
                setup_handles = build_setup_tab(session_state, api_key_state)
            with gr.Tab("About"):
                gr.Markdown(ABOUT_TOOL_MD)

            with gr.Tab("Ask"):
                ask_handles = build_ask_tab(session_state, api_key_state, session_memory_state)
            with gr.Tab("History"):
                history_handles = build_history_tab(session_state)
            # Admin tab: hidden by default
            admin_tab = gr.Tab("Admin", visible=False)
            with admin_tab:
                admin_handles = build_admin_tab(session_state, session_memory_state)

        # Auto-refresh Admin dropdowns when the Admin tab is selected (v18 fix)
        def _admin_tab_select_refresh(session_user):
            # Populate Admin dropdowns on tab select.
            # session_user is the session_state dict: {"username": ..., "role": ...}
            if not session_user or session_user.get("role") != "admin":
                return gr.update(choices=[], value=None), gr.update(choices=[], value=None)
            try:
                names = [row[0] for row in get_all_cohorts()]
            except Exception:
                names = []
            return gr.update(choices=names, value=None), gr.update(choices=names, value=None)

        admin_tab.select(
            _admin_tab_select_refresh,
            inputs=[session_state],
            outputs=[
                admin_handles["admin_delete_cohort_dropdown"],
                admin_handles["demo_cohort_dropdown"],
            ],
            queue=False,
        )
        # ---- Ask tab enable / disable JS (Fix Set 1) ----
        ASK_DISABLE_JS = """
        (session_state, role_state, api_key_state, session_memory_state) => {
          const setDisabled = (containerId, disabled) => {
            const container = document.getElementById(containerId);
            if (!container) return;
            const el = container.querySelector("textarea, input, button, select");
            if (!el) return;
            el.disabled = disabled;
          };

          setDisabled("ask_cohort_dropdown", true);
          setDisabled("ask_model_dropdown", true);
          setDisabled("ask_question_box", true);
          setDisabled("ask_improved_box", true);
          setDisabled("ask_improve_btn", true);
          setDisabled("ask_ask_btn", true);
          setDisabled("ask_refresh_btn", true);
              setDisabled("ask_refresh_models_btn", true);

          // CRITICAL: return inputs unchanged
          return [session_state, role_state, api_key_state, session_memory_state];
        }
        """

        ASK_ENABLE_JS = """
        (username, password, api_key, session_state, role_state, api_key_state, session_memory_state) => {
          const setDisabled = (containerId, disabled) => {
            const container = document.getElementById(containerId);
            if (!container) return;
            const el = container.querySelector("textarea, input, button, select");
            if (!el) return;
            el.disabled = disabled;
          };

          setDisabled("ask_cohort_dropdown", false);
          setDisabled("ask_model_dropdown", false);
          setDisabled("ask_question_box", false);
          setDisabled("ask_improved_box", false);
          setDisabled("ask_improve_btn", false);
          setDisabled("ask_ask_btn", false);
          setDisabled("ask_refresh_btn", false);
              setDisabled("ask_refresh_models_btn", false);

          // CRITICAL: return inputs unchanged so Python receives them
          return [username, password, api_key, session_state, role_state, api_key_state, session_memory_state];
        }
        """



        # ---- Login / Logout Callbacks ----
        # ---- Login / Logout Callbacks ----

        def _login_cb(username, password, api_key, current_session, current_role, current_api_key, session_mem):
            """
            Authenticate user and store session + API key in state.
            Also auto-populates cohort dropdowns for this user.

            POLICY:
              - API key is stored only in api_key_state (memory), never left in a textbox.
              - On ANY login attempt (success or failure), clear:
                  Username, Password, API Key textbox, Question, Improved Prompt
            """
            # print("LOGIN RAW:", repr(username), repr(password), repr(api_key))

            #print("DEBUG: login clicked")

            # Normalize inputs
            username = (username or "").strip()
            password = password or ""
            api_key = (api_key or "").strip()

            # Always-clear values for UI textboxes (return plain strings for reliability)
            CLEAR = ""
            clear_username = CLEAR
            clear_password = CLEAR
            clear_api_key_tb = CLEAR
            clear_question = CLEAR
            clear_improved = CLEAR
            reset_session_mem = init_session_memory_state()

            # Basic validation
            if not username or not password:
                return (
                    "❌ Username and password are required.",  # login_status
                    None,                                     # session_state
                    "anonymous",                               # role_state
                    "",                                       # api_key_state
                    clear_username,                            # username_tb
                    clear_password,                            # password_tb
                    clear_api_key_tb,                          # api_key_tb
                    "**Current user:** (none)",                # current_user_label
                    gr.update(choices=[], value=None),         # setup existing cohort dropdown
                    gr.update(choices=[], value=None),         # ask cohort dropdown
                        gr.update(choices=list_chat_models(), value=get_default_chat_model_id()),         # ask model dropdown
                    clear_question,                            # ask question_box
                    clear_improved,                            # ask improved_box
                    reset_session_mem,
                )

                            # EPIC 3.3.2 — Guard against unknown usernames to avoid downstream errors
            if username and username not in USERS:
                return (
                    f"❌ Unknown user `{username}`. Use `admin` or `demo1`.",  # login_status
                    None,                                     # session_state
                    "anonymous",                               # role_state
                    "",                                       # api_key_state
                    clear_username,                            # username_tb
                    clear_password,                            # password_tb
                    clear_api_key_tb,                          # api_key_tb
                    "**Current user:** (none)",                # current_user_label
                    gr.update(choices=[], value=None),         # setup existing cohort dropdown
                    gr.update(choices=[], value=None),         # ask cohort dropdown
                        gr.update(choices=list_chat_models(), value=get_default_chat_model_id()),         # ask model dropdown
                    clear_question,                            # ask question_box
                    clear_improved,                            # ask improved_box
                    gr.update(visible=False),                  # admin_tab
                    reset_session_mem,                         # session_memory_state
                )

            # API key is required and must be valid BEFORE allowing login.
            if not api_key or not str(api_key).strip():
                return (
                    "❌ A valid OpenAI API key is required to log in.",  # login_status
                    None,                                              # session_state
                    "anonymous",                                        # role_state
                    "",                                                # api_key_state
                    clear_username,                                     # username_tb
                    clear_password,                                     # password_tb
                    clear_api_key_tb,                                   # api_key_tb
                    "**Current user:** (none)",                         # current_user_label
                    gr.update(choices=[], value=None),                  # setup existing cohort dropdown
                    gr.update(choices=[], value=None),                  # ask cohort dropdown
                        gr.update(choices=list_chat_models(), value=get_default_chat_model_id()),         # ask model dropdown
                    clear_question,                                     # ask question_box
                    clear_improved,                                     # ask improved_box
                    gr.update(visible=False),                           # admin_tab
                    reset_session_mem,                                  # session_memory_state
                )

            # Validate key using a lightweight chat+embedding probe.
            try:
                _chat_model_for_validation = select_chat_model(
                    task_type="rag_answer",
                    context_size=0,
                    user_pref=None,
                )
                _embed_model_for_validation = DEFAULT_EMBED_MODEL
                _key_status = validate_openai_key_and_models(
                    api_key=api_key.strip(),
                    chat_model=_chat_model_for_validation,
                    embed_model=_embed_model_for_validation,
                )
            except Exception as e:
                _key_status = f"❌ Error validating key: {e}"

            if not str(_key_status).lstrip().startswith("✅"):
                return (
                    f"❌ Invalid OpenAI API key. {_key_status}",         # login_status
                    None,                                               # session_state
                    "anonymous",                                         # role_state
                    "",                                                 # api_key_state
                    clear_username,                                      # username_tb
                    clear_password,                                      # password_tb
                    clear_api_key_tb,                                    # api_key_tb
                    "**Current user:** (none)",                          # current_user_label
                    gr.update(choices=[], value=None),                   # setup existing cohort dropdown
                    gr.update(choices=[], value=None),                   # ask cohort dropdown
                        gr.update(choices=list_chat_models(), value=get_default_chat_model_id()),         # ask model dropdown
                    clear_question,                                      # ask question_box
                    clear_improved,                                      # ask improved_box
                    gr.update(visible=False),                            # admin_tab
                    reset_session_mem,                                   # session_memory_state
                )

            user_dict = authenticate_credentials(username, password)
            if not user_dict:
                return (
                    "❌ Invalid username or password.",         # login_status
                    None,                                     # session_state
                    "anonymous",                               # role_state
                    "",                                       # api_key_state
                    clear_username,                            # username_tb
                    clear_password,                            # password_tb
                    clear_api_key_tb,                          # api_key_tb
                    "**Current user:** (none)",                # current_user_label
                    gr.update(choices=[], value=None),         # setup existing cohort dropdown
                    gr.update(choices=[], value=None),         # ask cohort dropdown
                        gr.update(choices=list_chat_models(), value=get_default_chat_model_id()),         # ask model dropdown
                    clear_question,                            # ask question_box
                    clear_improved,                            # ask improved_box
                    reset_session_mem,
                    gr.update(visible=False)   # admin_tab

                )

            role = user_dict.get("role", "user")
            status = f"✅ Logged in as `{username}` (role: `{role}`)"
            label = f"**Current user:** `{username}` (role: `{role}`)"

            # Store API key ONLY in state (memory); do not leave it in textbox
            new_api_key_state = api_key

            # Populate cohorts for this user (respects sharing rules)
            try:
                cohort_names = get_cohorts_for_user(username)
            except Exception as e:
                trace_log(f"LOGIN get_cohorts_for_user ERROR: {e}")
                try:
                    cohort_names = list_cohorts()
                except Exception:
                    cohort_names = []

            # Build "name/owner" labels for the Setup tab dropdown
            try:
                rows = get_all_cohorts()  # (cohort_name, owner_user_id, ...)
                owner_by_name = {row[0]: (row[1] if len(row) > 1 else None) for row in rows}
                setup_labels = []
                for name in cohort_names:
                    owner = owner_by_name.get(name)
                    setup_labels.append(f"{name}/{owner}" if owner else name)
            except Exception as e:
                trace_log(f"LOGIN building setup_labels ERROR: {e}")
                setup_labels = cohort_names

            setup_existing_dd = gr.update(choices=setup_labels, value=None)
            ask_cohort_dd = gr.update(choices=cohort_names, value=None)

            # Populate chat models from registry (EPIC 5.2)
            ask_model_dd = gr.update(choices=list_chat_models(), value=get_default_chat_model_id())

            return (
                status,                 # login_status
                user_dict,              # session_state
                role,                   # role_state
                new_api_key_state,      # api_key_state
                clear_username,         # username_tb (cleared after login)
                clear_password,         # password_tb (cleared after login)
                clear_api_key_tb,       # api_key_tb (cleared after login)
                label,                  # current_user_label
                setup_existing_dd,      # setup existing cohort dropdown
                ask_cohort_dd,          # ask cohort dropdown
                    ask_model_dd,          # ask model dropdown
                clear_question,         # ask question_box (cleared after login)
                clear_improved,# ask improved_box (cleared after login)
                gr.update(visible=(role == "admin")),  # NEW admin_tab visibility
                reset_session_mem,
            )

        login_btn.click(
            _login_cb,
            inputs=[
                username_tb,
                password_tb,
                api_key_tb,
                session_state,
                role_state,
                api_key_state,
                session_memory_state,  # EPIC 3.1
            ],
            queue=False,
            js=ASK_ENABLE_JS,

            outputs=[
                login_status,
                session_state,
                role_state,
                api_key_state,
                username_tb,
                password_tb,
                api_key_tb,
                current_user_label,
                setup_handles["existing_cohort_dropdown"],
                ask_handles["cohort_dropdown"],
                    ask_handles["model_dropdown"],
                ask_handles["question_box"],
                ask_handles["improved_box"],
                admin_tab,   # NEW
                session_memory_state,      # EPIC 3.1
            ],

        )


        def _logout_cb(_session, _role, _api_key, _session_mem):
            """Logout: clear UI + in-memory state (no persistence changes)."""
            # Reset in-memory state
            new_session = None
            new_role = "guest"
            new_api_key = ""
            reset_session_mem = init_session_memory_state()

            CLEAR = ""

            # Setup tab resets
            reset_action = gr.update(value=DEFAULT_ACTION)
            reset_new_name = CLEAR
            reset_share = gr.update(value=False)
            reset_allow_clone = gr.update(value=False)
            reset_existing_dd = gr.update(choices=[], value=None)
            reset_files = gr.update(value=None)
            reset_build_status = CLEAR

            # Ask tab resets
            reset_ask_cohort = gr.update(choices=[], value=None)
            reset_model_dd = gr.update(choices=list_chat_models(), value=get_default_chat_model_id())
            reset_question = CLEAR
            reset_improved = CLEAR
            reset_answer = CLEAR
            reset_docs = CLEAR

            # History resets
            reset_hist_user = gr.update(value="")
            reset_hist_cohort = gr.update(value="")
            reset_hist_md = CLEAR

            # Admin resets
            reset_admin_md = CLEAR
            reset_admin_dd = gr.update(choices=[], value=None)
            reset_admin_status = CLEAR

            return (
                "🔒 Logged out.",          # login_status
                new_session,              # session_state
                new_role,                 # role_state
                new_api_key,              # api_key_state
                CLEAR,                    # username_tb
                CLEAR,                    # password_tb
                CLEAR,                    # api_key_tb
                "**Current user:** (none)",

                # Setup & Cohorts tab
                reset_action,
                reset_new_name,
                reset_share,
                reset_allow_clone,
                reset_existing_dd,
                reset_files,
                reset_build_status,

                # Ask tab
                reset_ask_cohort,
                reset_model_dd,
                reset_question,
                reset_improved,
                reset_answer,
                reset_docs,

                # History tab
                reset_hist_user,
                reset_hist_cohort,
                reset_hist_md,

                # Admin tab panes + delete dropdown
                reset_admin_md,
                reset_admin_md,
                reset_admin_md,
                reset_admin_md,
                reset_admin_dd,
                reset_admin_status,

                gr.update(visible=False),  # admin_tab
                reset_session_mem,         # session_memory_state
            )

        logout_btn.click(
            _logout_cb,
            inputs=[session_state, role_state, api_key_state, session_memory_state],
            outputs=[
                login_status,
                session_state,
                role_state,
                api_key_state,
                username_tb,
                password_tb,
                api_key_tb,
                current_user_label,

                # Setup & Cohorts tab
                setup_handles["action_radio"],
                setup_handles["new_cohort_name"],
                setup_handles["share_checkbox"],
                setup_handles["allow_clone_checkbox"],
                setup_handles["existing_cohort_dropdown"],
                setup_handles["file_uploader"],
                setup_handles["build_status"],

                # Ask tab
                ask_handles["cohort_dropdown"],
                ask_handles["model_dropdown"],
                ask_handles["question_box"],
                ask_handles["improved_box"],
                ask_handles["answer_markdown"],
                ask_handles["docs_markdown"],

                # History tab
                history_handles["history_user_filter"],
                history_handles["history_cohort_filter"],
                history_handles["history_markdown"],

                # Admin tab
                admin_handles["admin_stats_md"],
                admin_handles["admin_users_md"],
                admin_handles["admin_cohorts_md"],
                admin_handles["admin_audit_md"],
                admin_handles["admin_delete_cohort_dropdown"],
                admin_handles["admin_delete_status_md"],
                admin_tab,
                session_memory_state,
            ],
            queue=False,
            js=ASK_DISABLE_JS,
        )

    return demo

# ---- Create & Launch the App ----
if __name__ == "__main__":
    demo = build_interface()
    # In Colab: share=False is usually fine; set to True if you want a public link.
    demo.launch(share=False, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
